# 1. Introduction

The purpose of this notebook is to perform initial cleaning of the Dublin Bus historical data, to merge the bus data with historical weather data, and to create new features based on the data.

# 2. Setup & Data Load

Import required modules and packages.

In [ ]:
# import time so that various tasks can be timed
import time

# import math for mathematical functions
import math

# import pandas and numpy for data analysis
import pandas as pd
import numpy as np

# import transform_data function for transforming data into segment format
from transform_data import transform_data

# import convert_timestamp for various timestamp conversion functions
import convert_timestamp

# import matplotlib & seaborn for plotting and visualisaion
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

Set the max number of columns & rows to display.

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

Load the bus data from file.

In [ ]:
start = time.time()
df = pd.read_csv('/data_analytics/data/august_bus_data.csv', sep=";", na_values=['\\N'])
end = time.time()
print(end - start)

Perform a check to see how many rows and columns are in the file.

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("Before any data cleaning, the dataframe contains", rows, "rows and", cols, "columns.")
print()

# 3. Initial Checks on the Bus Data

## 3.1 Check for Duplicate Rows & Columns

In [ ]:
print()
print('Duplicate rows:', df.duplicated()[df.duplicated() == True].shape[0])
print('Duplicate columns:',df.columns.size - df.columns.unique().size)

There are no duplicate rows or columns in the bus data.

## 3.2 Check for Null Features

In [ ]:
df.describe().T

There are no null features in the bus data.

## 3.3 Assign Features as Continuous or Categorical

Assign categorical and continous features, and update the type of all categorical features to 'category'.

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', 'progrnumber', 'stoppointid']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

## 3.4 Check for Constant Categorical Features

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

There are no constant features to be droppped.

## 3.5 Check for Constant Continuous Features

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are no constant features to be droppped.

# 4. Checks for Missing Data

## 4.1 Categorical Features

Print details for the categorical columns:

In [ ]:
df[categorical_columns].describe().T

There is a full count for all categorical features.

## 4.2 Continuous Features

Print details for the continuous columns:

In [ ]:
df[continuous_columns].describe().T

There is a full count for all continuous features.

# 5. Drop Legacy Bus Stops

We will remove data for any bus stops that don't appear in the latest GTFS data:

In [ ]:
legacy_stops = (124,128,132,139,140,347,349,401,403,430,431,432,433,434,462,464,465,466,467,468,486,661,662,663,674,765,788,805,806,815,816,863,864,865,866,867,869,870,873,874,935,936,951,952,953,963,964,965,966,967,968,973,974,975,976,979,988,989,990,991,997,998,999,1000,1001,1002,1003,1004,1006,1007,1008,1009,1010,1012,1039,1040,1041,1050,1131,1132,1133,1134,1138,1139,1252,1364,1391,1392,1394,1395,1400,1421,1513,1530,1545,1601,1602,1603,1605,1644,1645,1646,1650,1651,1652,1766,1824,1825,1830,1831,1856,1857,1880,1881,2027,2028,2029,2030,2037,2038,2047,2048,2049,2050,2052,2079,2110,2111,2112,2113,2116,2117,2130,2199,2212,2226,2228,2237,2290,2294,2345,2363,2364,2449,2451,2464,2520,2521,2533,2534,2544,2545,2576,2618,2619,2620,2762,2763,2780,2781,2782,2783,2784,2785,2786,2787,2788,2789,2790,2791,2793,2794,2796,2797,2798,2799,2800,2801,2806,2807,2808,2872,2873,2874,2877,2878,2879,2880,2882,2883,2885,2886,2887,2960,3046,3058,3078,3083,3087,3094,3095,3096,3097,3098,3099,3100,3101,3102,3103,3105,3106,3107,3108,3111,3165,3166,3167,3168,3169,3171,3173,3174,3175,3176,3177,3178,3179,3182,3183,3187,3188,3189,3190,3191,3192,3193,3194,3195,3198,3199,3200,3201,3241,3242,3248,3249,3254,3255,3256,3260,3261,3262,3265,3266,3267,3268,3269,3270,3271,3272,3273,3274,3275,3276,3277,3278,3279,3285,3289,3290,3291,3292,3293,3294,3295,3296,3297,3298,3306,3307,3309,3312,3343,3346,3353,3354,3357,3358,3360,3361,3362,3442,3534,3545,3546,3547,3548,3551,3560,3561,3586,3739,4095,4097,4103,4104,4105,4106,4108,4109,4110,4111,4112,4113,4137,4138,4139,4140,4142,4143,4144,4145,4146,4147,4167,4169,4176,4183,4184,4185,4186,4187,4188,4189,4190,4192,4194,4196,4199,4232,4234,4235,4236,4237,4238,4239,4240,4241,4242,4243,4244,4289,4290,4291,4292,4293,4294,4313,4326,4327,4332,4337,4341,4357,4359,4360,4361,4365,4366,4367,4368,4375,4381,4389,4390,4391,4393,4395,4396,4414,4417,4419,4420,4421,4422,4457,4473,4474,4503,4527,4529,4582,4584,4586,4587,4597,4637,4650,4651,4652,4653,4654,4655,4656,4657,4658,4659,4660,4661,4678,4686,4687,4771,4772,4773,4781,4784,4785,4790,4791,4792,4820,4826,4849,4983,5006,5007,5009,5011,5034,5055,5088,5089,5105,5106,5138,5141,5167,5168,5190,6000,6009,6019,6039,6058,6073,6082,6103,6126,6127,6128,6133,6134,6138,6139,6179,6180,6196,6197,6198,6199,6200,6201,6207,6333,7026,7129,7155,7164,7165,7205,7220,7269,7279,7295,7297,7298,7323,7324,7351,7360,7369,7372,7373,7393,7394,7404,7405,7407,7447,7448,7457,7478,7479,7480,7481,7483,7484,7485,7486,7487,7490,7492,7495,7497,7500,7501,7502,7503,7504,7522,7529,7537,7538,7539,7540,7541,7542,7544,7546,7547,7553,7554,7558,7566,7570,7575,7576,7580,7589,7593,7594,7599,7604,7607,7617,7620,7621,7626,7627,7634,7635,7636,7637,7638,7647,7648,7651,7652,7653,7655,7658,7659,7662,7666,7668,7669,7682,7685,7688,7689)

In [ ]:
df = df.loc[~df['stoppointid'].isin(legacy_stops)]

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After removing legacy stops, the dataframe contains", rows, "rows and", cols, "columns.")
print()

# 6. Transform the Bus Data

Bus data must be transformed so that each row of data holds information on one journey segment. This is done by calling the *transform_data* function:

In [ ]:
start = time.time()
df = transform_data(df)
end = time.time()
print(end - start)

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After transforming the data, the dataframe contains", rows, "rows and", cols, "columns.")
print()

## 6.1 Check for Missing Data

First re-assign the transformed data as continuous or categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid',  \
                         'progrnumber_first', 'stoppointid_first', \
                          'progrnumber_next', 'stoppointid_next']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

Then check for missing data:

In [ ]:
# Print details for the categorical columns
df[categorical_columns].describe().T

In [ ]:
# Print details for the continuous columns
df[continuous_columns].describe().T

There are no rows with missing data.

## 6.2 Export the Data

The transformed data is exported at this point so that it can be reloaded easily without having to re-run the whole notebook:

In [ ]:
start = time.time()
df.to_hdf('/data_analytics/data/august_bus_data_transformed.hdf', key='transformed', format='table')
end = time.time()
print(end - start)

Import the data when required:

In [ ]:
df = pd.read_hdf('/data_analytics/data/august_bus_data_transformed.hdf')

# 7. Import Weather Data

The cleaned weather data is loaded from a csv:

In [ ]:
df_weather = pd.read_csv('/data_analytics/data/weather_cleaned.csv', sep=",", na_values=['\\N'])

Assign features as continuous or categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns_weather = df_weather[['record_date']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns_weather:
    df_weather[column] = df_weather[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns_weather = df_weather.select_dtypes(['float64', 'int64']).columns

# 8. Combine Bus and Weather Data

## 8.1 Split Timestamp for Weather Data

To merge the data, timestamps must be split into month, day and hour.

New features are added:

In [ ]:
df_weather['month'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_month_weather(x))

In [ ]:
df_weather['day'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_day_weather(x))

In [ ]:
df_weather['hour'] = df_weather['record_date'].map(lambda x: convert_timestamp.timestamp_to_hour_weather(x))

New features are updated to be categorical:

In [ ]:
# Select columns containing categorical data
categorical_columns_weather = df_weather[['record_date', 'month', 'day', 'hour']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns_weather:
    df_weather[column] = df_weather[column].astype('category')

## 8.2 Split Timestamp for Bus Data

New features are added:

In [ ]:
start = time.time()
df['month'] = df.apply (lambda row: convert_timestamp.timestamp_to_month_bus(row['dayofservice'], \
                                                                                   row['actualtime_arr_stop_first']), axis=1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
df['day'] = df.apply (lambda row: convert_timestamp.timestamp_to_day_bus(row['dayofservice'], \
                                                                               row['actualtime_arr_stop_first']), axis=1)
end = time.time()
print(end - start)

In [ ]:
df['hour'] = df['actualtime_arr_stop_first'].map(lambda x: convert_timestamp.timestamp_to_hour_bus(x))

## 8.3 Merge the Dataframes

In [ ]:
df = pd.merge(df, df_weather,  how='left', left_on=['month','day', 'hour'],\
                     right_on = ['month','day', 'hour'])

Check that there are no rows missing weather data:

In [ ]:
df[df.rain.isnull()]

Update the data types for the new dataframe:

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', \
                         'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next',\
                          'month', 'day', 'hour', 'record_date']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

# 9. Create New Features

## 9.1 Day of Week Feature

The new feature is added:

In [ ]:
df['day_of_week'] = df['dayofservice'].map(lambda x: convert_timestamp.timestamp_to_day_of_week(x))

## 9.2 Weekend/Weekday Binary Feature

The new feature is added:

In [ ]:
df['weekday'] = df['dayofservice'].map(lambda x: convert_timestamp.timestamp_to_weekday_weekend(x))

## 9.3 Bank Holiday Binary Feature

Make list of bank holidays for 2018 (based on https://www.officeholidays.com/countries/ireland/2018):

In [ ]:
holidays = ['2018-01-01 00:00:00', '2018-03-19 00:00:00', '2018-04-02 00:00:00', '2018-05-07 00:00:00', '2018-06-04 00:00:00',\
           '2018-08-06 00:00:00', '2018-10-29 00:00:00', '2018-12-25 00:00:00', '2018-12-26 00:00:00']

The new feature is added:

In [ ]:
df['bank_holiday'] = df['dayofservice'].map(lambda x: convert_timestamp.timestamp_to_bank_holiday(x, holidays))

## 9.4 Time Difference Feature

The new feature is added:

In [ ]:
df['time_diff'] = df['actualtime_arr_stop_next'] - df['actualtime_arr_stop_first']

Check for rows where the time difference is less than or equal to zero:

In [ ]:
df.loc[df['time_diff'] <= 0]

Drop the rows that fail this test:

In [ ]:
df = df.loc[df['time_diff'] > 0]

## 9.5 Segment Feature

The new feature is added:

In [ ]:
# based on https://stackoverflow.com/questions/11858472/string-concatenation-of-two-pandas-columns
df = df.assign(segment=(df['stoppointid_first'].astype(int)).astype(str).str.cat((df['stoppointid_next'].astype(int)).astype(str), sep='_'))

Drop any segments that don't appear in the GTFS data:

In [ ]:
# make a list of all segments from the GTFS data
gtfs_segments = ['10_12','100_1111','1005_1788','1013_1015','1014_1015','1015_1016','1015_1074','1015_2812','1016_1017','1017_1018','1018_1019','1019_1020','102_6239','1020_1076','1021_6042','1022_1023','1023_1024','1024_895','1025_1026','1026_1027','1027_1028','1028_2818','1035_1314','1036_4709','1037_1038','1038_2998','104_6241','1042_1043','1043_1044','1044_1045','1045_4710','1046_1047','1047_1048','1048_1049','1049_4333','105_6253','1051_1052','1052_1053','1053_1054','1054_1036','1055_2868','1062_1063','1063_1064','1064_1065','1065_1066','1066_1067','1067_1068','1068_1069','1069_1070','107_6125','1070_1071','1071_4528','1072_1352','1072_7577','1073_3641','1074_6074','1076_1021','1076_1077','1077_1078','1077_2479','1078_1079','1079_1080','1080_1081','1081_1082','1081_2914','1082_1083','1083_1085','1085_1086','1085_1121','1086_1087','1087_1088','1088_1089','1089_1090','1090_1091','1090_2413','1091_1092','1092_1093','1093_1094','1094_1095','1095_1096','1096_1097','1096_1101','1097_1099','1099_6132','11_14','110_112','1100_100','1101_1102','1102_1103','1103_1104','1105_1107','1107_1108','1108_1109','1109_1110','1110_1111','1111_1112','1111_2323','1111_2404','1112_1113','1113_1114','1114_1115','1115_2437','1117_1118','1118_1119','1119_1120','112_113','1120_1164','1121_1122','1122_1123','1123_1124','1124_1125','1125_1127','1127_1130','113_114','1130_1140','1130_2550','1135_7064','1137_5131','114_115','1140_1141','1140_2515','1141_1142','1142_1143','1143_1144','1144_1145','1145_4869','115_37','1150_1151','1151_1152','1152_1153','1153_1154','1154_946','1155_1157','1157_1158','1158_1159','1159_1160','116_117','1160_1161','1161_1162','1162_1163','1163_1164','1163_1337','1164_1165','1165_1166','1166_1167','1167_1168','1168_1169','1169_1170','117_118','1170_1069','1171_1172','1172_1173','1172_286','1172_4509','1172_7671','1173_1174','1174_1175','1174_4725','1175_15','1176_1177','1177_1178','1178_1179','1179_5060','1179_7112','118_119','1181_7350','1182_1550','1184_497','1185_1186','1186_1187','1187_1188','1188_216','119_44','1190_1191','1191_1192','1192_1193','1193_1194','1194_1195','1195_664','1196_1197','1197_1198','1198_1199','1198_1222','1198_4782','1199_1200','12_14','1200_4385','1201_1202','1201_4695','1202_1203','1203_6115','1205_1206','1206_1207','1207_1208','1208_1209','1209_1210','1210_1211','1210_3575','1211_3575','1212_1213','1213_1214','1214_1215','1215_1216','1216_1217','1217_4563','1218_1258','1218_1270','1219_1220','1220_1221','1221_664','1222_1223','1223_1224','1224_1225','1225_1226','1226_1227','1227_1228','1227_1253','1228_1244','1229_1231','1231_1232','1231_1235','1232_1233','1233_1234','1234_7358','1235_1251','1237_1238','1238_1239','1239_247','1240_1241','1241_1242','1242_1277','1243_4696','1244_1246','1244_254','1245_4693','1246_1229','1247_1248','1248_1273','1249_1247','1250_4698','1251_253','1251_7130','1253_969','1254_1255','1255_1256','1256_4694','1258_1259','1259_1260','126_6182','1260_1262','1260_6130','1262_4692','1265_6115','1266_1267','1267_1268','1268_1269','1269_1265','127_112','1270_1272','1272_1273','1273_1274','1274_1275','1275_1276','1276_1277','1277_1219','1278_1282','1278_1934','1279_1282','1279_1283','1279_1934','1280_6348','1280_6366','1281_1329','1282_4456','1283_4456','1283_7579','1284_1285','1285_1016','1285_1286','1285_1287','1286_1288','1287_1288','1288_1289','1289_1290','1289_1360','129_130','1290_1291','1291_1292','1291_2389','1292_1293','1293_1294','1294_1295','1295_1296','1296_1297','1297_1298','1298_1121','1298_1299','1299_1300','130_131','1300_1301','1301_1302','1302_1303','1303_1304','1303_5129','1304_1305','1305_1306','1305_1320','1306_1307','1307_1308','1308_1309','1309_1310','131_164','1310_1311','1311_1312','1312_1313','1313_1055','1314_1316','1315_1317','1316_1315','1317_1318','1318_1319','1319_1281','1320_1321','1321_1322','1322_1323','1323_1324','1324_7068','1325_1326','1326_1327','1327_1328','1328_1329','1329_1330','133_134','1330_1331','1330_2919','1331_1332','1332_1333','1333_1334','1334_1335','1334_2985','1334_7293','1335_1336','1336_1337','1337_1338','1338_1339','1339_1340','134_4890','1340_1341','1341_1342','1342_1343','1343_1344','1344_1345','1345_1347','1347_1348','1347_2634','1348_1349','1349_1350','1350_1352','1352_1353','1353_1354','1353_1355','1354_7578','1355_1357','1357_1358','1357_1359','1358_298','1358_319','1358_326','1358_340','1358_342','1358_4494','1358_5192','1358_7564','1358_7665','1359_274','1359_279','1359_319','1359_320','1359_4724','136_137','1360_1362','1362_1363','1363_1365','1365_1365','1365_1366','1366_1367','1367_1369','1367_7455','1369_1370','137_193','1370_1371','1371_1372','1371_1454','1372_1373','1373_1374','1374_1375','1374_1946','1375_1376','1376_1377','1377_1378','1378_1379','1378_4514','1379_1380','138_133','138_167','1380_1381','1381_1382','1382_1383','1383_1384','1384_1385','1385_1348','1386_1387','1387_1388','1388_1389','1389_3355','1396_2317','1397_1398','1398_1399','1399_5148','14_15','14_53','1401_1402','1402_1403','1403_1404','1404_1405','1405_4856','1406_1407','1407_1409','1409_2095','141_142','1410_1411','1411_1412','1412_1413','1413_1414','1414_1415','1415_1417','1417_1418','1418_5149','1419_6094','142_143','1423_1424','1424_1425','1424_2186','1424_2190','1425_1426','1426_1427','1426_1463','1427_1428','1428_1429','1429_1430','143_115','1430_1432','1432_1433','1433_1434','1434_1435','1435_7439','1436_3952','1438_4855','1439_1440','1440_1386','1441_1442','1442_7414','1443_1444','1443_1616','1443_1647','1444_1445','1445_1447','1445_1647','1445_4319','1445_4407','1445_7078','1447_2637','1449_1450','1450_1451','1451_1452','1451_2191','1452_2640','1453_1456','1454_1455','1455_1453','1456_1457','1457_1458','1458_1459','1459_1460','146_147','1460_1462','1462_5149','1463_4399','1464_1469','1465_1466','1466_1467','1467_1468','1468_1377','1469_1465','147_184','1471_1472','1472_4408','1473_1474','1474_1476','1476_7453','1478_1479','1478_315','1479_315','1479_7622','148_149','1480_1358','1480_2912','1481_7320','1482_1494','1484_1481','1485_4518','1486_1487','1487_1488','1488_1489','1489_1490','149_150','1491_1492','1492_1493','1493_1482','1494_1495','1495_1496','1496_1515','1499_1485','15_17','15_834','150_27','1500_302','1505_4509','1506_1507','1507_1508','1508_1509','1509_1510','151_153','1510_1511','1511_1512','1511_1592','1512_1538','1514_980','1515_1499','1517_1678','1518_1519','1519_1520','1520_7661','1521_4546','1523_1524','1524_1525','1525_1526','1526_6080','1528_1710','1529_4542','153_154','1531_1532','1532_1533','1533_1534','1534_1535','1535_1536','1536_1537','1537_186','1538_4543','1538_4548','1539_1540','154_155','1540_7426','1546_7072','1546_7560','1547_1548','1548_1280','1549_1523','155_156','1550_1551','1551_1552','1552_1553','1552_332','1553_7023','1554_1555','1555_1556','1556_1557','1557_1181','1558_1549','1558_2957','1558_4542','1559_1560','1559_987','156_157','156_7430','1560_1280','1566_1567','1567_173','1567_4372','157_158','1573_1574','1574_1575','1575_4583','1576_1577','1577_1578','1578_4364','1579_1580','1579_7673','158_159','1580_7237','1581_1582','1582_4793','1584_1585','1584_983','1585_1586','1586_1587','1586_1600','1587_1591','1587_4539','159_1539','1591_4542','1592_1593','1593_1594','1594_1595','1595_1596','1596_4363','1597_1599','1598_1597','1599_1573','16_73','1600_7701','1604_1573','1606_1607','1607_1608','1608_1609','1609_1610','1610_7471','1614_1615','1615_1478','1616_1617','1617_1618','1618_1619','1619_195','1620_220','1622_1623','1623_1624','1624_1625','1625_1626','1626_1627','1627_1628','1628_1629','1629_1630','1629_6234','1630_3670','1630_7347','1630_7348','1631_1632','1632_5053','1633_1634','1634_1635','1635_1636','1636_1637','1637_1638','1638_1639','1638_992','1639_1640','164_165','1640_231','1641_1642','1642_213','1643_7571','1647_1648','1647_1649','1648_1649','1649_1911','165_166','1654_1655','1655_1656','1656_1657','1657_4458','1658_1659','1659_1660','166_7458','1660_1661','1661_1662','1661_4523','1662_1664','1662_1666','1664_1665','1665_1666','1666_1667','1666_1807','1666_1808','1667_1668','1668_1669','1669_1670','167_168','1670_1671','1671_1672','1672_1673','1673_1674','1674_1675','1675_1849','1675_7380','1676_1517','1678_1679','1679_4520','168_169','1680_1681','1681_1682','1682_1683','1683_1684','1684_1685','1685_1686','1686_1687','1687_1688','1688_1689','1689_1690','1689_4899','169_141','1690_1691','1691_1692','1692_1693','1693_1694','1694_1695','1695_1696','1696_1697','1696_1700','1697_1698','1698_1699','1699_1700','17_18','1700_1701','1700_1906','1701_1702','1702_16','1702_1703','1703_1704','1703_1905','1704_1705','1705_1706','1706_1707','1707_1708','1708_1709','1709_1528','1710_1711','1711_1712','1712_1713','1713_1714','1713_1715','1714_1715','1715_1478','1715_7453','1719_1828','1724_1725','1725_1726','1726_1727','1727_1728','1728_1729','1729_1730','1729_1731','173_174','1730_1731','1731_1732','1731_1738','1732_1733','1733_1734','1734_1735','1735_1736','1736_1737','1737_1738','1738_1739','1738_614','1739_614','174_175','1740_1741','1741_1742','1741_1747','1742_1743','1743_1744','1744_1745','1745_1746','1746_1747','1747_1748','1747_1749','1748_1749','1749_1751','1749_544','175_176','1750_1769','1751_1752','1752_1753','1753_1754','1754_1759','1755_1756','1756_1757','1757_1794','1758_1750','1759_1758','176_181','1760_1761','1761_1762','1762_1763','1763_1764','1764_1765','1765_1767','1769_1770','177_178','1770_1771','1771_1760','1772_1773','1773_1774','1774_1775','1775_1776','1776_1777','1777_1778','1778_1755','178_1539','1782_1783','1783_1784','1784_1785','1785_1786','1786_624','1788_1789','1789_7199','179_7431','1791_1792','1792_1793','1793_962','1794_7469','1795_1806','1796_1797','1797_1798','1798_1799','1799_1800','18_19','1800_1801','1801_1277','1802_1795','1803_77','1804_1803','1805_1662','1805_1806','1806_1660','1807_7167','1808_1813','1808_1816','1808_2961','1808_4744','1808_7389','1808_7475','181_182','1813_1814','1814_1816','1814_1852','1816_1817','1816_6171','1817_1818','1818_1819','1819_1822','182_183','1820_2468','1822_4994','1826_1827','1827_7382','1828_1829','1829_4997','183_184','1833_1834','1834_1820','1834_1835','1835_1836','1836_1837','1837_2338','1838_2338','184_185','1840_1841','1841_1842','1841_4314','1842_7374','1844_6172','1845_1696','1845_7166','1847_1696','1849_1851','185_186','1850_1867','1851_1813','1852_1853','1853_1854','1854_1844','1858_1859','1859_1860','186_187','186_838','1860_4492','1861_1862','1862_1863','1863_1864','1864_1865','1865_1866','1866_1850','1867_4489','1869_1870','187_188','1870_1871','1871_1872','1872_1873','1873_1874','1874_1875','1875_1876','1876_1877','1877_1878','1878_1879','1879_1899','188_189','1882_7379','1883_1884','1884_1902','1887_1888','1888_1889','1889_1890','189_190','1890_7240','1891_7156','1893_1894','1894_1912','1895_1896','1896_1897','1897_6088','1898_1858','1899_1887','1899_6107','19_21','190_1614','190_191','190_192','1902_1903','1903_1904','1904_1840','1905_1804','1905_1906','1906_1907','1906_1909','1907_1908','1908_1909','1909_1713','191_819','1911_1805','1911_1913','1911_802','1912_1895','1913_1914','1914_1805','192_795','192_819','193_138','1934_1935','1934_2310','1934_2385','1935_1445','1935_1937','1937_1938','1938_1939','1939_1940','194_195','1940_1941','1941_1942','1941_4516','1942_1943','1943_1944','1944_1945','1945_1946','1946_1947','1947_1948','1947_2642','1948_1949','1949_1950','195_196','1950_1951','1951_1952','1952_1953','1953_1954','1954_1955','1955_1956','1956_1957','1956_2109','1957_1958','1958_1959','1959_1960','196_197','1960_4379','1963_4549','1964_4674','1965_1966','1966_1975','1968_1969','1969_1970','197_198','1970_1971','1971_2114','1971_2160','1972_1973','1973_1974','1974_1975','1975_1977','1975_1982','1977_1978','1978_1980','198_199','1980_1981','1981_1982','1981_4406','1982_2642','1982_4406','1982_7435','1984_1985','1985_1986','1986_1987','1987_1988','1987_1989','1988_1989','1988_2642','1989_1375','1989_1472','1989_1990','1989_1995','199_1506','199_200','1990_1992','1992_1993','1993_1994','1994_1995','1995_1996','1995_1998','1996_1997','1997_1998','1998_1999','1998_2001','1998_2002','1999_2001','2_192','2_4726','2_4962','200_201','2001_2002','2002_1358','2002_1359','2003_1358','2007_2008','2007_4571','2008_2009','2008_2071','2009_2010','201_148','201_202','2010_435','2011_2012','2012_3322','2013_2014','2014_2016','2014_4636','2015_2017','2016_2015','2016_2017','2017_2018','2017_3259','2017_7362','2018_2019','2019_2020','202_151','2020_2021','2021_2022','2022_4565','2023_2024','2024_2025','2025_6086','2026_7226','203_204','2031_2032','2032_2033','2033_2034','2034_2035','2035_2036','2036_2039','2036_2040','2039_2040','204_205','2040_2041','2040_3047','2041_2042','2041_3071','2042_2043','2043_2044','2044_2045','2045_2046','2046_4568','205_1620','205_206','205_215','2051_6086','2053_2054','2054_2055','2055_4565','2055_4732','2056_2057','2057_2058','2058_2059','2059_2060','206_207','2060_2061','2060_2062','2061_2062','2062_2063','2062_2064','2063_2064','2064_2065','2064_4727','2065_4727','2066_2067','2067_4728','2068_2069','2069_2070','207_208','2070_2084','2071_2072','2072_2073','2073_2074','2074_2075','2075_2076','2076_460','208_209','2080_2081','2081_2082','2082_2083','2083_2084','2084_768','2085_2086','2086_4455','2086_479','2087_7053','2088_2089','2089_763','2089_770','209_210','2094_1406','2095_2096','2096_2097','2097_2099','2099_2101','21_7602','21_85','210_1643','210_211','2101_2102','2102_2103','2102_2726','2103_2332','2103_7154','2105_2728','2105_6140','2109_3427','211_212','2114_2115','2115_6011','2118_2119','2119_2120','212_213','2120_2121','2121_2123','2123_2685','2124_2125','2125_2126','2126_2127','2127_2128','2128_2129','2129_4797','213_214','2133_6244','2134_2135','2134_3378','2135_2164','2136_2137','2137_2138','2137_7201','2138_2139','2139_2140','214_4432','2140_2141','2141_2142','2142_2143','2143_2145','2145_4670','2146_2147','2147_2148','2148_2149','2149_2150','215_216','2150_2151','2151_2152','2152_2153','2153_2154','2154_6245','2156_6014','2158_2159','2159_4409','216_217','2160_2161','2160_4665','2161_4665','2162_2163','2163_2168','2164_2165','2165_2166','2166_2167','2166_4545','2166_4554','2166_7229','2167_3383','2168_4536','2169_2170','217_218','217_242','2170_4672','2171_7160','2172_2159','2172_4409','2173_2174','2174_3425','2176_1982','2176_2177','2177_2178','2178_7354','218_219','2181_2182','2182_2778','2183_2184','2184_2185','2185_1423','2185_7043','2186_2187','2187_2188','2188_2189','2189_1436','219_220','2190_4434','2191_2192','2192_2193','2193_2194','2194_2195','2195_2196','2196_2197','2197_2198','2198_2200','22_23','220_1622','220_221','2200_2201','2201_4401','2201_7239','2202_2203','2203_2204','2204_4888','2205_2687','2206_4798','2207_2208','2208_2210','221_222','2210_2211','2211_2241','2213_2214','2213_4688','2214_2215','2215_2216','2215_7184','2216_4599','2217_2218','2218_2219','2219_2220','2219_3365','222_223','2220_2221','2221_2223','2223_2224','2224_3886','2225_4620','2227_7049','2229_2230','223_224','2230_2231','2231_2233','2232_2234','2233_2232','2234_2235','2235_2236','2235_2239','2236_2238','2236_2239','2238_5056','2239_2241','224_225','2241_2242','2242_2243','2242_7435','2243_2244','2244_2245','2245_2246','2246_2247','2247_2248','2248_2249','2249_2250','225_226','2250_2251','2251_1472','2255_2256','2256_2717','2257_2258','2258_2259','2259_2260','226_228','2260_2261','2261_4469','2262_2264','2264_2265','2265_2269','2269_2270','227_230','2270_2274','2274_7601','2278_2279','2279_2720','2279_6251','228_229','2280_2281','2281_2282','2282_4468','2283_616','2289_1822','229_227','23_24','230_231','231_1641','231_232','2310_2311','2311_2312','2311_5025','2312_2313','2313_2314','2314_2315','2315_2094','2317_2318','2318_2319','2319_2320','232_233','2320_2321','2321_2322','2322_6163','2323_2324','2324_2325','2325_2326','2326_5150','2327_2328','2328_2329','2329_2330','233_234','2330_2331','2331_1397','2331_1424','2332_2333','2333_2334','2334_4662','2335_2336','2336_2337','2337_2339','2338_1840','2338_1845','2339_2340','2339_2420','234_235','2340_2601','2346_4435','2349_2351','235_236','2351_2558','2351_2621','2351_4445','2352_2573','2353_2357','2353_4643','2357_2358','2357_2359','2357_2566','2358_2359','2359_2360','2359_7122','236_1190','236_237','2360_4340','2360_4348','2369_2370','237_213','2370_2371','2371_2372','2372_2373','2373_1424','2373_2377','2374_2146','2377_2378','2377_2431','2378_2183','2379_2380','2380_2382','2382_5099','2383_1479','2383_2002','2383_2003','2385_2386','2386_2387','2387_2388','2388_1290','2389_2390','2390_2391','2391_2392','2392_2393','2393_2394','2394_1090','2394_2395','2395_2396','2396_2397','2397_2398','2398_2399','2399_2400','24_25','2400_2401','2401_2402','2402_2403','2403_1096','2404_2405','2405_2406','2406_2408','2408_2409','2409_2410','2410_2411','2411_2412','2412_2438','2413_2414','2414_2415','2415_2416','2416_2417','2417_2418','2418_2419','2419_2452','242_243','2420_2421','2421_2423','2423_2424','2424_2447','2425_2426','2426_2427','2427_2428','2428_2429','2429_2430','243_244','243_254','2430_2371','2431_2432','2432_2433','2433_2327','2433_2434','2434_2435','2435_2436','2436_2437','2437_1117','2437_2438','2438_2439','2439_2440','244_245','2440_2441','2441_2442','2442_2443','2443_2444','2444_1344','2445_2457','2446_2459','2447_5133','2448_5026','245_246','2450_2490','2452_2453','2453_2454','2454_2455','2455_2456','2456_2445','2457_4392','2458_2446','2459_2460','246_248','2460_2461','2461_2462','2462_2463','2463_2431','2465_2466','2466_2467','2467_2494','2468_7319','247_1240','2471_2472','2472_2473','2473_2474','2474_2475','2475_2476','2476_2477','2477_2478','2478_1170','2479_2480','248_249','2480_2481','2481_2482','2482_2483','2483_2484','2484_2485','2485_2486','2486_2487','2487_2488','2488_2489','2489_2448','249_250','2490_2491','2491_2492','2492_2493','2493_2465','2494_2495','2495_2496','2496_2497','2497_2471','2498_2499','2499_2500','2499_7216','25_27','250_251','2500_2501','2501_7623','2508_2278','251_252','2510_7242','2515_2516','2516_2517','2517_2518','2518_4749','252_255','2522_2524','2523_2532','2524_2523','253_5033','2532_4877','2535_2536','2536_2616','2538_2606','2539_2540','254_1246','254_244','2540_2542','2540_4882','2542_3004','2547_2548','2548_2600','2548_946','255_234','2550_2516','2550_2551','2551_2553','2553_2554','2554_2425','2554_2555','2555_2556','2556_2557','2557_4436','2558_2559','2558_6170','2559_2560','2560_2561','2561_2562','2562_2564','2564_4927','2566_2567','2567_2568','2568_2569','2569_2570','2569_7550','2570_2571','2570_7222','2571_2359','2573_2574','2574_2575','2574_2577','2575_4929','2577_2578','2578_2579','2579_7395','2583_2584','2584_2585','2585_2586','2586_2587','2587_2588','2588_2589','2589_2590','2590_2591','2591_2592','2592_2357','2594_2595','2595_2596','2596_2597','2596_2611','2597_2599','2599_2600','2599_4577','2600_1155','2601_4446','2602_2613','2603_2557','2604_2605','2605_2538','2606_2607','2607_2608','2608_2609','2609_2610','261_270','261_271','2610_2554','2611_2602','2613_2614','2614_2615','2615_2535','2616_2617','2617_2557','262_270','2621_2622','2622_2623','2622_2750','2623_2624','2624_4441','2626_2669','2628_2629','2629_6095','263_271','263_277','263_4508','263_4725','263_6071','2631_2359','2632_2633','2633_2369','2634_2635','2635_2636','2636_2383','2636_5097','2637_1956','2637_2638','2637_2721','2638_2640','2638_2721','264_6059','2640_2641','2641_1948','2641_2643','2642_2643','2642_7435','2643_2644','2644_2645','2644_2709','2645_2646','2646_2647','2647_2648','2648_2649','2649_2650','265_271','265_6059','2650_2652','2652_2653','2653_2654','2654_2655','2655_2656','2656_2657','2656_2668','2657_2658','2658_2659','2659_2660','2660_2661','2661_2662','2662_2909','2663_2664','2664_2665','2665_2666','2666_2667','2667_2696','2668_2672','2669_4641','2670_2910','2672_2673','2673_2206','2674_2675','2675_2124','2676_2118','2678_2681','2679_4691','2680_2676','2680_4677','2681_2680','2682_4796','2685_2686','2686_7510','2687_2688','2688_2689','2689_2696','2696_2697','2697_2700','2697_2713','27_28','270_335','270_336','2700_2701','2701_2702','2702_2703','2703_2704','2704_2706','2706_6070','2708_5134','2709_2710','271_1278','271_340','271_404','271_4521','271_496','2710_2711','2711_2712','2712_2655','2713_2714','2714_2715','2715_2716','2716_2718','2717_2257','2718_2719','2718_7435','2719_1989','2719_7435','272_335','2720_2280','2721_7012','2722_1471','2722_4413','2725_2737','2726_2727','2727_2105','2728_2729','2729_2730','273_405','2730_6083','2733_2734','2734_5169','2735_2736','2736_2725','2737_2738','2738_2739','2739_7177','274_1505','274_2','274_7','2740_2741','2741_2742','2742_2743','2743_2745','2745_2746','2746_2747','2747_2748','2748_7062','2750_2751','2751_2752','2752_2753','2753_2754','2754_2755','2755_2756','2756_2764','2764_7178','2765_2766','2766_2767','2767_2772','2768_2769','2769_2770','277_288','277_292','277_336','2770_2771','2771_2773','2772_2768','2773_2774','2774_2777','2775_2181','2777_2775','2778_2185','278_10','278_2','278_7615','278_8','279_3','279_7','2795_756','28_29','2804_376','2809_2810','2809_408','2809_7612','281_261','281_4','281_6','281_7','2810_2811','2810_7385','2810_7387','2810_749','2810_750','2810_842','2810_843','2811_749','2811_842','2811_843','2812_2813','2813_2814','2814_852','2815_2816','2816_2817','2817_2818','2818_2819','2819_2820','2819_3012','2820_2822','2822_2823','2823_5032','2824_2825','2825_2826','2826_2827','2826_2842','2827_2828','2828_2829','2829_2830','2830_2831','2831_2832','2832_2833','2832_4349','2833_6006','2833_7375','2835_7176','2836_2837','2837_2838','2838_2839','2839_2840','284_4504','2840_2841','2840_2845','2841_2865','2842_2843','2843_2844','2844_2845','2845_2846','2846_2847','2847_4397','2848_2849','2849_2850','2850_2851','2851_2852','2852_4707','2852_4987','2853_2854','2854_2855','2855_2856','2856_2857','2857_2858','2858_2859','2859_2860','286_271','2860_2861','2861_2829','2861_2862','2862_2863','2863_2864','2864_4486','2865_2866','2866_2825','2866_2892','2866_6044','2867_1035','2868_2892','288_1171','289_496','2892_2893','2893_2894','2894_2895','2895_2896','2896_2897','2897_1062','2897_2898','2898_3562','2899_884','29_4680','2900_2901','2901_2902','2902_2903','2903_1074','2906_2907','2907_2908','2909_2906','2909_6020','2910_2911','2911_7507','2912_1358','2914_2915','2914_2989','2915_2916','2916_2917','2917_2918','2918_1300','2919_2920','292_1171','2920_2921','2921_2922','2922_2923','2923_2924','2924_2925','2925_2948','2925_7442','2927_2929','2929_2930','2930_7441','2934_2935','2935_2936','2936_2937','2937_2938','2938_7440','2940_2948','2941_2942','2942_2943','2942_7065','2943_2944','2944_2945','2945_2946','2946_2947','2947_1305','2948_2949','2949_2950','2950_2954','2954_2955','2954_7067','2955_7067','2956_2965','2957_2958','2958_4938','2961_4901','2962_2963','2963_2964','2964_4175','2965_2967','2965_2983','2966_2967','2967_2968','2968_2969','2969_2970','297_407','297_496','2970_2971','2970_4988','2971_5051','2976_2977','2977_2978','2978_2979','2979_2980','298_496','2980_2981','2981_2954','2981_2991','2983_2984','2984_2927','2984_2941','2984_7449','2985_2986','2986_2988','2988_3001','2989_2990','299_497','2990_1042','2991_2992','2992_1325','2993_2994','2994_4180','2995_3154','2996_3258','2997_4153','2998_2999','2999_3000','3_12','3_192','30_69','300_496','300_497','3000_3001','3001_3002','3002_1166','3003_2522','3004_4756','3005_4757','3006_4871','3007_6283','3008_6089','301_336','301_4495','3011_2896','3012_877','3014_3015','3015_3016','3016_3017','3017_3018','3018_3019','3019_3020','302_333','302_340','302_4495','3020_3021','3021_3022','3022_441','3023_3024','3024_3025','3025_3026','3026_3027','3027_3028','3028_3029','3029_3030','3030_3031','3031_875','3032_3033','3033_3034','3034_3036','3034_3114','3036_3037','3037_3038','3038_3039','3039_3040','3040_3041','3041_2035','3041_3042','3042_2035','3045_4150','3047_3202','3048_3049','3049_3050','3050_3051','3051_3052','3052_3053','3053_3054','3054_3055','3055_3056','3056_3057','3057_3059','3057_4830','3059_3060','3060_3061','3061_3062','3062_3063','3063_3064','3064_3065','3065_3066','3066_3067','3067_3202','3068_3069','3069_3070','307_308','3070_2041','3071_3072','3072_3073','3073_3074','3074_3075','3075_3076','3076_3077','3077_3079','3077_3084','3079_3080','308_309','3080_3081','3080_3082','3081_3084','3081_7188','3082_3084','3082_3114','3082_7660','3084_469','3085_3034','3088_3704','3089_3090','309_2383','309_7412','3090_3706','3091_3092','3092_3093','3093_3535','31_32','3112_3924','3114_3115','3114_4353','3115_3116','3116_3117','3116_6085','3117_3118','3118_3119','3119_3120','312_1444','3120_3122','3122_3123','3123_3124','3124_3125','3125_3126','3126_3127','3127_3128','3128_3130','3129_3130','3130_3131','3131_3132','3132_3133','3133_3134','3133_5046','3134_3135','3135_3136','3135_7363','3136_3138','3138_3139','3139_4124','3140_3141','3141_3142','3142_3143','3142_3224','3143_3144','3144_3145','3144_5047','3144_7639','3145_3146','3145_5128','3146_3147','3147_3148','3148_3149','3148_5127','3149_3150','315_313','315_335','315_336','315_404','315_406','315_4495','3150_3151','3151_3153','3152_2995','3153_3152','3154_3155','3155_3156','3156_3158','3158_3159','3159_3160','3160_3161','3161_3162','3162_3163','3163_3164','3164_3079','3164_3084','3164_7188','317_312','318_312','3181_451','3184_449','319_1443','319_274','319_278','319_279','319_281','319_7491','3196_3345','32_33','320_278','320_281','3202_3203','3203_3204','3204_3205','3204_4568','3204_7641','3205_3206','3206_3207','3207_3208','3208_3209','3209_3210','3210_4981','3211_3212','3212_3213','3213_3214','3214_3215','3215_3216','3216_3217','3217_3218','3218_3219','3219_3220','3219_3537','322_323','322_7697','3220_3221','3220_5046','3221_4982','3222_3223','3223_3224','3224_3225','3225_3226','3226_3227','3227_3228','3228_3229','3229_7472','3229_7644','323_332','323_6234','3230_3231','3230_3232','3231_3246','3232_3233','3233_3234','3234_4462','3235_3236','3236_3237','3237_3238','3238_3239','3239_3240','324_1555','324_1634','324_327','324_6235','3240_3048','3240_3068','3243_3244','3244_3245','3245_3212','3246_3247','3247_7056','325_1443','3258_2060','3259_7361','326_1443','327_6016','327_7113','328_1443','3280_3282','3282_3479','3283_3284','3284_3310','329_1443','33_7270','3310_3481','3317_3318','3318_3320','3320_2064','3321_2066','3321_4727','3322_3563','3328_3329','3329_3330','333_747','3330_3331','3331_3351','3332_3333','3333_3334','3334_3352','3335_3336','3336_3337','3337_3338','3338_3339','3339_3340','334_406','334_7581','3340_4324','3344_7421','3345_4727','335_1278','335_404','335_4521','3351_3332','3352_3335','3355_3356','3356_1441','336_1278','336_1279','336_7581','3365_3366','3366_3367','3367_3886','3368_4626','3368_7137','3370_7168','3370_7231','3372_3373','3373_3374','3374_2233','3376_3377','3377_3378','3378_3379','3379_4669','3383_3384','3384_3385','3385_3386','3386_3387','3387_4559','3389_3390','3390_6119','3392_3393','3393_3394','3394_3396','3396_3397','3397_3398','3398_3399','3399_6187','340_350','3400_3401','3401_3402','3402_3403','3403_3404','3404_3405','3405_3406','3406_3410','3407_3408','3408_3409','3409_4558','3410_3407','3411_3412','3412_3413','3413_3414','3414_3415','3415_3416','3416_3417','3417_3418','3418_3419','3419_2374','3419_3420','3419_4762','342_1279','342_4495','342_4521','3420_3421','3421_3422','3422_3423','3422_4671','3423_2156','3423_4409','3425_2176','3426_3434','3427_3428','3428_3429','3429_3432','3432_3426','3432_3434','3434_3435','3434_3453','3435_3453','3436_7432','3437_3439','3437_3440','3438_5022','3439_3440','3440_3441','3440_4557','3441_4560','3443_3444','3444_3445','3444_3447','3445_3446','3446_3447','3447_3448','3447_3454','3448_3449','3449_3436','345_318','345_320','3453_3437','3454_3455','3455_3456','3455_5022','3456_3457','3457_3438','346_317','3461_4532','3461_4537','3465_4673','3466_3467','3467_3469','3469_3470','3470_3471','3471_3472','3472_3473','3473_3474','3473_5015','3474_3475','3475_3476','3476_3477','3477_3478','3478_3280','3479_4086','3481_3482','3482_3483','3483_3484','3484_3485','3485_3486','3486_3487','3487_3488','3487_7415','3488_3489','3489_3492','3491_6007','3492_2835','3493_3494','3494_3495','3495_3496','3496_3497','3497_3498','3498_3499','3499_3500','35_36','350_2809','350_351','3500_3501','3501_3502','3502_3503','3503_3504','3504_3505','3505_3506','3506_3507','3507_3508','3508_3510','351_352','3510_3512','3512_3514','3514_3515','3515_3516','3516_3517','3517_3518','3518_3519','3519_3521','352_353','3521_3523','3523_3524','3524_3525','3525_3527','3527_3528','3528_3529','3529_3532','353_354','3532_3533','3533_3491','3535_3536','3536_3537','3537_3538','3538_7556','3539_3540','354_355','3540_3541','3541_3542','3542_3543','3542_3544','3543_3141','3544_3552','355_356','3552_3553','3553_3554','3554_3555','3555_3556','3556_3557','3557_7317','356_357','3562_2899','3563_444','3565_3825','3566_3567','3567_7292','3569_3571','357_390','3571_3801','3572_3701','3573_3662','3574_3641','3575_3576','3576_3577','3576_3648','3577_3578','3578_3579','3579_3580','3580_3640','3583_3584','3584_3585','3585_3587','3585_3625','3587_3588','3588_3589','3588_3590','3589_3591','3589_3605','3590_3589','3591_3592','3592_3593','3593_3594','3594_3595','3595_3596','3596_3597','3597_3598','3598_4465','3599_3600','3599_3610','36_37','3600_3601','3601_3602','3602_3604','3604_3616','3605_3606','3606_3607','3607_3608','3607_6051','3608_3609','3609_3598','3610_3611','3611_6135','3612_3613','3613_3615','3615_3616','3616_3618','3618_3619','3619_3620','3619_6087','3620_3621','3620_3622','3621_3622','3622_3623','3623_4387','3624_3587','3625_3626','3626_3627','3627_3628','3628_6052','3629_3630','3630_3631','3631_3632','3632_3633','3633_3634','3634_3635','3635_3636','3636_905','3639_3644','3640_3574','3641_3642','3642_3643','3643_3583','3644_3645','3645_3646','3646_3647','3647_3661','3648_3649','3649_3650','3650_7328','3651_3652','3652_3653','3653_3654','3653_6055','3654_5084','3655_5086','3656_3657','3656_7194','3657_3658','3658_3659','3658_7329','3659_3660','3660_3647','3661_3573','3662_1212','3663_7347','3664_3813','3665_7400','3669_3671','3669_7349','3669_7399','3669_7611','3670_3671','3671_3672','3672_3674','3674_3675','3675_3676','3675_5030','3676_3677','3676_5073','3677_3678','3678_3679','3679_3711','3679_5075','3682_3864','3686_5077','3689_3690','3689_3691','3690_5079','3690_6117','3691_3572','3691_3692','3692_3694','3694_3695','3695_3696','3695_7115','3696_3697','3697_3698','3698_3699','3699_3885','37_38','3701_3702','3702_3088','3704_3705','3705_733','3706_3707','3707_3708','3708_3709','3709_3710','371_391','3710_4925','3711_3712','3711_7189','3712_3713','3712_4916','3712_6359','3713_3714','3714_3715','3714_3719','3715_3755','3715_5175','3718_3720','3719_3718','372_373','3720_7691','3721_3722','3722_5069','3723_3724','3724_3725','3725_3726','3725_3738','3726_7039','3728_3729','3729_3730','373_374','3730_3732','3732_3733','3733_3734','3734_3735','3735_7039','3736_3738','3738_7170','374_375','3740_3741','3741_5165','3743_3780','3744_3715','3744_3745','3745_3746','3746_3749','3748_6050','3749_3750','375_2804','3750_3752','3750_6252','3751_3689','3751_7114','3752_3751','3755_3756','3756_3757','3757_3758','3758_3759','3759_3760','376_377','3760_3761','3761_3762','3762_3763','3763_3764','3764_3765','3765_3766','3766_3767','3767_3768','3768_3770','3769_3771','377_378','3770_3769','3771_3772','3772_3773','3773_3774','3773_5176','3774_7172','3775_3777','3777_3778','3778_7690','378_380','3780_3744','3781_3782','3782_3783','3783_3784','3784_3785','3785_3786','3786_3787','3787_3788','3788_3789','3789_3790','3790_3791','3791_3792','3792_3793','3793_3794','3794_3795','3795_3797','3797_3798','3798_3799','3798_6076','3799_6076','38_39','380_2087','3801_3802','3802_3805','3805_7175','3806_3807','3807_3808','3808_3809','3809_7174','381_382','3811_3812','3812_3664','3813_3814','3814_3815','3815_3816','3816_3566','3818_3819','3819_3820','382_4451','3820_3822','3822_3823','3823_3824','3824_3565','3825_3826','3826_3827','3827_3828','3828_3829','3829_3831','383_384','3831_3832','3832_3833','3833_3834','3834_3835','3835_3836','3836_3837','3837_3838','3838_3840','384_385','3840_3842','3842_3843','3843_3844','3844_3845','3845_3846','3846_3848','3848_3849','3849_3850','385_387','3850_3851','3851_3852','3852_3853','3853_3854','3854_3855','3855_3856','3856_3857','3857_3858','3858_3859','3859_3860','3860_3861','3861_3862','3862_3863','3863_3748','3864_3865','3865_4910','3865_7133','3867_7135','387_388','388_389','3881_5096','3884_3686','3885_1631','3885_3663','3886_3887','3886_3888','3887_3888','3888_3890','3888_7186','389_393','3890_3891','3891_3949','3891_7187','3892_3893','3893_3895','3895_3896','3896_3897','3897_3898','3898_3899','3899_3900','39_116','39_40','390_372','3900_3901','3901_3902','3902_3904','3904_3905','3905_3906','3905_7085','3906_3907','3907_3908','3907_3909','3908_3909','3909_5118','391_392','3910_3911','3911_3912','3912_3913','3913_3914','3914_3925','3914_7075','3917_3918','3918_3926','3918_5114','3919_3920','392_395','3920_3921','3921_3922','3922_3923','3923_3112','3923_7079','3924_3927','3925_3008','3926_7074','3927_3928','3928_3929','3929_7108','393_371','3935_3936','3936_3938','3936_7110','3938_3939','3939_3940','3940_3941','3941_3943','3943_3944','3944_3945','3945_3946','3946_4001','3947_3892','3949_3950','395_396','3950_2229','3950_3372','3952_2190','3954_3955','3955_3956','3956_3957','3957_3958','3957_3993','3958_3959','3959_3960','3959_7234','396_397','3960_3961','3960_6156','3961_3962','3962_3963','3963_3964','3964_3965','3965_3968','3966_5113','3968_3966','3968_7771','397_398','398_399','3980_6089','3981_3982','3982_3983','3982_7704','3983_3985','3985_3986','3986_3987','3986_7313','3987_3989','3989_3990','399_400','3990_3991','3991_3998','3993_3994','3994_3995','3995_3996','3996_3997','3996_3998','3997_3998','3998_3999','3999_4000','4_194','4_4962','4_7555','40_146','40_41','400_279','400_281','400_303','400_319','400_346','400_4522','400_7392','400_7564','400_7581','4000_4001','4001_3949','4001_7193','4002_5112','4003_4404','4004_4005','4005_4006','4006_4007','4007_4008','4008_4010','4009_4011','4010_4009','4011_5110','4012_4013','4013_4014','4014_4098','4015_4017','4017_4018','4018_4019','4019_4020','4020_4100','4021_4022','4022_4023','4023_4024','4024_4027','4027_7250','404_747','405_408','4051_4052','4052_4054','4054_4055','4055_4057','4056_4133','4057_4101','4058_4059','4059_4060','406_2810','406_4530','406_747','4060_4061','4061_4062','4062_4063','4063_4064','4064_4056','4065_4066','4066_4067','4067_4068','4068_4069','4069_4070','407_2498','4070_4071','4071_4072','4072_4073','4073_2583','408_409','4086_4087','4087_4088','4088_4089','4089_4090','409_410','409_411','4090_4091','4091_6097','4093_4094','4094_4096','4096_4114','4098_4015','41_42','410_411','4100_4021','4101_4102','4102_4058','411_412','411_414','4114_4115','4115_4116','4116_4117','4117_4118','4118_4119','4119_4120','412_413','4120_4121','4121_4122','4122_4123','4123_3283','4124_4125','4125_4126','4126_4127','4127_4128','4128_4129','4129_4130','413_414','4130_4131','4131_7294','4132_4134','4132_4171','4133_4065','4134_4135','4135_4136','4136_4207','414_415','414_416','4148_4149','4149_3045','415_416','4150_4151','4151_2997','4151_4156','4153_4154','4154_4416','4156_4168','416_417','4168_4170','417_418','417_485','4170_4132','4170_4153','4171_4172','4172_4173','4173_2962','4175_5091','4177_4178','4178_4179','4179_2993','418_419','4180_4181','4181_4182','4182_4151','419_420','42_43','420_2088','420_421','4201_4202','4202_4203','4203_4204','4203_5090','4204_4205','4205_4206','4206_3140','4207_4208','4208_4209','4209_4210','421_422','4210_4211','4211_4212','4211_4215','4212_4213','4213_5139','4214_4215','4215_4216','4216_4217','4216_4222','4217_4218','4218_4219','4219_4220','422_423','4220_4221','4221_4222','4222_4224','4224_4225','4225_4226','4225_7462','4226_4227','4227_4228','4228_4229','4229_4230','423_424','4230_4231','4230_4248','4231_7352','424_425','4246_4248','4248_4249','4248_4252','4249_4250','425_426','4250_4251','4251_4252','4252_4253','4252_4305','4252_6137','4253_4254','4253_4268','4253_4305','4254_6137','4256_4257','4257_4258','4258_4259','4259_7272','426_427','4260_4261','4260_7296','4261_7271','4262_4263','4263_4264','4264_7296','4265_4266','4266_4253','4266_4267','4266_4307','4267_4268','4268_4269','4268_4273','4269_4270','427_428','4270_4271','4271_4273','4273_4274','4273_4277','4274_4275','4275_4276','4276_4277','4277_4278','4277_7462','4278_4279','4279_4280','428_429','4280_4281','4281_4282','4282_4283','4283_4284','4284_4285','4284_4295','4285_4286','4286_4287','4287_4288','4288_4295','429_3032','429_6334','4295_4296','4296_4297','4296_4301','4297_4299','4299_4300','43_44','4300_4301','4301_4302','4302_4591','4302_5135','4303_4304','4304_4148','4305_4306','4306_4307','4306_4309','4307_4308','4308_4253','4308_4267','4309_4254','4314_1687','4315_4316','4316_4317','4317_7013','4319_1476','4320_1476','4321_7342','4323_1845','4324_4769','4325_3328','4330_3679','4331_4339','4333_4334','4334_4712','4336_1051','4339_7616','4339_7692','4342_4348','4344_2349','4347_2349','4347_7181','4348_4646','4348_7122','4349_3184','435_2011','435_7353','4350_5143','4353_4355','4354_3196','4355_4354','4362_2959','4362_7025','4363_1598','4364_1579','4369_6219','437_459','4371_832','4372_179','4376_4377','4377_4890','4379_1963','4379_1968','4380_1172','4380_496','4382_1185','4382_1196','4383_4699','4384_518','4384_519','4385_1201','4386_653','4387_3634','4388_4700','4392_2458','4397_2848','4399_4400','44_7603','4400_1464','4401_2202','4401_2213','4404_4405','4405_3991','4406_1984','4407_2637','4407_4320','4408_1473','4409_1972','4409_2173','4409_4550','441_442','4410_2678','4411_2239','4411_4715','4413_1476','4413_1480','4413_7453','4415_301','4415_302','4416_4201','442_443','443_444','4432_119','4433_3752','4434_2379','4435_2594','4435_2604','4435_2632','4436_5008','4438_3432','444_445','4440_2628','4441_2626','4445_2352','4446_2603','445_4779','4451_383','4455_381','4456_1284','4458_1658','4460_453','4462_3235','4464_1869','4465_944','4466_633','4468_2283','4469_2262','447_5045','4472_4386','4475_3023','4477_3321','4479_4376','448_3181','4481_4485','4485_822','4486_2865','4489_4747','449_450','4492_1861','4493_4362','4495_1278','4495_1279','4495_2809','4495_4522','4495_4720','4495_746','4495_7581','4498_1660','4499_4934','45_46','450_3181','4500_4481','4501_4935','4502_4936','4504_7028','4508_7402','4509_499','4509_7708','451_447','4513_272','4513_286','4514_4515','4515_1995','4516_1369','4518_509','4520_4825','4521_1283','4521_1934','4521_1935','4522_1934','4523_4524','4524_4322','4525_4526','4526_1700','4528_1072','453_4477','453_454','4530_748','4532_4534','4533_4844','4534_4535','4535_2169','4536_6003','4537_2176','4538_7396','4539_1606','454_4475','4542_1531','4543_1560','4545_4552','4546_1559','4548_1514','4548_980','4549_1964','4550_4551','4551_1965','4552_4554','4554_7016','4554_7229','4554_7230','4555_3418','4555_4576','4557_3441','4558_3411','4559_3389','4560_5126','4561_4629','4562_7386','4563_1218','4564_3443','4564_3444','4565_4566','4566_4567','4567_2031','4567_3238','4568_4569','4569_2023','4569_4570','4570_2056','4571_2013','4571_2014','4573_4893','4574_4575','4575_1680','4576_3418','4577_4578','4578_1100','4583_1518','4583_1576','4588_4589','4589_7364','459_2080','4590_7366','4591_4590','4592_980','4593_4594','4593_4595','4593_7516','4594_4595','4595_4596','4596_4563','4599_2217','4599_4601','4599_7184','46_47','460_2011','4601_5058','4602_4603','4603_4604','4604_4605','4605_4562','4606_7142','4607_4608','4607_7140','4608_4609','4609_4610','461_2068','4610_4611','4611_4612','4612_4613','4613_3370','4616_2227','4617_4555','4619_4561','4620_7563','4623_4619','4624_4625','4625_4628','4626_4627','4627_4624','4628_4623','4629_4630','4630_4631','4631_4632','4632_5164','4633_2234','4636_2016','4640_4347','4640_7214','4641_4642','4642_2355','4643_4644','4644_4645','4645_4440','4646_4647','4647_2346','4647_4435','4662_2335','4663_4664','4664_2160','4665_4666','4666_4667','4667_2162','4667_4668','4668_3376','4668_4677','4669_2146','4669_2165','4670_2165','4670_3420','4671_4672','4672_3465','4673_2172','4674_4663','4677_2136','4680_31','4680_90','4683_107','4688_4689','4689_4410','469_470','4690_4411','4691_2682','4691_4796','4692_1245','4693_1254','4694_1249','4695_1243','4696_4697','4697_1250','4698_4383','4699_4388','47_48','470_471','4700_1266','4705_475','4707_4708','4708_4336','4709_1037','471_472','4710_1046','4712_2853','4713_4714','4714_4960','4717_2499','472_473','4720_1443','4724_263','4725_273','4725_334','4725_4495','4725_6059','4725_6071','4726_7555','4727_4728','4727_768','4728_437','4728_461','473_4705','4731_2023','4732_3243','4742_4743','4743_4744','4743_4745','4744_4745','4745_4989','4746_1845','4747_1845','4747_1882','4749_4750','475_476','4750_3006','4750_4751','4751_4754','4754_4755','4755_3003','4756_3005','4757_4758','4758_6343','4759_4760','476_477','4760_2547','4762_4763','4763_6245','4765_4766','4766_4767','4767_4768','4768_1845','4768_4325','4769_4770','477_478','4770_4765','4779_7057','478_479','478_7053','4782_1782','4788_6273','479_480','4793_6068','4795_2686','4795_4688','4796_4690','4797_7473','4798_2674','4798_4715','4799_2205','48_49','480_481','4800_5157','4804_738','4806_5156','481_482','482_483','4825_7233','483_485','4830_7667','4843_4905','4844_4177','4847_4848','4848_4350','485_487','485_488','4854_1438','4855_1439','4856_4857','4857_307','4861_4862','4863_4864','4864_4865','4865_4577','4869_4870','487_488','4870_3007','4871_4872','4872_4873','4873_4874','4874_2532','4877_4878','4878_7180','488_489','488_491','4880_2539','4880_2606','4882_4883','4883_4884','4884_4885','4885_6343','4886_4887','4887_1150','4888_4798','489_490','4890_4369','4891_7458','4893_4894','4894_4895','4895_7217','4897_4898','4898_4574','4899_4900','49_51','490_491','4900_4746','4901_4902','4902_1672','4903_1883','4903_7219','4905_4906','4906_4907','4907_4908','4908_4909','4909_3884','4909_4921','491_492','491_493','4910_4911','4911_4912','4912_4913','4913_4914','4914_4915','4915_4957','4915_7654','4916_4917','4917_4918','4918_4919','4919_4920','492_493','4920_4910','4921_4922','4922_4923','4923_4924','4924_4958','4925_3679','4925_4926','4926_3678','4927_4928','4928_2352','4929_7460','493_494','4930_4931','4931_7459','4932_4933','4933_2590','4934_4500','4935_4502','4936_79','4937_1547','4938_1549','494_495','495_400','495_496','495_7587','495_7588','4952_768','4953_768','4957_7133','4958_3752','496_497','496_515','496_522','496_523','4960_4932','4962_264','4962_4725','4962_6059','497_515','4974_4976','4976_7126','4978_4979','4979_4986','4980_2289','4981_3211','4984_448','4985_6219','4986_7107','4987_2978','4988_2853','4989_1845','4989_4990','499_500','4990_4991','4991_4992','4992_1719','4994_4995','4995_4996','4996_1826','4997_4998','4998_4999','4999_1833','500_501','5008_4640','501_1484','5013_5014','5014_7291','5015_7290','5022_3461','5022_6002','5023_5024','5024_5161','5025_4854','5026_2450','5028_3696','5028_7115','5029_5028','5030_5031','5031_6033','5032_2824','5032_2867','5033_7130','5044_3751','5044_4433','5045_4460','5046_7639','5047_3146','5047_3224','5051_5052','5052_2975','5053_1633','5056_2239','5058_4602','5060_5061','5061_1231','5062_691','5064_7173','5069_3723','5071_5072','5072_4321','5072_7342','5073_3679','5073_5074','5074_6054','5075_5076','5076_3682','5077_3694','5077_5078','5078_3689','5079_3694','5081_5082','5082_3639','5084_6054','5086_3656','509_510','5090_4204','5091_5092','5092_7574','5096_7145','5097_2383','5099_2383','51_52','510_511','511_513','5110_4012','5111_4051','5112_4003','5113_3918','5118_3910','5119_5120','512_513','5120_5121','5121_5122','5122_3461','5123_5124','5124_5125','5125_6001','5127_5128','5128_2060','5128_2996','5129_5130','513_4513','513_7402','5130_1135','5131_5132','5132_1331','5133_2596','5134_2718','5135_4303','5136_4262','5139_4214','5140_1500','5140_620','5142_7567','5143_2837','5144_7027','5144_7702','5145_276','5147_68','5148_1401','5149_1419','515_516','5150_2327','5154_4804','5155_4806','5156_7552','5157_5154','516_4384','5160_5155','5161_5162','5162_2571','5162_4880','5164_4633','5165_3743','5165_7169','5169_2735','5171_2976','5174_6078','5175_3755','5176_3774','518_2255','519_521','5192_342','5192_350','52_261','52_262','52_263','52_265','52_277','52_4725','521_522','522_1741','522_523','523_1740','523_4382','523_525','523_669','525_526','526_527','527_528','528_529','529_530','53_54','530_531','531_709','532_533','532_649','533_534','534_535','535_536','536_537','537_538','538_539','539_540','54_55','540_541','541_542','542_543','543_544','544_545','544_546','545_546','546_547','546_910','546_914','547_548','548_549','549_550','549_677','55_56','550_551','551_552','552_553','553_554','554_555','555_557','557_558','558_559','559_560','56_58','560_562','562_563','563_564','564_565','565_566','566_567','566_707','567_568','568_569','569_570','57_58','570_571','571_572','572_573','573_574','574_575','575_576','576_577','577_579','579_580','58_4499','58_59','580_581','581_582','582_583','583_584','584_585','585_586','586_587','587_588','588_589','588_590','589_590','59_60','590_1729','590_591','591_592','592_593','593_594','594_595','595_596','596_597','597_598','598_599','599_600','6_194','60_1802','60_820','600_601','600_654','6001_4438','6002_5119','6003_5123','6004_3','6004_7','6005_1546','6005_7072','6006_3493','6007_2836','601_602','6010_3656','6011_2133','6013_6247','6014_2158','6014_6242','6015_5029','6016_6017','6017_7192','602_603','6020_6023','6021_7507','6023_6026','6024_6021','6025_6024','6026_6027','6027_6029','6028_6025','603_604','6030_6028','6033_3089','604_605','6041_2826','6042_1022','6048_5064','605_606','6050_3749','6050_7398','6051_6053','6052_3613','6053_7327','6054_4330','6055_1073','6057_2183','6059_334','6059_404','606_607','6068_1584','607_608','6070_2708','6074_748','6074_793','6075_2631','6076_6077','6077_3569','6078_7112','6079_6315','608_609','6080_1585','6080_983','6083_2733','6085_2026','6085_2053','6086_3161','6086_7419','6087_3620','6088_1898','6089_3981','609_610','6094_2102','6095_7062','6097_4093','610_611','6107_6108','6107_6109','6108_6109','6109_6110','611_612','6110_7020','6110_7029','6111_6112','6112_6113','6113_1898','6113_4323','6115_1205','6115_4593','6117_3655','6117_5079','6119_3392','612_613','6122_35','6124_4004','6125_7092','613_614','6130_1262','6132_2551','6132_4861','6135_3612','6137_4256','6137_4956','614_4415','614_615','614_675','6140_6144','6141_6142','6142_2181','6143_6011','6144_6145','6145_6146','6146_6147','6147_6149','6149_6150','615_616','6150_6143','6152_6153','6153_6243','6154_6155','6155_1987','6155_6141','6156_6165','6157_6158','6158_3964','6158_7314','6159_6160','616_617','6160_6161','6161_3991','6162_7022','6163_1095','6164_6159','6165_6157','617_618','6170_5023','6170_7183','6171_4974','6172_4493','618_619','6182_322','6185_6186','6186_6216','6187_6188','6188_6215','619_675','620_1172','620_621','620_7568','620_7569','621_289','621_291','621_293','621_4495','6215_6184','6216_3400','6219_6229','6228_4788','6228_7111','6229_6348','6229_6364','6229_7131','6230_6228','6234_332','6235_327','6238_6182','6239_104','624_625','6241_105','6242_6152','6243_6154','6244_2134','6245_2156','6245_4671','6245_6013','6247_2679','625_626','6251_5140','6251_7706','6252_301','6252_302','6253_7191','626_627','627_628','6273_136','6273_4891','6276_6362','6277_6278','6278_6279','6279_6280','628_629','6282_6335','6283_6282','6285_6286','6286_6287','6287_6288','6288_6342','629_630','6290_7120','6298_6336','63_64','630_4466','6310_6311','6311_4495','6311_7399','6312_7197','6315_6316','6316_7236','6318_6319','6319_7246','6320_4594','6324_6325','6325_6341','6326_4886','633_634','6334_7660','6335_6326','6336_7069','6339_6324','634_1005','6341_6277','6342_6298','6343_4759','6348_1182','635_636','6359_7189','636_637','6361_6367','6362_7276','6363_6361','6364_1558','6365_6276','6366_6230','6367_4985','637_638','638_639','639_640','64_65','640_641','641_642','642_643','643_644','644_645','645_1796','649_650','65_66','650_651','651_4472','653_722','654_655','655_656','656_657','657_658','658_659','659_604','66_4498','664_614','664_665','665_666','666_667','667_668','668_614','669_670','670_671','671_672','672_4382','675_1184','675_1500','675_303','675_4380','675_4415','675_620','675_621','677_678','678_679','679_680','68_30','680_681','681_682','682_683','683_684','684_685','685_686','686_688','688_689','689_690','69_7686','69_828','690_5062','691_692','692_693','692_694','692_695','693_585','694_695','695_696','696_697','697_698','698_699','699_700','7_11','7_194','700_701','7009_1604','701_702','7010_7703','7011_2171','7011_7160','7012_2201','7012_2644','7013_6162','7014_4315','7015_4617','7016_4619','7017_7018','7018_7030','702_703','7020_7029','7021_6111','7021_7163','7022_1554','7023_7014','7025_4464','7027_4538','7028_5147','7029_7038','703_704','7030_6111','7030_7021','7031_4897','7038_7011','7039_7204','704_705','7043_1424','7044_993','7047_7161','7048_7699','7049_2236','705_706','7050_7185','7053_2088','7056_4731','7057_4847','7057_5142','706_567','706_707','7062_6075','7063_1137','7064_7066','7065_7063','7066_2924','7067_2956','7067_2984','7068_2966','7069_7443','707_708','7070_7445','7073_7097','7074_3919','7075_3925','7078_1449','7079_7080','708_731','7080_7081','7081_7565','7082_7083','7083_7084','7084_7108','7085_7086','7086_7087','7087_7088','7088_7089','7089_7091','709_532','7091_3910','7092_110','7097_7098','7098_7099','7099_7100','71_72','710_711','7100_7101','7101_7505','7102_7103','7103_7104','7104_7105','7105_7106','7106_6005','7107_7102','7108_3935','711_712','7110_3938','7111_6365','7112_972','7113_127','7114_7116','7115_6310','7116_3694','7117_7118','7118_6289','7119_7127','712_713','7120_7119','7121_7183','7122_7121','7123_4347','7124_7343','7125_7344','7126_7124','7127_7143','7128_7117','713_714','7130_1237','7132_1558','7132_6230','7133_7134','7134_3867','7135_7136','7136_3874','7137_7139','7138_3370','7139_4623','714_715','7140_7141','7141_7138','7142_7015','7143_3368','7143_7168','7144_3881','7145_3884','7145_4843','7149_7277','715_716','7153_6056','7154_7153','7156_7020','7158_7048','7159_7017','7159_7388','716_717','7160_7700','7161_7162','7163_1893','7166_1847','7167_1808','7168_4616','7169_3743','717_718','7170_3740','7172_3775','7173_3811','7175_3806','7176_2836','7177_2740','7178_2765','718_719','7180_7123','7180_7331','7183_4880','7184_2225','7184_7050','7185_7186','7186_7187','7187_3947','7187_3954','7187_4001','7188_3082','719_720','7191_6238','7192_4683','7193_7049','7194_7196','7195_3653','7196_6310','7197_7195','72_5144','720_680','720_721','7201_2138','7204_3728','7204_3736','7207_7288','7208_7249','721_680','721_693','7214_4347','7216_2501','7217_7227','7218_7031','7219_7218','722_723','7222_6170','7224_1581','7227_6172','7229_3418','7229_4617','723_724','7230_4617','7231_7128','7233_4573','7234_4002','7236_7245','7237_7241','7238_7224','7239_4401','724_725','7240_1891','7241_7247','7242_7238','7243_7126','7245_6317','7246_6320','7247_7243','7248_7207','7248_7254','7249_7253','725_726','7250_7248','7250_7252','7251_4051','7252_7248','7253_7251','7254_7257','7255_7249','7256_7255','7257_7258','7258_7260','7259_7256','726_727','7260_7263','7261_7259','7262_7261','7263_7264','7264_7268','7265_7262','7266_7265','7268_7270','727_728','7270_6122','7270_7266','7271_5136','7272_4956','7276_7149','7277_6363','7278_7289','728_729','7280_7281','7281_7284','7282_7278','7283_7282','7284_7287','7286_7283','7287_7208','7288_7286','7289_7280','729_730','7290_5016','7291_3485','7292_3818','7293_1335','7294_4132','7294_4156','7296_4265','7296_4266','73_74','730_5160','731_710','7313_6164','7314_3964','7317_3224','7318_4978','7318_4980','7319_7342','7320_1486','7325_3317','7325_4984','7326_7418','7327_3629','7328_3651','7329_3659','733_6015','7330_2051','7331_2559','7333_783','7334_1838','7340_783','7342_7125','7343_4978','7343_4980','7343_7318','7344_2510','7344_7334','7347_3669','7348_3669','7348_3671','7349_1631','7350_1558','7350_6230','7352_4246','7353_4571','7353_7584','7354_1975','7358_1231','7358_1235','7358_5174','7361_7362','7362_3129','7362_3134','7363_4588','7364_7365','7365_4210','7366_7367','7367_7368','7368_3143','7370_7339','7374_1845','7375_3466','7379_4903','738_739','7380_1676','7381_4742','7382_7381','7386_4607','7388_7017','7389_4324','7389_7025','739_740','7391_493','7392_1443','7392_312','7395_6124','7396_5145','7397_5140','7397_6252','7398_6252','7398_7397','7399_7398','7399_7611','74_4371','740_741','7400_3669','7401_3665','7402_4380','7402_4717','741_742','7412_7413','7413_2383','7414_1396','7415_7416','7416_7325','7417_3471','7418_7417','7419_7420','742_743','7420_3345','7421_7422','7422_6085','7423_1566','7424_7423','7425_7424','7426_7427','7427_7428','7428_138','743_744','7430_177','7431_175','7432_3454','7433_4677','7435_2722','7435_4413','7438_1410','7439_7454','744_745','7440_2940','7441_2933','7442_7446','7443_7444','7444_2941','7445_6339','7446_7070','7449_2941','745_600','7453_1478','7453_1480','7453_2001','7454_1380','7455_7438','7458_4479','7458_7425','746_844','7460_4930','7461_4230','7461_4281','7462_7461','7469_1724','747_748','747_842','747_843','7471_1533','7472_3230','7473_2679','7475_6107','748_7390','748_749','749_494','749_750','749_751','75_4501','750_751','7505_5071','7505_7678','7507_2663','751_752','7510_2207','7510_4799','7513_807','7514_2279','7516_6079','752_7339','752_753','753_7339','753_7370','753_754','754_755','755_756','7550_2570','7551_4800','7552_924','7555_4962','7556_3539','7559_7073','7559_7107','756_757','7560_7679','7563_3368','7563_7231','7564_340','7564_4521','7565_7082','7567_7326','757_758','7571_207','7574_4177','7577_1353','7578_7582','7579_1285','758_759','758_760','7581_1283','7582_297','7582_299','7582_319','7582_340','7582_407','7584_3344','7585_7587','7586_7587','7587_303','7587_328','7587_345','7587_407','7587_496','7587_7588','7588_281','7588_325','7588_328','7588_329','7588_4717','759_760','7591_496','7592_3815','760_2007','760_761','760_762','7601_2508','7602_22','7602_85','7603_45','761_762','7611_7398','7612_2811','7616_5081','762_763','7622_297','7622_334','7623_2508','7623_3676','7623_3715','7623_3719','7623_5030','7623_6312','7623_7401','763_2007','763_4952','763_4953','763_767','7639_7640','7640_3134','7640_5047','7641_7642','7642_7643','7643_3214','7644_7645','7645_7646','7646_3238','7654_4843','7660_3114','7661_1521','7667_3091','767_768','7671_513','7672_826','7673_7674','7674_7675','7675_7676','7676_7677','7677_7559','7678_7560','7679_7680','768_2085','768_769','768_770','768_772','7680_7681','7681_7738','7683_1582','769_770','7690_3781','7691_3721','7692_6010','7698_7113','7699_7159','77_78','770_771','770_773','7700_7047','7701_7010','7702_7009','7703_5147','7704_3983','7705_6251','7706_7707','7707_512','7708_7709','7709_7710','771_772','7710_7711','7711_7705','772_417','772_773','772_775','773_774','7738_7683','774_775','774_906','775_776','775_906','776_777','777_779','777_906','7771_3966','779_780','78_79','780_781','781_782','782_783','783_784','783_785','784_785','785_786','786_791','786_792','786_793','79_80','790_7585','791_7585','792_320','792_7586','793_274','793_320','793_495','793_7586','795_796','796_56','796_57','796_797','797_798','798_799','799_800','8_12','80_82','800_801','801_802','802_1654','802_803','803_804','804_7513','807_808','808_809','809_810','81_817','810_811','811_812','812_813','813_814','814_81','817_818','818_819','819_261','819_263','819_264','819_265','819_4725','82_817','82_83','820_821','821_63','822_823','823_824','824_825','825_826','826_827','827_71','828_829','829_830','83_84','830_831','831_75','832_833','833_77','834_835','835_836','836_837','837_1506','838_839','839_840','84_51','840_841','841_49','842_1014','842_845','842_846','843_1013','843_845','844_846','845_846','846_7340','846_847','846_848','847_848','848_2795','848_758','848_759','848_849','849_850','85_203','85_86','850_851','851_852','852_853','853_854','854_855','855_2815','855_856','856_857','857_858','858_859','859_860','86_87','860_861','861_862','862_3014','87_88','875_876','876_3011','876_878','877_876','878_879','879_880','88_89','880_881','881_882','882_883','883_884','884_885','885_886','886_887','887_2900','887_903','89_27','893_894','894_1067','895_1025','896_897','897_893','90_91','903_904','904_906','905_4331','906_907','906_909','907_908','908_909','909_786','909_790','909_792','91_129','91_92','910_911','911_912','912_913','913_914','914_938','915_916','916_917','917_918','918_919','919_7551','92_93','924_925','925_927','927_938','928_588','928_929','929_930','93_94','930_931','931_587','938_940','94_102','94_126','940_941','941_942','942_943','943_3599','944_945','945_947','946_1155','947_948','948_949','949_928','950_915','956_957','957_958','958_959','959_960','960_961','961_962','962_635','969_1176','972_1625','980_981','981_1518','981_1576','983_984','984_985','985_1529','987_4937','992_7044','993_5060']

In [ ]:
df = df.loc[df['segment'].isin(gtfs_segments)]

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After removing legacy segments, the dataframe contains", rows, "rows and", cols, "columns.")
print()

## 9.6 Rain Binary Feature

Create a binary feature that has value 1 when there is rain, and value 0 otherwise:

In [ ]:
df.loc[df.rain == 0, 'rain_binary'] = 0
df.loc[df.rain != 0, 'rain_binary'] = 1

## 9.7 Peak Time Feature

Create a feature that has value 1 during peak traffic times, and value 0 otherwise (based on https://www.irishexaminer.com/breakingnews/ireland/rush-hour-now-lasts-for-six-hours-each-day-842746.html):

In [ ]:
df.loc[(df.hour.isin([7,8,9,16,17,18]) & df.day_of_week.isin([0,1,2,3,4])), 'peak'] = 1
df.loc[~(df.hour.isin([7,8,9,16,17,18]) & df.day_of_week.isin([0,1,2,3,4])), 'peak'] = 0

## 9.8 Mean Feature

We will use mean/target encoding for the **time_diff** feature as it has such high cardinality.

In [ ]:
means = df.groupby('segment')['time_diff'].mean().round()

In [ ]:
df['segment_mean'] = df['segment'].map(means)

## 9.9 Median Feature

We will create a feature that has the median time_diff of each segment:

In [ ]:
medians = df.groupby('segment')['time_diff'].median().round()

In [ ]:
df['segment_median'] = df['segment'].map(medians)

## 9.10 Standard Deviation Feature

We will create a feature that has the standard deviation of time_diff for each segment:

In [ ]:
stds = df.groupby('segment')['time_diff'].std().round()

In [ ]:
df['segment_std'] = df['segment'].map(stds)

## 9.11 Update Data Types for the New Features

In [ ]:
# Select columns containing categorical data
categorical_columns = df[['dayofservice', 'tripid', 'lineid', \
                         'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next',\
                          'month', 'day', 'hour', 'record_date', 'day_of_week', 'weekday', 'bank_holiday', 'segment', \
                         'rain_binary', 'peak']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df.select_dtypes(['float64', 'int64']).columns

# 10. Data Integrity Checks

## 10.1 Deal with Outliers

Make a list of all segments:

In [ ]:
# make a list of all segments
segments = ['10_12','100_1111','1005_1788','1013_1015','1014_1015','1015_1016','1015_1074','1015_2812','1016_1017','1017_1018','1018_1019','1019_1020','102_6239','1020_1076','1021_6042','1022_1023','1023_1024','1024_895','1025_1026','1026_1027','1027_1028','1028_2818','1035_1314','1036_4709','1037_1038','1038_2998','104_6241','1042_1043','1043_1044','1044_1045','1045_4710','1046_1047','1047_1048','1048_1049','1049_4333','105_6253','1051_1052','1052_1053','1053_1054','1054_1036','1055_2868','1062_1063','1063_1064','1064_1065','1065_1066','1066_1067','1067_1068','1068_1069','1069_1070','107_6125','1070_1071','1071_4528','1072_1352','1072_7577','1073_3641','1074_6074','1076_1021','1076_1077','1077_1078','1077_2479','1078_1079','1079_1080','1080_1081','1081_1082','1081_2914','1082_1083','1083_1085','1085_1086','1085_1121','1086_1087','1087_1088','1088_1089','1089_1090','1090_1091','1090_2413','1091_1092','1092_1093','1093_1094','1094_1095','1095_1096','1096_1097','1096_1101','1097_1099','1099_6132','11_14','110_112','1100_100','1101_1102','1102_1103','1103_1104','1105_1107','1107_1108','1108_1109','1109_1110','1110_1111','1111_1112','1111_2323','1111_2404','1112_1113','1113_1114','1114_1115','1115_2437','1117_1118','1118_1119','1119_1120','112_113','1120_1164','1121_1122','1122_1123','1123_1124','1124_1125','1125_1127','1127_1130','113_114','1130_1140','1130_2550','1135_7064','1137_5131','114_115','1140_1141','1140_2515','1141_1142','1142_1143','1143_1144','1144_1145','1145_4869','115_37','1150_1151','1151_1152','1152_1153','1153_1154','1154_946','1155_1157','1157_1158','1158_1159','1159_1160','116_117','1160_1161','1161_1162','1162_1163','1163_1164','1163_1337','1164_1165','1165_1166','1166_1167','1167_1168','1168_1169','1169_1170','117_118','1170_1069','1171_1172','1172_1173','1172_286','1172_4509','1172_7671','1173_1174','1174_1175','1174_4725','1175_15','1176_1177','1177_1178','1178_1179','1179_5060','1179_7112','118_119','1181_7350','1182_1550','1184_497','1185_1186','1186_1187','1187_1188','1188_216','119_44','1190_1191','1191_1192','1192_1193','1193_1194','1194_1195','1195_664','1196_1197','1197_1198','1198_1199','1198_1222','1198_4782','1199_1200','12_14','1200_4385','1201_1202','1201_4695','1202_1203','1203_6115','1205_1206','1206_1207','1207_1208','1208_1209','1209_1210','1210_1211','1210_3575','1211_3575','1212_1213','1213_1214','1214_1215','1215_1216','1216_1217','1217_4563','1218_1258','1218_1270','1219_1220','1220_1221','1221_664','1222_1223','1223_1224','1224_1225','1225_1226','1226_1227','1227_1228','1227_1253','1228_1244','1229_1231','1231_1232','1231_1235','1232_1233','1233_1234','1234_7358','1235_1251','1237_1238','1238_1239','1239_247','1240_1241','1241_1242','1242_1277','1243_4696','1244_1246','1244_254','1245_4693','1246_1229','1247_1248','1248_1273','1249_1247','1250_4698','1251_253','1251_7130','1253_969','1254_1255','1255_1256','1256_4694','1258_1259','1259_1260','126_6182','1260_1262','1260_6130','1262_4692','1265_6115','1266_1267','1267_1268','1268_1269','1269_1265','127_112','1270_1272','1272_1273','1273_1274','1274_1275','1275_1276','1276_1277','1277_1219','1278_1282','1278_1934','1279_1282','1279_1283','1279_1934','1280_6348','1280_6366','1281_1329','1282_4456','1283_4456','1283_7579','1284_1285','1285_1016','1285_1286','1285_1287','1286_1288','1287_1288','1288_1289','1289_1290','1289_1360','129_130','1290_1291','1291_1292','1291_2389','1292_1293','1293_1294','1294_1295','1295_1296','1296_1297','1297_1298','1298_1121','1298_1299','1299_1300','130_131','1300_1301','1301_1302','1302_1303','1303_1304','1303_5129','1304_1305','1305_1306','1305_1320','1306_1307','1307_1308','1308_1309','1309_1310','131_164','1310_1311','1311_1312','1312_1313','1313_1055','1314_1316','1315_1317','1316_1315','1317_1318','1318_1319','1319_1281','1320_1321','1321_1322','1322_1323','1323_1324','1324_7068','1325_1326','1326_1327','1327_1328','1328_1329','1329_1330','133_134','1330_1331','1330_2919','1331_1332','1332_1333','1333_1334','1334_1335','1334_2985','1334_7293','1335_1336','1336_1337','1337_1338','1338_1339','1339_1340','134_4890','1340_1341','1341_1342','1342_1343','1343_1344','1344_1345','1345_1347','1347_1348','1347_2634','1348_1349','1349_1350','1350_1352','1352_1353','1353_1354','1353_1355','1354_7578','1355_1357','1357_1358','1357_1359','1358_298','1358_319','1358_326','1358_340','1358_342','1358_4494','1358_5192','1358_7564','1358_7665','1359_274','1359_279','1359_319','1359_320','1359_4724','136_137','1360_1362','1362_1363','1363_1365','1365_1365','1365_1366','1366_1367','1367_1369','1367_7455','1369_1370','137_193','1370_1371','1371_1372','1371_1454','1372_1373','1373_1374','1374_1375','1374_1946','1375_1376','1376_1377','1377_1378','1378_1379','1378_4514','1379_1380','138_133','138_167','1380_1381','1381_1382','1382_1383','1383_1384','1384_1385','1385_1348','1386_1387','1387_1388','1388_1389','1389_3355','1396_2317','1397_1398','1398_1399','1399_5148','14_15','14_53','1401_1402','1402_1403','1403_1404','1404_1405','1405_4856','1406_1407','1407_1409','1409_2095','141_142','1410_1411','1411_1412','1412_1413','1413_1414','1414_1415','1415_1417','1417_1418','1418_5149','1419_6094','142_143','1423_1424','1424_1425','1424_2186','1424_2190','1425_1426','1426_1427','1426_1463','1427_1428','1428_1429','1429_1430','143_115','1430_1432','1432_1433','1433_1434','1434_1435','1435_7439','1436_3952','1438_4855','1439_1440','1440_1386','1441_1442','1442_7414','1443_1444','1443_1616','1443_1647','1444_1445','1445_1447','1445_1647','1445_4319','1445_4407','1445_7078','1447_2637','1449_1450','1450_1451','1451_1452','1451_2191','1452_2640','1453_1456','1454_1455','1455_1453','1456_1457','1457_1458','1458_1459','1459_1460','146_147','1460_1462','1462_5149','1463_4399','1464_1469','1465_1466','1466_1467','1467_1468','1468_1377','1469_1465','147_184','1471_1472','1472_4408','1473_1474','1474_1476','1476_7453','1478_1479','1478_315','1479_315','1479_7622','148_149','1480_1358','1480_2912','1481_7320','1482_1494','1484_1481','1485_4518','1486_1487','1487_1488','1488_1489','1489_1490','149_150','1491_1492','1492_1493','1493_1482','1494_1495','1495_1496','1496_1515','1499_1485','15_17','15_834','150_27','1500_302','1505_4509','1506_1507','1507_1508','1508_1509','1509_1510','151_153','1510_1511','1511_1512','1511_1592','1512_1538','1514_980','1515_1499','1517_1678','1518_1519','1519_1520','1520_7661','1521_4546','1523_1524','1524_1525','1525_1526','1526_6080','1528_1710','1529_4542','153_154','1531_1532','1532_1533','1533_1534','1534_1535','1535_1536','1536_1537','1537_186','1538_4543','1538_4548','1539_1540','154_155','1540_7426','1546_7072','1546_7560','1547_1548','1548_1280','1549_1523','155_156','1550_1551','1551_1552','1552_1553','1552_332','1553_7023','1554_1555','1555_1556','1556_1557','1557_1181','1558_1549','1558_2957','1558_4542','1559_1560','1559_987','156_157','156_7430','1560_1280','1566_1567','1567_173','1567_4372','157_158','1573_1574','1574_1575','1575_4583','1576_1577','1577_1578','1578_4364','1579_1580','1579_7673','158_159','1580_7237','1581_1582','1582_4793','1584_1585','1584_983','1585_1586','1586_1587','1586_1600','1587_1591','1587_4539','159_1539','1591_4542','1592_1593','1593_1594','1594_1595','1595_1596','1596_4363','1597_1599','1598_1597','1599_1573','16_73','1600_7701','1604_1573','1606_1607','1607_1608','1608_1609','1609_1610','1610_7471','1614_1615','1615_1478','1616_1617','1617_1618','1618_1619','1619_195','1620_220','1622_1623','1623_1624','1624_1625','1625_1626','1626_1627','1627_1628','1628_1629','1629_1630','1629_6234','1630_3670','1630_7347','1630_7348','1631_1632','1632_5053','1633_1634','1634_1635','1635_1636','1636_1637','1637_1638','1638_1639','1638_992','1639_1640','164_165','1640_231','1641_1642','1642_213','1643_7571','1647_1648','1647_1649','1648_1649','1649_1911','165_166','1654_1655','1655_1656','1656_1657','1657_4458','1658_1659','1659_1660','166_7458','1660_1661','1661_1662','1661_4523','1662_1664','1662_1666','1664_1665','1665_1666','1666_1667','1666_1807','1666_1808','1667_1668','1668_1669','1669_1670','167_168','1670_1671','1671_1672','1672_1673','1673_1674','1674_1675','1675_1849','1675_7380','1676_1517','1678_1679','1679_4520','168_169','1680_1681','1681_1682','1682_1683','1683_1684','1684_1685','1685_1686','1686_1687','1687_1688','1688_1689','1689_1690','1689_4899','169_141','1690_1691','1691_1692','1692_1693','1693_1694','1694_1695','1695_1696','1696_1697','1696_1700','1697_1698','1698_1699','1699_1700','17_18','1700_1701','1700_1906','1701_1702','1702_16','1702_1703','1703_1704','1703_1905','1704_1705','1705_1706','1706_1707','1707_1708','1708_1709','1709_1528','1710_1711','1711_1712','1712_1713','1713_1714','1713_1715','1714_1715','1715_1478','1715_7453','1719_1828','1724_1725','1725_1726','1726_1727','1727_1728','1728_1729','1729_1730','1729_1731','173_174','1730_1731','1731_1732','1731_1738','1732_1733','1733_1734','1734_1735','1735_1736','1736_1737','1737_1738','1738_1739','1738_614','1739_614','174_175','1740_1741','1741_1742','1741_1747','1742_1743','1743_1744','1744_1745','1745_1746','1746_1747','1747_1748','1747_1749','1748_1749','1749_1751','1749_544','175_176','1750_1769','1751_1752','1752_1753','1753_1754','1754_1759','1755_1756','1756_1757','1757_1794','1758_1750','1759_1758','176_181','1760_1761','1761_1762','1762_1763','1763_1764','1764_1765','1765_1767','1769_1770','177_178','1770_1771','1771_1760','1772_1773','1773_1774','1774_1775','1775_1776','1776_1777','1777_1778','1778_1755','178_1539','1782_1783','1783_1784','1784_1785','1785_1786','1786_624','1788_1789','1789_7199','179_7431','1791_1792','1792_1793','1793_962','1794_7469','1795_1806','1796_1797','1797_1798','1798_1799','1799_1800','18_19','1800_1801','1801_1277','1802_1795','1803_77','1804_1803','1805_1662','1805_1806','1806_1660','1807_7167','1808_1813','1808_1816','1808_2961','1808_4744','1808_7389','1808_7475','181_182','1813_1814','1814_1816','1814_1852','1816_1817','1816_6171','1817_1818','1818_1819','1819_1822','182_183','1820_2468','1822_4994','1826_1827','1827_7382','1828_1829','1829_4997','183_184','1833_1834','1834_1820','1834_1835','1835_1836','1836_1837','1837_2338','1838_2338','184_185','1840_1841','1841_1842','1841_4314','1842_7374','1844_6172','1845_1696','1845_7166','1847_1696','1849_1851','185_186','1850_1867','1851_1813','1852_1853','1853_1854','1854_1844','1858_1859','1859_1860','186_187','186_838','1860_4492','1861_1862','1862_1863','1863_1864','1864_1865','1865_1866','1866_1850','1867_4489','1869_1870','187_188','1870_1871','1871_1872','1872_1873','1873_1874','1874_1875','1875_1876','1876_1877','1877_1878','1878_1879','1879_1899','188_189','1882_7379','1883_1884','1884_1902','1887_1888','1888_1889','1889_1890','189_190','1890_7240','1891_7156','1893_1894','1894_1912','1895_1896','1896_1897','1897_6088','1898_1858','1899_1887','1899_6107','19_21','190_1614','190_191','190_192','1902_1903','1903_1904','1904_1840','1905_1804','1905_1906','1906_1907','1906_1909','1907_1908','1908_1909','1909_1713','191_819','1911_1805','1911_1913','1911_802','1912_1895','1913_1914','1914_1805','192_795','192_819','193_138','1934_1935','1934_2310','1934_2385','1935_1445','1935_1937','1937_1938','1938_1939','1939_1940','194_195','1940_1941','1941_1942','1941_4516','1942_1943','1943_1944','1944_1945','1945_1946','1946_1947','1947_1948','1947_2642','1948_1949','1949_1950','195_196','1950_1951','1951_1952','1952_1953','1953_1954','1954_1955','1955_1956','1956_1957','1956_2109','1957_1958','1958_1959','1959_1960','196_197','1960_4379','1963_4549','1964_4674','1965_1966','1966_1975','1968_1969','1969_1970','197_198','1970_1971','1971_2114','1971_2160','1972_1973','1973_1974','1974_1975','1975_1977','1975_1982','1977_1978','1978_1980','198_199','1980_1981','1981_1982','1981_4406','1982_2642','1982_4406','1982_7435','1984_1985','1985_1986','1986_1987','1987_1988','1987_1989','1988_1989','1988_2642','1989_1375','1989_1472','1989_1990','1989_1995','199_1506','199_200','1990_1992','1992_1993','1993_1994','1994_1995','1995_1996','1995_1998','1996_1997','1997_1998','1998_1999','1998_2001','1998_2002','1999_2001','2_192','2_4726','2_4962','200_201','2001_2002','2002_1358','2002_1359','2003_1358','2007_2008','2007_4571','2008_2009','2008_2071','2009_2010','201_148','201_202','2010_435','2011_2012','2012_3322','2013_2014','2014_2016','2014_4636','2015_2017','2016_2015','2016_2017','2017_2018','2017_3259','2017_7362','2018_2019','2019_2020','202_151','2020_2021','2021_2022','2022_4565','2023_2024','2024_2025','2025_6086','2026_7226','203_204','2031_2032','2032_2033','2033_2034','2034_2035','2035_2036','2036_2039','2036_2040','2039_2040','204_205','2040_2041','2040_3047','2041_2042','2041_3071','2042_2043','2043_2044','2044_2045','2045_2046','2046_4568','205_1620','205_206','205_215','2051_6086','2053_2054','2054_2055','2055_4565','2055_4732','2056_2057','2057_2058','2058_2059','2059_2060','206_207','2060_2061','2060_2062','2061_2062','2062_2063','2062_2064','2063_2064','2064_2065','2064_4727','2065_4727','2066_2067','2067_4728','2068_2069','2069_2070','207_208','2070_2084','2071_2072','2072_2073','2073_2074','2074_2075','2075_2076','2076_460','208_209','2080_2081','2081_2082','2082_2083','2083_2084','2084_768','2085_2086','2086_4455','2086_479','2087_7053','2088_2089','2089_763','2089_770','209_210','2094_1406','2095_2096','2096_2097','2097_2099','2099_2101','21_7602','21_85','210_1643','210_211','2101_2102','2102_2103','2102_2726','2103_2332','2103_7154','2105_2728','2105_6140','2109_3427','211_212','2114_2115','2115_6011','2118_2119','2119_2120','212_213','2120_2121','2121_2123','2123_2685','2124_2125','2125_2126','2126_2127','2127_2128','2128_2129','2129_4797','213_214','2133_6244','2134_2135','2134_3378','2135_2164','2136_2137','2137_2138','2137_7201','2138_2139','2139_2140','214_4432','2140_2141','2141_2142','2142_2143','2143_2145','2145_4670','2146_2147','2147_2148','2148_2149','2149_2150','215_216','2150_2151','2151_2152','2152_2153','2153_2154','2154_6245','2156_6014','2158_2159','2159_4409','216_217','2160_2161','2160_4665','2161_4665','2162_2163','2163_2168','2164_2165','2165_2166','2166_2167','2166_4545','2166_4554','2166_7229','2167_3383','2168_4536','2169_2170','217_218','217_242','2170_4672','2171_7160','2172_2159','2172_4409','2173_2174','2174_3425','2176_1982','2176_2177','2177_2178','2178_7354','218_219','2181_2182','2182_2778','2183_2184','2184_2185','2185_1423','2185_7043','2186_2187','2187_2188','2188_2189','2189_1436','219_220','2190_4434','2191_2192','2192_2193','2193_2194','2194_2195','2195_2196','2196_2197','2197_2198','2198_2200','22_23','220_1622','220_221','2200_2201','2201_4401','2201_7239','2202_2203','2203_2204','2204_4888','2205_2687','2206_4798','2207_2208','2208_2210','221_222','2210_2211','2211_2241','2213_2214','2213_4688','2214_2215','2215_2216','2215_7184','2216_4599','2217_2218','2218_2219','2219_2220','2219_3365','222_223','2220_2221','2221_2223','2223_2224','2224_3886','2225_4620','2227_7049','2229_2230','223_224','2230_2231','2231_2233','2232_2234','2233_2232','2234_2235','2235_2236','2235_2239','2236_2238','2236_2239','2238_5056','2239_2241','224_225','2241_2242','2242_2243','2242_7435','2243_2244','2244_2245','2245_2246','2246_2247','2247_2248','2248_2249','2249_2250','225_226','2250_2251','2251_1472','2255_2256','2256_2717','2257_2258','2258_2259','2259_2260','226_228','2260_2261','2261_4469','2262_2264','2264_2265','2265_2269','2269_2270','227_230','2270_2274','2274_7601','2278_2279','2279_2720','2279_6251','228_229','2280_2281','2281_2282','2282_4468','2283_616','2289_1822','229_227','23_24','230_231','231_1641','231_232','2310_2311','2311_2312','2311_5025','2312_2313','2313_2314','2314_2315','2315_2094','2317_2318','2318_2319','2319_2320','232_233','2320_2321','2321_2322','2322_6163','2323_2324','2324_2325','2325_2326','2326_5150','2327_2328','2328_2329','2329_2330','233_234','2330_2331','2331_1397','2331_1424','2332_2333','2333_2334','2334_4662','2335_2336','2336_2337','2337_2339','2338_1840','2338_1845','2339_2340','2339_2420','234_235','2340_2601','2346_4435','2349_2351','235_236','2351_2558','2351_2621','2351_4445','2352_2573','2353_2357','2353_4643','2357_2358','2357_2359','2357_2566','2358_2359','2359_2360','2359_7122','236_1190','236_237','2360_4340','2360_4348','2369_2370','237_213','2370_2371','2371_2372','2372_2373','2373_1424','2373_2377','2374_2146','2377_2378','2377_2431','2378_2183','2379_2380','2380_2382','2382_5099','2383_1479','2383_2002','2383_2003','2385_2386','2386_2387','2387_2388','2388_1290','2389_2390','2390_2391','2391_2392','2392_2393','2393_2394','2394_1090','2394_2395','2395_2396','2396_2397','2397_2398','2398_2399','2399_2400','24_25','2400_2401','2401_2402','2402_2403','2403_1096','2404_2405','2405_2406','2406_2408','2408_2409','2409_2410','2410_2411','2411_2412','2412_2438','2413_2414','2414_2415','2415_2416','2416_2417','2417_2418','2418_2419','2419_2452','242_243','2420_2421','2421_2423','2423_2424','2424_2447','2425_2426','2426_2427','2427_2428','2428_2429','2429_2430','243_244','243_254','2430_2371','2431_2432','2432_2433','2433_2327','2433_2434','2434_2435','2435_2436','2436_2437','2437_1117','2437_2438','2438_2439','2439_2440','244_245','2440_2441','2441_2442','2442_2443','2443_2444','2444_1344','2445_2457','2446_2459','2447_5133','2448_5026','245_246','2450_2490','2452_2453','2453_2454','2454_2455','2455_2456','2456_2445','2457_4392','2458_2446','2459_2460','246_248','2460_2461','2461_2462','2462_2463','2463_2431','2465_2466','2466_2467','2467_2494','2468_7319','247_1240','2471_2472','2472_2473','2473_2474','2474_2475','2475_2476','2476_2477','2477_2478','2478_1170','2479_2480','248_249','2480_2481','2481_2482','2482_2483','2483_2484','2484_2485','2485_2486','2486_2487','2487_2488','2488_2489','2489_2448','249_250','2490_2491','2491_2492','2492_2493','2493_2465','2494_2495','2495_2496','2496_2497','2497_2471','2498_2499','2499_2500','2499_7216','25_27','250_251','2500_2501','2501_7623','2508_2278','251_252','2510_7242','2515_2516','2516_2517','2517_2518','2518_4749','252_255','2522_2524','2523_2532','2524_2523','253_5033','2532_4877','2535_2536','2536_2616','2538_2606','2539_2540','254_1246','254_244','2540_2542','2540_4882','2542_3004','2547_2548','2548_2600','2548_946','255_234','2550_2516','2550_2551','2551_2553','2553_2554','2554_2425','2554_2555','2555_2556','2556_2557','2557_4436','2558_2559','2558_6170','2559_2560','2560_2561','2561_2562','2562_2564','2564_4927','2566_2567','2567_2568','2568_2569','2569_2570','2569_7550','2570_2571','2570_7222','2571_2359','2573_2574','2574_2575','2574_2577','2575_4929','2577_2578','2578_2579','2579_7395','2583_2584','2584_2585','2585_2586','2586_2587','2587_2588','2588_2589','2589_2590','2590_2591','2591_2592','2592_2357','2594_2595','2595_2596','2596_2597','2596_2611','2597_2599','2599_2600','2599_4577','2600_1155','2601_4446','2602_2613','2603_2557','2604_2605','2605_2538','2606_2607','2607_2608','2608_2609','2609_2610','261_270','261_271','2610_2554','2611_2602','2613_2614','2614_2615','2615_2535','2616_2617','2617_2557','262_270','2621_2622','2622_2623','2622_2750','2623_2624','2624_4441','2626_2669','2628_2629','2629_6095','263_271','263_277','263_4508','263_4725','263_6071','2631_2359','2632_2633','2633_2369','2634_2635','2635_2636','2636_2383','2636_5097','2637_1956','2637_2638','2637_2721','2638_2640','2638_2721','264_6059','2640_2641','2641_1948','2641_2643','2642_2643','2642_7435','2643_2644','2644_2645','2644_2709','2645_2646','2646_2647','2647_2648','2648_2649','2649_2650','265_271','265_6059','2650_2652','2652_2653','2653_2654','2654_2655','2655_2656','2656_2657','2656_2668','2657_2658','2658_2659','2659_2660','2660_2661','2661_2662','2662_2909','2663_2664','2664_2665','2665_2666','2666_2667','2667_2696','2668_2672','2669_4641','2670_2910','2672_2673','2673_2206','2674_2675','2675_2124','2676_2118','2678_2681','2679_4691','2680_2676','2680_4677','2681_2680','2682_4796','2685_2686','2686_7510','2687_2688','2688_2689','2689_2696','2696_2697','2697_2700','2697_2713','27_28','270_335','270_336','2700_2701','2701_2702','2702_2703','2703_2704','2704_2706','2706_6070','2708_5134','2709_2710','271_1278','271_340','271_404','271_4521','271_496','2710_2711','2711_2712','2712_2655','2713_2714','2714_2715','2715_2716','2716_2718','2717_2257','2718_2719','2718_7435','2719_1989','2719_7435','272_335','2720_2280','2721_7012','2722_1471','2722_4413','2725_2737','2726_2727','2727_2105','2728_2729','2729_2730','273_405','2730_6083','2733_2734','2734_5169','2735_2736','2736_2725','2737_2738','2738_2739','2739_7177','274_1505','274_2','274_7','2740_2741','2741_2742','2742_2743','2743_2745','2745_2746','2746_2747','2747_2748','2748_7062','2750_2751','2751_2752','2752_2753','2753_2754','2754_2755','2755_2756','2756_2764','2764_7178','2765_2766','2766_2767','2767_2772','2768_2769','2769_2770','277_288','277_292','277_336','2770_2771','2771_2773','2772_2768','2773_2774','2774_2777','2775_2181','2777_2775','2778_2185','278_10','278_2','278_7615','278_8','279_3','279_7','2795_756','28_29','2804_376','2809_2810','2809_408','2809_7612','281_261','281_4','281_6','281_7','2810_2811','2810_7385','2810_7387','2810_749','2810_750','2810_842','2810_843','2811_749','2811_842','2811_843','2812_2813','2813_2814','2814_852','2815_2816','2816_2817','2817_2818','2818_2819','2819_2820','2819_3012','2820_2822','2822_2823','2823_5032','2824_2825','2825_2826','2826_2827','2826_2842','2827_2828','2828_2829','2829_2830','2830_2831','2831_2832','2832_2833','2832_4349','2833_6006','2833_7375','2835_7176','2836_2837','2837_2838','2838_2839','2839_2840','284_4504','2840_2841','2840_2845','2841_2865','2842_2843','2843_2844','2844_2845','2845_2846','2846_2847','2847_4397','2848_2849','2849_2850','2850_2851','2851_2852','2852_4707','2852_4987','2853_2854','2854_2855','2855_2856','2856_2857','2857_2858','2858_2859','2859_2860','286_271','2860_2861','2861_2829','2861_2862','2862_2863','2863_2864','2864_4486','2865_2866','2866_2825','2866_2892','2866_6044','2867_1035','2868_2892','288_1171','289_496','2892_2893','2893_2894','2894_2895','2895_2896','2896_2897','2897_1062','2897_2898','2898_3562','2899_884','29_4680','2900_2901','2901_2902','2902_2903','2903_1074','2906_2907','2907_2908','2909_2906','2909_6020','2910_2911','2911_7507','2912_1358','2914_2915','2914_2989','2915_2916','2916_2917','2917_2918','2918_1300','2919_2920','292_1171','2920_2921','2921_2922','2922_2923','2923_2924','2924_2925','2925_2948','2925_7442','2927_2929','2929_2930','2930_7441','2934_2935','2935_2936','2936_2937','2937_2938','2938_7440','2940_2948','2941_2942','2942_2943','2942_7065','2943_2944','2944_2945','2945_2946','2946_2947','2947_1305','2948_2949','2949_2950','2950_2954','2954_2955','2954_7067','2955_7067','2956_2965','2957_2958','2958_4938','2961_4901','2962_2963','2963_2964','2964_4175','2965_2967','2965_2983','2966_2967','2967_2968','2968_2969','2969_2970','297_407','297_496','2970_2971','2970_4988','2971_5051','2976_2977','2977_2978','2978_2979','2979_2980','298_496','2980_2981','2981_2954','2981_2991','2983_2984','2984_2927','2984_2941','2984_7449','2985_2986','2986_2988','2988_3001','2989_2990','299_497','2990_1042','2991_2992','2992_1325','2993_2994','2994_4180','2995_3154','2996_3258','2997_4153','2998_2999','2999_3000','3_12','3_192','30_69','300_496','300_497','3000_3001','3001_3002','3002_1166','3003_2522','3004_4756','3005_4757','3006_4871','3007_6283','3008_6089','301_336','301_4495','3011_2896','3012_877','3014_3015','3015_3016','3016_3017','3017_3018','3018_3019','3019_3020','302_333','302_340','302_4495','3020_3021','3021_3022','3022_441','3023_3024','3024_3025','3025_3026','3026_3027','3027_3028','3028_3029','3029_3030','3030_3031','3031_875','3032_3033','3033_3034','3034_3036','3034_3114','3036_3037','3037_3038','3038_3039','3039_3040','3040_3041','3041_2035','3041_3042','3042_2035','3045_4150','3047_3202','3048_3049','3049_3050','3050_3051','3051_3052','3052_3053','3053_3054','3054_3055','3055_3056','3056_3057','3057_3059','3057_4830','3059_3060','3060_3061','3061_3062','3062_3063','3063_3064','3064_3065','3065_3066','3066_3067','3067_3202','3068_3069','3069_3070','307_308','3070_2041','3071_3072','3072_3073','3073_3074','3074_3075','3075_3076','3076_3077','3077_3079','3077_3084','3079_3080','308_309','3080_3081','3080_3082','3081_3084','3081_7188','3082_3084','3082_3114','3082_7660','3084_469','3085_3034','3088_3704','3089_3090','309_2383','309_7412','3090_3706','3091_3092','3092_3093','3093_3535','31_32','3112_3924','3114_3115','3114_4353','3115_3116','3116_3117','3116_6085','3117_3118','3118_3119','3119_3120','312_1444','3120_3122','3122_3123','3123_3124','3124_3125','3125_3126','3126_3127','3127_3128','3128_3130','3129_3130','3130_3131','3131_3132','3132_3133','3133_3134','3133_5046','3134_3135','3135_3136','3135_7363','3136_3138','3138_3139','3139_4124','3140_3141','3141_3142','3142_3143','3142_3224','3143_3144','3144_3145','3144_5047','3144_7639','3145_3146','3145_5128','3146_3147','3147_3148','3148_3149','3148_5127','3149_3150','315_313','315_335','315_336','315_404','315_406','315_4495','3150_3151','3151_3153','3152_2995','3153_3152','3154_3155','3155_3156','3156_3158','3158_3159','3159_3160','3160_3161','3161_3162','3162_3163','3163_3164','3164_3079','3164_3084','3164_7188','317_312','318_312','3181_451','3184_449','319_1443','319_274','319_278','319_279','319_281','319_7491','3196_3345','32_33','320_278','320_281','3202_3203','3203_3204','3204_3205','3204_4568','3204_7641','3205_3206','3206_3207','3207_3208','3208_3209','3209_3210','3210_4981','3211_3212','3212_3213','3213_3214','3214_3215','3215_3216','3216_3217','3217_3218','3218_3219','3219_3220','3219_3537','322_323','322_7697','3220_3221','3220_5046','3221_4982','3222_3223','3223_3224','3224_3225','3225_3226','3226_3227','3227_3228','3228_3229','3229_7472','3229_7644','323_332','323_6234','3230_3231','3230_3232','3231_3246','3232_3233','3233_3234','3234_4462','3235_3236','3236_3237','3237_3238','3238_3239','3239_3240','324_1555','324_1634','324_327','324_6235','3240_3048','3240_3068','3243_3244','3244_3245','3245_3212','3246_3247','3247_7056','325_1443','3258_2060','3259_7361','326_1443','327_6016','327_7113','328_1443','3280_3282','3282_3479','3283_3284','3284_3310','329_1443','33_7270','3310_3481','3317_3318','3318_3320','3320_2064','3321_2066','3321_4727','3322_3563','3328_3329','3329_3330','333_747','3330_3331','3331_3351','3332_3333','3333_3334','3334_3352','3335_3336','3336_3337','3337_3338','3338_3339','3339_3340','334_406','334_7581','3340_4324','3344_7421','3345_4727','335_1278','335_404','335_4521','3351_3332','3352_3335','3355_3356','3356_1441','336_1278','336_1279','336_7581','3365_3366','3366_3367','3367_3886','3368_4626','3368_7137','3370_7168','3370_7231','3372_3373','3373_3374','3374_2233','3376_3377','3377_3378','3378_3379','3379_4669','3383_3384','3384_3385','3385_3386','3386_3387','3387_4559','3389_3390','3390_6119','3392_3393','3393_3394','3394_3396','3396_3397','3397_3398','3398_3399','3399_6187','340_350','3400_3401','3401_3402','3402_3403','3403_3404','3404_3405','3405_3406','3406_3410','3407_3408','3408_3409','3409_4558','3410_3407','3411_3412','3412_3413','3413_3414','3414_3415','3415_3416','3416_3417','3417_3418','3418_3419','3419_2374','3419_3420','3419_4762','342_1279','342_4495','342_4521','3420_3421','3421_3422','3422_3423','3422_4671','3423_2156','3423_4409','3425_2176','3426_3434','3427_3428','3428_3429','3429_3432','3432_3426','3432_3434','3434_3435','3434_3453','3435_3453','3436_7432','3437_3439','3437_3440','3438_5022','3439_3440','3440_3441','3440_4557','3441_4560','3443_3444','3444_3445','3444_3447','3445_3446','3446_3447','3447_3448','3447_3454','3448_3449','3449_3436','345_318','345_320','3453_3437','3454_3455','3455_3456','3455_5022','3456_3457','3457_3438','346_317','3461_4532','3461_4537','3465_4673','3466_3467','3467_3469','3469_3470','3470_3471','3471_3472','3472_3473','3473_3474','3473_5015','3474_3475','3475_3476','3476_3477','3477_3478','3478_3280','3479_4086','3481_3482','3482_3483','3483_3484','3484_3485','3485_3486','3486_3487','3487_3488','3487_7415','3488_3489','3489_3492','3491_6007','3492_2835','3493_3494','3494_3495','3495_3496','3496_3497','3497_3498','3498_3499','3499_3500','35_36','350_2809','350_351','3500_3501','3501_3502','3502_3503','3503_3504','3504_3505','3505_3506','3506_3507','3507_3508','3508_3510','351_352','3510_3512','3512_3514','3514_3515','3515_3516','3516_3517','3517_3518','3518_3519','3519_3521','352_353','3521_3523','3523_3524','3524_3525','3525_3527','3527_3528','3528_3529','3529_3532','353_354','3532_3533','3533_3491','3535_3536','3536_3537','3537_3538','3538_7556','3539_3540','354_355','3540_3541','3541_3542','3542_3543','3542_3544','3543_3141','3544_3552','355_356','3552_3553','3553_3554','3554_3555','3555_3556','3556_3557','3557_7317','356_357','3562_2899','3563_444','3565_3825','3566_3567','3567_7292','3569_3571','357_390','3571_3801','3572_3701','3573_3662','3574_3641','3575_3576','3576_3577','3576_3648','3577_3578','3578_3579','3579_3580','3580_3640','3583_3584','3584_3585','3585_3587','3585_3625','3587_3588','3588_3589','3588_3590','3589_3591','3589_3605','3590_3589','3591_3592','3592_3593','3593_3594','3594_3595','3595_3596','3596_3597','3597_3598','3598_4465','3599_3600','3599_3610','36_37','3600_3601','3601_3602','3602_3604','3604_3616','3605_3606','3606_3607','3607_3608','3607_6051','3608_3609','3609_3598','3610_3611','3611_6135','3612_3613','3613_3615','3615_3616','3616_3618','3618_3619','3619_3620','3619_6087','3620_3621','3620_3622','3621_3622','3622_3623','3623_4387','3624_3587','3625_3626','3626_3627','3627_3628','3628_6052','3629_3630','3630_3631','3631_3632','3632_3633','3633_3634','3634_3635','3635_3636','3636_905','3639_3644','3640_3574','3641_3642','3642_3643','3643_3583','3644_3645','3645_3646','3646_3647','3647_3661','3648_3649','3649_3650','3650_7328','3651_3652','3652_3653','3653_3654','3653_6055','3654_5084','3655_5086','3656_3657','3656_7194','3657_3658','3658_3659','3658_7329','3659_3660','3660_3647','3661_3573','3662_1212','3663_7347','3664_3813','3665_7400','3669_3671','3669_7349','3669_7399','3669_7611','3670_3671','3671_3672','3672_3674','3674_3675','3675_3676','3675_5030','3676_3677','3676_5073','3677_3678','3678_3679','3679_3711','3679_5075','3682_3864','3686_5077','3689_3690','3689_3691','3690_5079','3690_6117','3691_3572','3691_3692','3692_3694','3694_3695','3695_3696','3695_7115','3696_3697','3697_3698','3698_3699','3699_3885','37_38','3701_3702','3702_3088','3704_3705','3705_733','3706_3707','3707_3708','3708_3709','3709_3710','371_391','3710_4925','3711_3712','3711_7189','3712_3713','3712_4916','3712_6359','3713_3714','3714_3715','3714_3719','3715_3755','3715_5175','3718_3720','3719_3718','372_373','3720_7691','3721_3722','3722_5069','3723_3724','3724_3725','3725_3726','3725_3738','3726_7039','3728_3729','3729_3730','373_374','3730_3732','3732_3733','3733_3734','3734_3735','3735_7039','3736_3738','3738_7170','374_375','3740_3741','3741_5165','3743_3780','3744_3715','3744_3745','3745_3746','3746_3749','3748_6050','3749_3750','375_2804','3750_3752','3750_6252','3751_3689','3751_7114','3752_3751','3755_3756','3756_3757','3757_3758','3758_3759','3759_3760','376_377','3760_3761','3761_3762','3762_3763','3763_3764','3764_3765','3765_3766','3766_3767','3767_3768','3768_3770','3769_3771','377_378','3770_3769','3771_3772','3772_3773','3773_3774','3773_5176','3774_7172','3775_3777','3777_3778','3778_7690','378_380','3780_3744','3781_3782','3782_3783','3783_3784','3784_3785','3785_3786','3786_3787','3787_3788','3788_3789','3789_3790','3790_3791','3791_3792','3792_3793','3793_3794','3794_3795','3795_3797','3797_3798','3798_3799','3798_6076','3799_6076','38_39','380_2087','3801_3802','3802_3805','3805_7175','3806_3807','3807_3808','3808_3809','3809_7174','381_382','3811_3812','3812_3664','3813_3814','3814_3815','3815_3816','3816_3566','3818_3819','3819_3820','382_4451','3820_3822','3822_3823','3823_3824','3824_3565','3825_3826','3826_3827','3827_3828','3828_3829','3829_3831','383_384','3831_3832','3832_3833','3833_3834','3834_3835','3835_3836','3836_3837','3837_3838','3838_3840','384_385','3840_3842','3842_3843','3843_3844','3844_3845','3845_3846','3846_3848','3848_3849','3849_3850','385_387','3850_3851','3851_3852','3852_3853','3853_3854','3854_3855','3855_3856','3856_3857','3857_3858','3858_3859','3859_3860','3860_3861','3861_3862','3862_3863','3863_3748','3864_3865','3865_4910','3865_7133','3867_7135','387_388','388_389','3881_5096','3884_3686','3885_1631','3885_3663','3886_3887','3886_3888','3887_3888','3888_3890','3888_7186','389_393','3890_3891','3891_3949','3891_7187','3892_3893','3893_3895','3895_3896','3896_3897','3897_3898','3898_3899','3899_3900','39_116','39_40','390_372','3900_3901','3901_3902','3902_3904','3904_3905','3905_3906','3905_7085','3906_3907','3907_3908','3907_3909','3908_3909','3909_5118','391_392','3910_3911','3911_3912','3912_3913','3913_3914','3914_3925','3914_7075','3917_3918','3918_3926','3918_5114','3919_3920','392_395','3920_3921','3921_3922','3922_3923','3923_3112','3923_7079','3924_3927','3925_3008','3926_7074','3927_3928','3928_3929','3929_7108','393_371','3935_3936','3936_3938','3936_7110','3938_3939','3939_3940','3940_3941','3941_3943','3943_3944','3944_3945','3945_3946','3946_4001','3947_3892','3949_3950','395_396','3950_2229','3950_3372','3952_2190','3954_3955','3955_3956','3956_3957','3957_3958','3957_3993','3958_3959','3959_3960','3959_7234','396_397','3960_3961','3960_6156','3961_3962','3962_3963','3963_3964','3964_3965','3965_3968','3966_5113','3968_3966','3968_7771','397_398','398_399','3980_6089','3981_3982','3982_3983','3982_7704','3983_3985','3985_3986','3986_3987','3986_7313','3987_3989','3989_3990','399_400','3990_3991','3991_3998','3993_3994','3994_3995','3995_3996','3996_3997','3996_3998','3997_3998','3998_3999','3999_4000','4_194','4_4962','4_7555','40_146','40_41','400_279','400_281','400_303','400_319','400_346','400_4522','400_7392','400_7564','400_7581','4000_4001','4001_3949','4001_7193','4002_5112','4003_4404','4004_4005','4005_4006','4006_4007','4007_4008','4008_4010','4009_4011','4010_4009','4011_5110','4012_4013','4013_4014','4014_4098','4015_4017','4017_4018','4018_4019','4019_4020','4020_4100','4021_4022','4022_4023','4023_4024','4024_4027','4027_7250','404_747','405_408','4051_4052','4052_4054','4054_4055','4055_4057','4056_4133','4057_4101','4058_4059','4059_4060','406_2810','406_4530','406_747','4060_4061','4061_4062','4062_4063','4063_4064','4064_4056','4065_4066','4066_4067','4067_4068','4068_4069','4069_4070','407_2498','4070_4071','4071_4072','4072_4073','4073_2583','408_409','4086_4087','4087_4088','4088_4089','4089_4090','409_410','409_411','4090_4091','4091_6097','4093_4094','4094_4096','4096_4114','4098_4015','41_42','410_411','4100_4021','4101_4102','4102_4058','411_412','411_414','4114_4115','4115_4116','4116_4117','4117_4118','4118_4119','4119_4120','412_413','4120_4121','4121_4122','4122_4123','4123_3283','4124_4125','4125_4126','4126_4127','4127_4128','4128_4129','4129_4130','413_414','4130_4131','4131_7294','4132_4134','4132_4171','4133_4065','4134_4135','4135_4136','4136_4207','414_415','414_416','4148_4149','4149_3045','415_416','4150_4151','4151_2997','4151_4156','4153_4154','4154_4416','4156_4168','416_417','4168_4170','417_418','417_485','4170_4132','4170_4153','4171_4172','4172_4173','4173_2962','4175_5091','4177_4178','4178_4179','4179_2993','418_419','4180_4181','4181_4182','4182_4151','419_420','42_43','420_2088','420_421','4201_4202','4202_4203','4203_4204','4203_5090','4204_4205','4205_4206','4206_3140','4207_4208','4208_4209','4209_4210','421_422','4210_4211','4211_4212','4211_4215','4212_4213','4213_5139','4214_4215','4215_4216','4216_4217','4216_4222','4217_4218','4218_4219','4219_4220','422_423','4220_4221','4221_4222','4222_4224','4224_4225','4225_4226','4225_7462','4226_4227','4227_4228','4228_4229','4229_4230','423_424','4230_4231','4230_4248','4231_7352','424_425','4246_4248','4248_4249','4248_4252','4249_4250','425_426','4250_4251','4251_4252','4252_4253','4252_4305','4252_6137','4253_4254','4253_4268','4253_4305','4254_6137','4256_4257','4257_4258','4258_4259','4259_7272','426_427','4260_4261','4260_7296','4261_7271','4262_4263','4263_4264','4264_7296','4265_4266','4266_4253','4266_4267','4266_4307','4267_4268','4268_4269','4268_4273','4269_4270','427_428','4270_4271','4271_4273','4273_4274','4273_4277','4274_4275','4275_4276','4276_4277','4277_4278','4277_7462','4278_4279','4279_4280','428_429','4280_4281','4281_4282','4282_4283','4283_4284','4284_4285','4284_4295','4285_4286','4286_4287','4287_4288','4288_4295','429_3032','429_6334','4295_4296','4296_4297','4296_4301','4297_4299','4299_4300','43_44','4300_4301','4301_4302','4302_4591','4302_5135','4303_4304','4304_4148','4305_4306','4306_4307','4306_4309','4307_4308','4308_4253','4308_4267','4309_4254','4314_1687','4315_4316','4316_4317','4317_7013','4319_1476','4320_1476','4321_7342','4323_1845','4324_4769','4325_3328','4330_3679','4331_4339','4333_4334','4334_4712','4336_1051','4339_7616','4339_7692','4342_4348','4344_2349','4347_2349','4347_7181','4348_4646','4348_7122','4349_3184','435_2011','435_7353','4350_5143','4353_4355','4354_3196','4355_4354','4362_2959','4362_7025','4363_1598','4364_1579','4369_6219','437_459','4371_832','4372_179','4376_4377','4377_4890','4379_1963','4379_1968','4380_1172','4380_496','4382_1185','4382_1196','4383_4699','4384_518','4384_519','4385_1201','4386_653','4387_3634','4388_4700','4392_2458','4397_2848','4399_4400','44_7603','4400_1464','4401_2202','4401_2213','4404_4405','4405_3991','4406_1984','4407_2637','4407_4320','4408_1473','4409_1972','4409_2173','4409_4550','441_442','4410_2678','4411_2239','4411_4715','4413_1476','4413_1480','4413_7453','4415_301','4415_302','4416_4201','442_443','443_444','4432_119','4433_3752','4434_2379','4435_2594','4435_2604','4435_2632','4436_5008','4438_3432','444_445','4440_2628','4441_2626','4445_2352','4446_2603','445_4779','4451_383','4455_381','4456_1284','4458_1658','4460_453','4462_3235','4464_1869','4465_944','4466_633','4468_2283','4469_2262','447_5045','4472_4386','4475_3023','4477_3321','4479_4376','448_3181','4481_4485','4485_822','4486_2865','4489_4747','449_450','4492_1861','4493_4362','4495_1278','4495_1279','4495_2809','4495_4522','4495_4720','4495_746','4495_7581','4498_1660','4499_4934','45_46','450_3181','4500_4481','4501_4935','4502_4936','4504_7028','4508_7402','4509_499','4509_7708','451_447','4513_272','4513_286','4514_4515','4515_1995','4516_1369','4518_509','4520_4825','4521_1283','4521_1934','4521_1935','4522_1934','4523_4524','4524_4322','4525_4526','4526_1700','4528_1072','453_4477','453_454','4530_748','4532_4534','4533_4844','4534_4535','4535_2169','4536_6003','4537_2176','4538_7396','4539_1606','454_4475','4542_1531','4543_1560','4545_4552','4546_1559','4548_1514','4548_980','4549_1964','4550_4551','4551_1965','4552_4554','4554_7016','4554_7229','4554_7230','4555_3418','4555_4576','4557_3441','4558_3411','4559_3389','4560_5126','4561_4629','4562_7386','4563_1218','4564_3443','4564_3444','4565_4566','4566_4567','4567_2031','4567_3238','4568_4569','4569_2023','4569_4570','4570_2056','4571_2013','4571_2014','4573_4893','4574_4575','4575_1680','4576_3418','4577_4578','4578_1100','4583_1518','4583_1576','4588_4589','4589_7364','459_2080','4590_7366','4591_4590','4592_980','4593_4594','4593_4595','4593_7516','4594_4595','4595_4596','4596_4563','4599_2217','4599_4601','4599_7184','46_47','460_2011','4601_5058','4602_4603','4603_4604','4604_4605','4605_4562','4606_7142','4607_4608','4607_7140','4608_4609','4609_4610','461_2068','4610_4611','4611_4612','4612_4613','4613_3370','4616_2227','4617_4555','4619_4561','4620_7563','4623_4619','4624_4625','4625_4628','4626_4627','4627_4624','4628_4623','4629_4630','4630_4631','4631_4632','4632_5164','4633_2234','4636_2016','4640_4347','4640_7214','4641_4642','4642_2355','4643_4644','4644_4645','4645_4440','4646_4647','4647_2346','4647_4435','4662_2335','4663_4664','4664_2160','4665_4666','4666_4667','4667_2162','4667_4668','4668_3376','4668_4677','4669_2146','4669_2165','4670_2165','4670_3420','4671_4672','4672_3465','4673_2172','4674_4663','4677_2136','4680_31','4680_90','4683_107','4688_4689','4689_4410','469_470','4690_4411','4691_2682','4691_4796','4692_1245','4693_1254','4694_1249','4695_1243','4696_4697','4697_1250','4698_4383','4699_4388','47_48','470_471','4700_1266','4705_475','4707_4708','4708_4336','4709_1037','471_472','4710_1046','4712_2853','4713_4714','4714_4960','4717_2499','472_473','4720_1443','4724_263','4725_273','4725_334','4725_4495','4725_6059','4725_6071','4726_7555','4727_4728','4727_768','4728_437','4728_461','473_4705','4731_2023','4732_3243','4742_4743','4743_4744','4743_4745','4744_4745','4745_4989','4746_1845','4747_1845','4747_1882','4749_4750','475_476','4750_3006','4750_4751','4751_4754','4754_4755','4755_3003','4756_3005','4757_4758','4758_6343','4759_4760','476_477','4760_2547','4762_4763','4763_6245','4765_4766','4766_4767','4767_4768','4768_1845','4768_4325','4769_4770','477_478','4770_4765','4779_7057','478_479','478_7053','4782_1782','4788_6273','479_480','4793_6068','4795_2686','4795_4688','4796_4690','4797_7473','4798_2674','4798_4715','4799_2205','48_49','480_481','4800_5157','4804_738','4806_5156','481_482','482_483','4825_7233','483_485','4830_7667','4843_4905','4844_4177','4847_4848','4848_4350','485_487','485_488','4854_1438','4855_1439','4856_4857','4857_307','4861_4862','4863_4864','4864_4865','4865_4577','4869_4870','487_488','4870_3007','4871_4872','4872_4873','4873_4874','4874_2532','4877_4878','4878_7180','488_489','488_491','4880_2539','4880_2606','4882_4883','4883_4884','4884_4885','4885_6343','4886_4887','4887_1150','4888_4798','489_490','4890_4369','4891_7458','4893_4894','4894_4895','4895_7217','4897_4898','4898_4574','4899_4900','49_51','490_491','4900_4746','4901_4902','4902_1672','4903_1883','4903_7219','4905_4906','4906_4907','4907_4908','4908_4909','4909_3884','4909_4921','491_492','491_493','4910_4911','4911_4912','4912_4913','4913_4914','4914_4915','4915_4957','4915_7654','4916_4917','4917_4918','4918_4919','4919_4920','492_493','4920_4910','4921_4922','4922_4923','4923_4924','4924_4958','4925_3679','4925_4926','4926_3678','4927_4928','4928_2352','4929_7460','493_494','4930_4931','4931_7459','4932_4933','4933_2590','4934_4500','4935_4502','4936_79','4937_1547','4938_1549','494_495','495_400','495_496','495_7587','495_7588','4952_768','4953_768','4957_7133','4958_3752','496_497','496_515','496_522','496_523','4960_4932','4962_264','4962_4725','4962_6059','497_515','4974_4976','4976_7126','4978_4979','4979_4986','4980_2289','4981_3211','4984_448','4985_6219','4986_7107','4987_2978','4988_2853','4989_1845','4989_4990','499_500','4990_4991','4991_4992','4992_1719','4994_4995','4995_4996','4996_1826','4997_4998','4998_4999','4999_1833','500_501','5008_4640','501_1484','5013_5014','5014_7291','5015_7290','5022_3461','5022_6002','5023_5024','5024_5161','5025_4854','5026_2450','5028_3696','5028_7115','5029_5028','5030_5031','5031_6033','5032_2824','5032_2867','5033_7130','5044_3751','5044_4433','5045_4460','5046_7639','5047_3146','5047_3224','5051_5052','5052_2975','5053_1633','5056_2239','5058_4602','5060_5061','5061_1231','5062_691','5064_7173','5069_3723','5071_5072','5072_4321','5072_7342','5073_3679','5073_5074','5074_6054','5075_5076','5076_3682','5077_3694','5077_5078','5078_3689','5079_3694','5081_5082','5082_3639','5084_6054','5086_3656','509_510','5090_4204','5091_5092','5092_7574','5096_7145','5097_2383','5099_2383','51_52','510_511','511_513','5110_4012','5111_4051','5112_4003','5113_3918','5118_3910','5119_5120','512_513','5120_5121','5121_5122','5122_3461','5123_5124','5124_5125','5125_6001','5127_5128','5128_2060','5128_2996','5129_5130','513_4513','513_7402','5130_1135','5131_5132','5132_1331','5133_2596','5134_2718','5135_4303','5136_4262','5139_4214','5140_1500','5140_620','5142_7567','5143_2837','5144_7027','5144_7702','5145_276','5147_68','5148_1401','5149_1419','515_516','5150_2327','5154_4804','5155_4806','5156_7552','5157_5154','516_4384','5160_5155','5161_5162','5162_2571','5162_4880','5164_4633','5165_3743','5165_7169','5169_2735','5171_2976','5174_6078','5175_3755','5176_3774','518_2255','519_521','5192_342','5192_350','52_261','52_262','52_263','52_265','52_277','52_4725','521_522','522_1741','522_523','523_1740','523_4382','523_525','523_669','525_526','526_527','527_528','528_529','529_530','53_54','530_531','531_709','532_533','532_649','533_534','534_535','535_536','536_537','537_538','538_539','539_540','54_55','540_541','541_542','542_543','543_544','544_545','544_546','545_546','546_547','546_910','546_914','547_548','548_549','549_550','549_677','55_56','550_551','551_552','552_553','553_554','554_555','555_557','557_558','558_559','559_560','56_58','560_562','562_563','563_564','564_565','565_566','566_567','566_707','567_568','568_569','569_570','57_58','570_571','571_572','572_573','573_574','574_575','575_576','576_577','577_579','579_580','58_4499','58_59','580_581','581_582','582_583','583_584','584_585','585_586','586_587','587_588','588_589','588_590','589_590','59_60','590_1729','590_591','591_592','592_593','593_594','594_595','595_596','596_597','597_598','598_599','599_600','6_194','60_1802','60_820','600_601','600_654','6001_4438','6002_5119','6003_5123','6004_3','6004_7','6005_1546','6005_7072','6006_3493','6007_2836','601_602','6010_3656','6011_2133','6013_6247','6014_2158','6014_6242','6015_5029','6016_6017','6017_7192','602_603','6020_6023','6021_7507','6023_6026','6024_6021','6025_6024','6026_6027','6027_6029','6028_6025','603_604','6030_6028','6033_3089','604_605','6041_2826','6042_1022','6048_5064','605_606','6050_3749','6050_7398','6051_6053','6052_3613','6053_7327','6054_4330','6055_1073','6057_2183','6059_334','6059_404','606_607','6068_1584','607_608','6070_2708','6074_748','6074_793','6075_2631','6076_6077','6077_3569','6078_7112','6079_6315','608_609','6080_1585','6080_983','6083_2733','6085_2026','6085_2053','6086_3161','6086_7419','6087_3620','6088_1898','6089_3981','609_610','6094_2102','6095_7062','6097_4093','610_611','6107_6108','6107_6109','6108_6109','6109_6110','611_612','6110_7020','6110_7029','6111_6112','6112_6113','6113_1898','6113_4323','6115_1205','6115_4593','6117_3655','6117_5079','6119_3392','612_613','6122_35','6124_4004','6125_7092','613_614','6130_1262','6132_2551','6132_4861','6135_3612','6137_4256','6137_4956','614_4415','614_615','614_675','6140_6144','6141_6142','6142_2181','6143_6011','6144_6145','6145_6146','6146_6147','6147_6149','6149_6150','615_616','6150_6143','6152_6153','6153_6243','6154_6155','6155_1987','6155_6141','6156_6165','6157_6158','6158_3964','6158_7314','6159_6160','616_617','6160_6161','6161_3991','6162_7022','6163_1095','6164_6159','6165_6157','617_618','6170_5023','6170_7183','6171_4974','6172_4493','618_619','6182_322','6185_6186','6186_6216','6187_6188','6188_6215','619_675','620_1172','620_621','620_7568','620_7569','621_289','621_291','621_293','621_4495','6215_6184','6216_3400','6219_6229','6228_4788','6228_7111','6229_6348','6229_6364','6229_7131','6230_6228','6234_332','6235_327','6238_6182','6239_104','624_625','6241_105','6242_6152','6243_6154','6244_2134','6245_2156','6245_4671','6245_6013','6247_2679','625_626','6251_5140','6251_7706','6252_301','6252_302','6253_7191','626_627','627_628','6273_136','6273_4891','6276_6362','6277_6278','6278_6279','6279_6280','628_629','6282_6335','6283_6282','6285_6286','6286_6287','6287_6288','6288_6342','629_630','6290_7120','6298_6336','63_64','630_4466','6310_6311','6311_4495','6311_7399','6312_7197','6315_6316','6316_7236','6318_6319','6319_7246','6320_4594','6324_6325','6325_6341','6326_4886','633_634','6334_7660','6335_6326','6336_7069','6339_6324','634_1005','6341_6277','6342_6298','6343_4759','6348_1182','635_636','6359_7189','636_637','6361_6367','6362_7276','6363_6361','6364_1558','6365_6276','6366_6230','6367_4985','637_638','638_639','639_640','64_65','640_641','641_642','642_643','643_644','644_645','645_1796','649_650','65_66','650_651','651_4472','653_722','654_655','655_656','656_657','657_658','658_659','659_604','66_4498','664_614','664_665','665_666','666_667','667_668','668_614','669_670','670_671','671_672','672_4382','675_1184','675_1500','675_303','675_4380','675_4415','675_620','675_621','677_678','678_679','679_680','68_30','680_681','681_682','682_683','683_684','684_685','685_686','686_688','688_689','689_690','69_7686','69_828','690_5062','691_692','692_693','692_694','692_695','693_585','694_695','695_696','696_697','697_698','698_699','699_700','7_11','7_194','700_701','7009_1604','701_702','7010_7703','7011_2171','7011_7160','7012_2201','7012_2644','7013_6162','7014_4315','7015_4617','7016_4619','7017_7018','7018_7030','702_703','7020_7029','7021_6111','7021_7163','7022_1554','7023_7014','7025_4464','7027_4538','7028_5147','7029_7038','703_704','7030_6111','7030_7021','7031_4897','7038_7011','7039_7204','704_705','7043_1424','7044_993','7047_7161','7048_7699','7049_2236','705_706','7050_7185','7053_2088','7056_4731','7057_4847','7057_5142','706_567','706_707','7062_6075','7063_1137','7064_7066','7065_7063','7066_2924','7067_2956','7067_2984','7068_2966','7069_7443','707_708','7070_7445','7073_7097','7074_3919','7075_3925','7078_1449','7079_7080','708_731','7080_7081','7081_7565','7082_7083','7083_7084','7084_7108','7085_7086','7086_7087','7087_7088','7088_7089','7089_7091','709_532','7091_3910','7092_110','7097_7098','7098_7099','7099_7100','71_72','710_711','7100_7101','7101_7505','7102_7103','7103_7104','7104_7105','7105_7106','7106_6005','7107_7102','7108_3935','711_712','7110_3938','7111_6365','7112_972','7113_127','7114_7116','7115_6310','7116_3694','7117_7118','7118_6289','7119_7127','712_713','7120_7119','7121_7183','7122_7121','7123_4347','7124_7343','7125_7344','7126_7124','7127_7143','7128_7117','713_714','7130_1237','7132_1558','7132_6230','7133_7134','7134_3867','7135_7136','7136_3874','7137_7139','7138_3370','7139_4623','714_715','7140_7141','7141_7138','7142_7015','7143_3368','7143_7168','7144_3881','7145_3884','7145_4843','7149_7277','715_716','7153_6056','7154_7153','7156_7020','7158_7048','7159_7017','7159_7388','716_717','7160_7700','7161_7162','7163_1893','7166_1847','7167_1808','7168_4616','7169_3743','717_718','7170_3740','7172_3775','7173_3811','7175_3806','7176_2836','7177_2740','7178_2765','718_719','7180_7123','7180_7331','7183_4880','7184_2225','7184_7050','7185_7186','7186_7187','7187_3947','7187_3954','7187_4001','7188_3082','719_720','7191_6238','7192_4683','7193_7049','7194_7196','7195_3653','7196_6310','7197_7195','72_5144','720_680','720_721','7201_2138','7204_3728','7204_3736','7207_7288','7208_7249','721_680','721_693','7214_4347','7216_2501','7217_7227','7218_7031','7219_7218','722_723','7222_6170','7224_1581','7227_6172','7229_3418','7229_4617','723_724','7230_4617','7231_7128','7233_4573','7234_4002','7236_7245','7237_7241','7238_7224','7239_4401','724_725','7240_1891','7241_7247','7242_7238','7243_7126','7245_6317','7246_6320','7247_7243','7248_7207','7248_7254','7249_7253','725_726','7250_7248','7250_7252','7251_4051','7252_7248','7253_7251','7254_7257','7255_7249','7256_7255','7257_7258','7258_7260','7259_7256','726_727','7260_7263','7261_7259','7262_7261','7263_7264','7264_7268','7265_7262','7266_7265','7268_7270','727_728','7270_6122','7270_7266','7271_5136','7272_4956','7276_7149','7277_6363','7278_7289','728_729','7280_7281','7281_7284','7282_7278','7283_7282','7284_7287','7286_7283','7287_7208','7288_7286','7289_7280','729_730','7290_5016','7291_3485','7292_3818','7293_1335','7294_4132','7294_4156','7296_4265','7296_4266','73_74','730_5160','731_710','7313_6164','7314_3964','7317_3224','7318_4978','7318_4980','7319_7342','7320_1486','7325_3317','7325_4984','7326_7418','7327_3629','7328_3651','7329_3659','733_6015','7330_2051','7331_2559','7333_783','7334_1838','7340_783','7342_7125','7343_4978','7343_4980','7343_7318','7344_2510','7344_7334','7347_3669','7348_3669','7348_3671','7349_1631','7350_1558','7350_6230','7352_4246','7353_4571','7353_7584','7354_1975','7358_1231','7358_1235','7358_5174','7361_7362','7362_3129','7362_3134','7363_4588','7364_7365','7365_4210','7366_7367','7367_7368','7368_3143','7370_7339','7374_1845','7375_3466','7379_4903','738_739','7380_1676','7381_4742','7382_7381','7386_4607','7388_7017','7389_4324','7389_7025','739_740','7391_493','7392_1443','7392_312','7395_6124','7396_5145','7397_5140','7397_6252','7398_6252','7398_7397','7399_7398','7399_7611','74_4371','740_741','7400_3669','7401_3665','7402_4380','7402_4717','741_742','7412_7413','7413_2383','7414_1396','7415_7416','7416_7325','7417_3471','7418_7417','7419_7420','742_743','7420_3345','7421_7422','7422_6085','7423_1566','7424_7423','7425_7424','7426_7427','7427_7428','7428_138','743_744','7430_177','7431_175','7432_3454','7433_4677','7435_2722','7435_4413','7438_1410','7439_7454','744_745','7440_2940','7441_2933','7442_7446','7443_7444','7444_2941','7445_6339','7446_7070','7449_2941','745_600','7453_1478','7453_1480','7453_2001','7454_1380','7455_7438','7458_4479','7458_7425','746_844','7460_4930','7461_4230','7461_4281','7462_7461','7469_1724','747_748','747_842','747_843','7471_1533','7472_3230','7473_2679','7475_6107','748_7390','748_749','749_494','749_750','749_751','75_4501','750_751','7505_5071','7505_7678','7507_2663','751_752','7510_2207','7510_4799','7513_807','7514_2279','7516_6079','752_7339','752_753','753_7339','753_7370','753_754','754_755','755_756','7550_2570','7551_4800','7552_924','7555_4962','7556_3539','7559_7073','7559_7107','756_757','7560_7679','7563_3368','7563_7231','7564_340','7564_4521','7565_7082','7567_7326','757_758','7571_207','7574_4177','7577_1353','7578_7582','7579_1285','758_759','758_760','7581_1283','7582_297','7582_299','7582_319','7582_340','7582_407','7584_3344','7585_7587','7586_7587','7587_303','7587_328','7587_345','7587_407','7587_496','7587_7588','7588_281','7588_325','7588_328','7588_329','7588_4717','759_760','7591_496','7592_3815','760_2007','760_761','760_762','7601_2508','7602_22','7602_85','7603_45','761_762','7611_7398','7612_2811','7616_5081','762_763','7622_297','7622_334','7623_2508','7623_3676','7623_3715','7623_3719','7623_5030','7623_6312','7623_7401','763_2007','763_4952','763_4953','763_767','7639_7640','7640_3134','7640_5047','7641_7642','7642_7643','7643_3214','7644_7645','7645_7646','7646_3238','7654_4843','7660_3114','7661_1521','7667_3091','767_768','7671_513','7672_826','7673_7674','7674_7675','7675_7676','7676_7677','7677_7559','7678_7560','7679_7680','768_2085','768_769','768_770','768_772','7680_7681','7681_7738','7683_1582','769_770','7690_3781','7691_3721','7692_6010','7698_7113','7699_7159','77_78','770_771','770_773','7700_7047','7701_7010','7702_7009','7703_5147','7704_3983','7705_6251','7706_7707','7707_512','7708_7709','7709_7710','771_772','7710_7711','7711_7705','772_417','772_773','772_775','773_774','7738_7683','774_775','774_906','775_776','775_906','776_777','777_779','777_906','7771_3966','779_780','78_79','780_781','781_782','782_783','783_784','783_785','784_785','785_786','786_791','786_792','786_793','79_80','790_7585','791_7585','792_320','792_7586','793_274','793_320','793_495','793_7586','795_796','796_56','796_57','796_797','797_798','798_799','799_800','8_12','80_82','800_801','801_802','802_1654','802_803','803_804','804_7513','807_808','808_809','809_810','81_817','810_811','811_812','812_813','813_814','814_81','817_818','818_819','819_261','819_263','819_264','819_265','819_4725','82_817','82_83','820_821','821_63','822_823','823_824','824_825','825_826','826_827','827_71','828_829','829_830','83_84','830_831','831_75','832_833','833_77','834_835','835_836','836_837','837_1506','838_839','839_840','84_51','840_841','841_49','842_1014','842_845','842_846','843_1013','843_845','844_846','845_846','846_7340','846_847','846_848','847_848','848_2795','848_758','848_759','848_849','849_850','85_203','85_86','850_851','851_852','852_853','853_854','854_855','855_2815','855_856','856_857','857_858','858_859','859_860','86_87','860_861','861_862','862_3014','87_88','875_876','876_3011','876_878','877_876','878_879','879_880','88_89','880_881','881_882','882_883','883_884','884_885','885_886','886_887','887_2900','887_903','89_27','893_894','894_1067','895_1025','896_897','897_893','90_91','903_904','904_906','905_4331','906_907','906_909','907_908','908_909','909_786','909_790','909_792','91_129','91_92','910_911','911_912','912_913','913_914','914_938','915_916','916_917','917_918','918_919','919_7551','92_93','924_925','925_927','927_938','928_588','928_929','929_930','93_94','930_931','931_587','938_940','94_102','94_126','940_941','941_942','942_943','943_3599','944_945','945_947','946_1155','947_948','948_949','949_928','950_915','956_957','957_958','958_959','959_960','960_961','961_962','962_635','969_1176','972_1625','980_981','981_1518','981_1576','983_984','984_985','985_1529','987_4937','992_7044','993_5060']

Get all outliers i.e. rows where the value is more than three times the standard deviation for the segment.

In [ ]:
# based on https://stackoverflow.com/questions/45928046/identifying-statistical-outliers-with-pandas-groupby-and-individual-columns
stds = 3.0
z = df[['segment', 'time_diff']].groupby('segment').transform(lambda group: (group - group.mean()).div(group.std()))
outliers = z.abs() > stds

Print the outliers:

In [ ]:
df[outliers.any(axis=1)]

Remove the outliers:

In [ ]:
df_outliers = df[outliers.any(axis=1)]
df = pd.concat([df,df_outliers]).drop_duplicates(keep=False)

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
print()
print("After removing outliers, we have", rows, "rows and", cols, "columns.")
print()

## 10.2 Segment Checks

We need to check if we are missing data for any segments.

In [ ]:
# loop through each segment and check how many rows we have for it
# if there are no rows for a particular segment, then add it to the missing list
missing = []
for segment in segments:
    df_temp = df.loc[(df['segment'] == segment)]
    if df_temp.shape[0] == 0:
        missing.append(segment)

In [ ]:
len(missing)

In [ ]:
print("Missing Segments")
for seg in missing:
    print(seg)

These segments do not appear in the data, but exist in current Dublin Bus routes. As such we must deal with them in some way as predictions may be requested for them.

# 11. Export the Cleaned Data

## 11.1 Remove Features

Remove features that we no longer need at this point:

- dayofservice
- progrnumber_first
- stoppointid_first
- progrnumber_next
- stoppointid_next
- actualtime_arr_stop_next
- record_date

In [ ]:
df = df.drop(columns=['dayofservice', 'progrnumber_first', 'stoppointid_first', 'progrnumber_next', 'stoppointid_next', \
                     'actualtime_arr_stop_next', 'record_date'])

## 11.2 Export the Data

The cleaned data is exported at this point so that it can be reloaded easily without having to re-run the whole notebook:

In [ ]:
df.to_hdf('/data_analytics/data/august_bus_data_cleaned.hdf', key='cleaned', format='table')

Import the data when required:

In [ ]:
df = pd.read_hdf('/data_analytics/data/august_bus_data_cleaned.hdf')

# 8. Plot Features

## 8.1 Individual Features

### 8.1.1 Histograms for the Continuous Features

In [ ]:
df[continuous_columns].hist(figsize=(30,30))

### 8.1.2 Box Plots for the Continuous Features

In [ ]:
for col in continuous_columns:
    f = df[col].plot(kind='box', figsize=(10,5))
    plt.show()

### 8.1.3 Bar Plots for the Categorical Features

In [ ]:
# select the features to plot
# some categorical features will not be plotted here due to the high number of values e.g. bus stop no.
categorical_columns_plot = df[['lineid', 'month', 'day', 'hour', 'day_of_week', 'weekday', 'bank_holiday']].columns

In [ ]:
for column in categorical_columns_plot:
    f = df[column].value_counts().plot(kind='bar', title=column, figsize=(8,6))
    plt.show()

## 8.2 Correlations between Features

### 8.2.1 Correlation Matrix for Continuous Features

In [ ]:
# Correlation matrix using code found on https://stanford.edu/~mwaskom/software/seaborn/examples/many_pairwise_correlations.html
sns.set(style="white")

# Calculate correlation of all pairs of continuous features
corr = df[continuous_columns].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 10))

# Generate a custom colormap - blue and red
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,
            square=True, xticklabels=True, yticklabels=True,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
plt.yticks(rotation = 0)
plt.xticks(rotation = 90)

We are mainly interested here in the correlation between the weather data and *time_diff* or *actualtime_arr_stop_next*. The correlation for these features with the weather data is quite low.

Some of the weather features have very high correlation with each other e.g. temp with wetb and vappr with dewpt. The weather features we can use will be dependent on what is available in the weather forecast we receive. However, if all features are available in the forecast we may need to remove some of the features due to high correlation.

### 8.2.2 Correlations between the Continuous Features & the Target Feature

We have two options for our target feature:

- actualtime_arr_stop_next
- time_diff

I will plot scatter plots for each of the following combinations:

- actualtime_arr_stop_first vs. actualtime_arr_stop_next
- rain vs. actualtime_arr_stop_next
- temp vs. actualtime_arr_stop_next
- rhum vs. actualtime_arr_stop_next
- msl vs. actualtime_arr_stop_next
- actualtime_arr_stop_first vs. time_diff
- rain vs. time_diff
- temp vs. time_diff
- rhum vs. time_diff
- msl vs. time_diff

In [ ]:
df.plot.scatter(x='actualtime_arr_stop_first', y='time_diff', c='purple')

In [ ]:
df.plot.scatter(x='rain', y='time_diff', c='purple')

In [ ]:
df.plot.scatter(x='temp', y='time_diff', c='purple')

In [ ]:
df.plot.scatter(x='rhum', y='time_diff', c='purple')

In [ ]:
df.plot.scatter(x='msl', y='time_diff', c='purple')

### 8.2.3 Correlations between the Categorical Features & the Target Feature

We have two options for our target feature:

- actualtime_arr_stop_next
- time_diff

I will plot box plots for each.

In [ ]:
for col in categorical_columns_plot:
    df.boxplot(column=['actualtime_arr_stop_next'], by=[col], figsize=(8,5))

In [ ]:
for col in categorical_columns_plot:
    df.boxplot(column=['time_diff'], by=[col], figsize=(8,5))

## 8.3 Plots by Segment

I will repeat some of the above graphs breaking them down by segment. I will plot for time_diff as this seems to have a stronger correlation based on the above graphs (for categorical features at least).

In [ ]:
# make a list of all segments
segments = [(226,228),(228,229),(229,227),(227,230),(230,231),(231,1641),(1641,1642),(1642,213),(213,214),(214,4432),(4432,119),(119,44),(44,7603),(7603,45),(45,46),(46,47),(47,48),(48,49),(49,51),(51,52),(52,265),(265,271),(271,340),(340,350),(350,351),(351,352),(352,353),(353,354),(354,355),(355,356),(356,357),(357,390),(390,372),(372,373),(373,374),(374,375),(375,2804),(2804,376),(376,377),(377,378),(378,380),(381,382),(382,4451),(4451,383),(383,384),(384,385),(385,387),(387,388),(388,389),(389,393),(393,371),(371,391),(391,392),(392,395),(395,396),(396,397),(397,398),(398,399),(399,400),(400,319),(319,278),(278,10),(10,12),(12,14),(14,15),(15,17),(17,18),(18,19),(19,21),(21,7602),(7602,85),(85,203),(203,204),(204,205),(205,1620),(1620,220),(220,221),(221,222),(222,223),(223,224),(224,225),(225,226),(278,7615),(6122,35),(35,36),(36,37),(37,38),(38,39),(39,40),(40,41),(41,42),(42,43),(43,44),(52,261),(261,271),(271,404),(404,747),(747,842),(842,845),(845,846),(846,847),(847,848),(848,849),(849,850),(850,851),(851,852),(852,853),(853,854),(854,855),(855,856),(856,857),(857,858),(858,859),(859,860),(860,861),(861,862),(862,3014),(3014,3015),(3015,3016),(3016,3017),(3017,3018),(3018,3019),(3019,3020),(3020,3021),(3021,3022),(3022,441),(441,442),(442,443),(443,444),(444,445),(445,4779),(4779,7057),(7057,4847),(4847,4848),(449,450),(450,3181),(3181,451),(451,447),(447,5045),(5045,4460),(4460,453),(453,454),(454,4475),(4475,3023),(3023,3024),(3024,3025),(3025,3026),(3026,3027),(3027,3028),(3028,3029),(3029,3030),(3030,3031),(3031,875),(875,876),(876,878),(878,879),(879,880),(880,881),(881,882),(882,883),(883,884),(884,885),(885,886),(886,887),(887,903),(903,904),(904,906),(906,907),(907,908),(908,909),(909,786),(786,793),(793,320),(320,278),(7602,22),(22,23),(23,24),(24,25),(25,27),(27,28),(28,29),(29,4680),(4680,31),(31,32),(32,33),(33,7270),(7270,6122),(324,6235),(6235,327),(327,6016),(6016,6017),(6017,7192),(7192,4683),(4683,107),(107,6125),(6125,7092),(7092,110),(110,112),(112,113),(113,114),(114,115),(115,37),(39,116),(116,117),(117,118),(118,119),(261,270),(270,335),(335,4521),(4521,1935),(1935,1937),(1937,1938),(1938,1939),(1939,1940),(1940,1941),(1941,1942),(1942,1943),(1943,1944),(1944,1945),(1945,1946),(1946,1947),(1947,1948),(1948,1949),(1949,1950),(1950,1951),(1951,1952),(1952,1953),(1953,1954),(1954,1955),(1955,1956),(1956,1957),(1957,1958),(1958,1959),(1959,1960),(1960,4379),(4379,1963),(1963,4549),(4549,1964),(1964,4674),(4674,4663),(4663,4664),(4664,2160),(2160,2161),(2161,4665),(4665,4666),(4666,4667),(4667,4668),(4668,4677),(4677,2136),(2136,2137),(2137,7201),(7201,2138),(2138,2139),(2139,2140),(2140,2141),(2141,2142),(2142,2143),(2143,2145),(2145,4670),(4670,2165),(2165,2166),(2166,4545),(4545,4552),(4552,4554),(4554,7229),(2166,7229),(4554,7230),(7229,4617),(4617,4555),(4555,4576),(4576,3418),(3418,3419),(3419,2374),(2374,2146),(2146,2147),(2147,2148),(2148,2149),(2149,2150),(2150,2151),(2151,2152),(2152,2153),(2153,2154),(2154,6245),(6245,4671),(4671,4672),(4672,3465),(3465,4673),(4673,2172),(2172,2159),(2159,4409),(4409,4550),(4550,4551),(4551,1965),(1965,1966),(1966,1975),(1975,1977),(1977,1978),(1978,1980),(1980,1981),(1981,1982),(1982,4406),(4406,1984),(1984,1985),(1985,1986),(1986,1987),(1987,1988),(1988,1989),(1989,1990),(1990,1992),(1992,1993),(1993,1994),(1994,1995),(1995,1996),(1996,1997),(1997,1998),(1998,2001),(2001,2002),(2002,1359),(1359,319),(319,281),(281,7),(7,11),(11,14),(85,86),(86,87),(87,88),(88,89),(89,27),(4680,90),(90,91),(91,92),(92,93),(93,94),(94,102),(102,6239),(6239,104),(104,6241),(6241,105),(105,6253),(6253,7191),(7191,6238),(6238,6182),(6182,322),(322,323),(323,6234),(6234,332),(7229,3418),(1998,1999),(1999,2001),(21,85),(4555,3418),(7230,4617),(248,249),(249,250),(250,251),(251,252),(252,255),(255,234),(234,235),(235,236),(236,1190),(1190,1191),(1191,1192),(1192,1193),(1193,1194),(1194,1195),(1195,664),(664,665),(665,666),(666,667),(667,668),(668,614),(614,615),(615,616),(616,617),(617,618),(618,619),(619,675),(675,4415),(4415,301),(301,336),(336,7581),(7581,1283),(1283,7579),(7579,1285),(1285,1016),(1016,1017),(1017,1018),(1018,1019),(1019,1020),(1020,1076),(1076,1077),(1077,1078),(1078,1079),(1079,1080),(1080,1081),(1081,2914),(2914,2989),(2989,2990),(2990,1042),(1042,1043),(1043,1044),(1044,1045),(1045,4710),(4710,1046),(1046,1047),(1047,1048),(1048,1049),(1049,4333),(4333,4334),(4334,4712),(4712,2853),(2853,2854),(2854,2855),(2855,2856),(2856,2857),(2857,2858),(2858,2859),(2859,2860),(2860,2861),(2861,2862),(2862,2863),(2863,2864),(2864,4486),(4486,2865),(2865,2866),(2866,6044),(6041,2826),(2826,2842),(2842,2843),(2843,2844),(2844,2845),(2845,2846),(2846,2847),(2847,4397),(4397,2848),(2848,2849),(2849,2850),(2850,2851),(2851,2852),(2852,4707),(4707,4708),(4708,4336),(4336,1051),(1051,1052),(1052,1053),(1053,1054),(1054,1036),(1036,4709),(4709,1037),(1037,1038),(1038,2998),(2998,2999),(2999,3000),(3000,3001),(3001,3002),(3002,1166),(1166,1167),(1167,1168),(1168,1169),(1169,1170),(1170,1069),(1069,1070),(1070,1071),(1071,4528),(4528,1072),(1072,7577),(7577,1353),(1353,1354),(1354,7578),(7578,7582),(7582,297),(297,496),(496,497),(497,515),(515,516),(516,4384),(4384,519),(519,521),(521,522),(522,523),(523,669),(669,670),(670,671),(671,672),(672,4382),(4382,1185),(1185,1186),(1186,1187),(1187,1188),(1188,216),(216,217),(217,242),(242,243),(243,244),(244,245),(245,246),(246,248),(243,254),(254,244),(6318,6319),(6319,7246),(7246,6320),(6320,4594),(4594,4595),(4595,4596),(4596,4563),(4563,1218),(1218,1270),(1270,1272),(1272,1273),(1273,1274),(1274,1275),(1275,1276),(1276,1277),(1277,1219),(1219,1220),(1220,1221),(1221,664),(301,4495),(4495,7581),(1081,1082),(1082,1083),(1083,1085),(1085,1121),(1121,1122),(1122,1123),(1123,1124),(1124,1125),(1125,1127),(1127,1130),(1130,1140),(1140,1141),(1141,1142),(1142,1143),(1143,1144),(1144,1145),(1145,4869),(4869,4870),(4870,3007),(3007,6283),(6283,6282),(6282,6335),(6335,6326),(6326,4886),(4886,4887),(4887,1150),(1150,1151),(1151,1152),(1152,1153),(1153,1154),(1154,946),(946,1155),(1155,1157),(1157,1158),(1158,1159),(1159,1160),(1160,1161),(1161,1162),(1162,1163),(1163,1164),(1164,1165),(1165,1166),(7582,299),(299,497),(4382,1196),(1196,1197),(1197,1198),(1198,1199),(1199,1200),(1200,4385),(4385,1201),(1201,1202),(1202,1203),(1203,6115),(6115,4593),(4593,7516),(7516,6079),(6079,6315),(6315,6316),(6316,7236),(7236,7245),(7245,6317),(7347,3669),(3669,7349),(7349,1631),(1631,1632),(1632,5053),(5053,1633),(1633,1634),(1634,1635),(1635,1636),(1636,1637),(1637,1638),(1638,1639),(1639,1640),(1640,231),(231,232),(232,233),(233,234),(236,237),(237,213),(52,262),(262,270),(270,336),(336,1279),(1279,1282),(1282,4456),(4456,1284),(1284,1285),(1285,1287),(1287,1288),(1288,1289),(1289,1290),(1290,1291),(1291,1292),(1292,1293),(1293,1294),(1294,1295),(1295,1296),(1296,1297),(1297,1298),(1298,1299),(1299,1300),(1300,1301),(1301,1302),(1302,1303),(1303,1304),(1304,1305),(1305,1320),(1320,1321),(1321,1322),(1322,1323),(1323,1324),(1324,7068),(7068,2966),(2966,2967),(2967,2968),(2968,2969),(2969,2970),(2970,2971),(2971,5051),(5051,5052),(5052,2975),(208,209),(209,210),(210,211),(211,212),(212,213),(5171,2976),(2976,2977),(2977,2978),(2978,2979),(2979,2980),(2980,2981),(2981,2991),(2991,2992),(2992,1325),(1325,1326),(1326,1327),(1327,1328),(1328,1329),(1329,1330),(1330,1331),(1331,1332),(1332,1333),(1333,1334),(1334,7293),(7293,1335),(1335,1336),(1336,1337),(1337,1338),(1338,1339),(1339,1340),(1340,1341),(1341,1342),(1342,1343),(1343,1344),(1344,1345),(1345,1347),(1347,1348),(1348,1349),(1349,1350),(1350,1352),(1352,1353),(1353,1355),(1355,1357),(1357,1359),(1359,320),(278,8),(8,12),(205,215),(215,216),(217,218),(218,219),(219,220),(220,1622),(1622,1623),(1623,1624),(1624,1625),(1625,1626),(1626,1627),(1627,1628),(1628,1629),(1629,1630),(1630,7347),(7391,493),(493,494),(494,495),(495,400),(400,7588),(7588,7392),(7392,312),(312,1444),(1444,1445),(1445,7078),(7078,1449),(1449,1450),(1450,1451),(1451,2191),(2191,2192),(2192,2193),(2193,2194),(2194,2195),(2195,2196),(2196,2197),(2197,2198),(2198,2200),(2200,2201),(2201,7239),(7239,4401),(4401,2213),(2213,2214),(2214,2215),(2215,2216),(2216,4599),(4599,2217),(2217,2218),(2218,2219),(2219,2220),(2220,2221),(2221,2223),(2223,2224),(2224,3886),(3886,3887),(3887,3888),(3888,3890),(3890,3891),(3891,3949),(3949,3950),(3950,2229),(2229,2230),(2230,2231),(2231,2233),(2233,2232),(2232,2234),(2234,2235),(2235,2236),(2236,2238),(2238,5056),(5056,2239),(2239,2241),(2241,2242),(2242,2243),(2243,2244),(2244,2245),(2245,2246),(2246,2247),(2247,2248),(2248,2249),(2249,2250),(2250,2251),(2251,1472),(1472,4408),(4408,1473),(1473,1474),(1474,1476),(1476,7453),(7453,1478),(1478,1479),(1479,315),(315,406),(406,4530),(4530,748),(748,7390),(400,346),(346,317),(317,312),(4401,2202),(2202,2203),(2203,2204),(2204,4888),(4888,4798),(4798,4715),(4795,2686),(2686,7510),(7510,2207),(2207,2208),(2208,2210),(2210,2211),(2211,2241),(406,2810),(2810,7387),(1218,1258),(1258,1259),(1259,1260),(1260,6130),(6130,1262),(1262,4692),(4692,1245),(1245,4693),(4693,1254),(1254,1255),(1255,1256),(1256,4694),(4694,1249),(1249,1247),(1247,1248),(1248,1273),(675,1500),(1500,302),(302,4495),(4495,4522),(4522,1934),(1934,2310),(2310,2311),(2311,2312),(2312,2313),(2313,2314),(2314,2315),(2315,2094),(2094,1406),(1406,1407),(1407,1409),(1409,2095),(2095,2096),(2096,2097),(2097,2099),(2099,2101),(2101,2102),(2102,2103),(2103,2332),(2332,2333),(2333,2334),(2334,4662),(4662,2335),(2335,2336),(2336,2337),(2337,2339),(2339,2340),(2340,2601),(2601,4446),(4446,2603),(2603,2557),(2557,4436),(4436,5008),(5008,4640),(4640,4347),(4347,2349),(2349,2351),(2351,2621),(2621,2622),(2622,2623),(2623,2624),(2624,4441),(4441,2626),(2626,2669),(2669,4641),(4641,4642),(4642,2355),(1260,1262),(400,4522),(2353,4643),(4643,4644),(4644,4645),(4645,4440),(4440,2628),(2628,2629),(2629,6095),(6095,7062),(7062,6075),(6075,2631),(2631,2359),(2359,2360),(2360,4348),(4348,4646),(4646,4647),(4647,2346),(2346,4435),(4435,2632),(2632,2633),(2633,2369),(2369,2370),(2370,2371),(2371,2372),(2372,2373),(2373,2377),(2377,2378),(2378,2183),(2183,2184),(2184,2185),(2185,7043),(7043,1424),(1424,2186),(2186,2187),(2187,2188),(2188,2189),(2189,1436),(1436,3952),(3952,2190),(2190,4434),(4434,2379),(2379,2380),(2380,2382),(2382,5099),(5099,2383),(2383,2002),(2002,1358),(1358,298),(298,496),(1201,4695),(4695,1243),(1243,4696),(4696,4697),(4697,1250),(1250,4698),(4698,4383),(4383,4699),(4699,4388),(4388,4700),(4700,1266),(1266,1267),(1267,1268),(1268,1269),(1269,1265),(1265,6115),(4593,4595),(2377,2431),(2431,2432),(2432,2433),(2433,2327),(2327,2328),(2328,2329),(2329,2330),(2330,2331),(2331,1424),(621,289),(289,496),(496,515),(523,525),(525,526),(526,527),(527,528),(528,529),(529,530),(530,531),(531,709),(709,532),(532,533),(533,534),(534,535),(535,536),(536,537),(537,538),(538,539),(539,540),(540,541),(541,542),(542,543),(543,544),(544,545),(545,546),(546,547),(547,548),(548,549),(549,550),(550,551),(551,552),(552,553),(553,554),(554,555),(555,557),(557,558),(558,559),(559,560),(560,562),(562,563),(563,564),(564,565),(565,566),(566,567),(566,707),(707,708),(708,731),(731,710),(710,711),(711,712),(712,713),(713,714),(714,715),(715,716),(716,717),(717,718),(718,719),(719,720),(720,721),(721,693),(693,585),(567,568),(568,569),(569,570),(570,571),(571,572),(572,573),(573,574),(574,575),(575,576),(576,577),(577,579),(579,580),(580,581),(581,582),(582,583),(583,584),(584,585),(585,586),(586,587),(587,588),(588,589),(589,590),(590,591),(591,592),(592,593),(593,594),(594,595),(595,596),(596,597),(597,598),(598,599),(599,600),(600,601),(601,602),(602,603),(603,604),(604,605),(605,606),(606,607),(607,608),(608,609),(609,610),(610,611),(611,612),(612,613),(613,614),(675,620),(620,7569),(546,910),(910,911),(911,912),(912,913),(913,914),(914,938),(938,940),(940,941),(941,942),(942,943),(943,3599),(3599,3600),(3600,3601),(3601,3602),(3602,3604),(3604,3616),(3616,3618),(3618,3619),(3619,3620),(3620,3622),(3622,3623),(3623,4387),(3624,3587),(3587,3588),(3588,3590),(3590,3589),(3589,3591),(3591,3592),(3592,3593),(3593,3594),(3594,3595),(3595,3596),(3596,3597),(3597,3598),(3598,4465),(4465,944),(944,945),(945,947),(947,948),(948,949),(949,928),(928,929),(929,930),(930,931),(931,587),(292,1171),(1171,1172),(1172,1173),(1173,1174),(1174,1175),(1175,15),(1630,3670),(3670,3671),(3671,3672),(3672,3674),(3674,3675),(3675,3676),(3676,3677),(3677,3678),(3678,3679),(3679,3711),(3711,3712),(3712,3713),(3713,3714),(3714,3715),(3715,5175),(5175,3755),(3755,3756),(3756,3757),(3757,3758),(3758,3759),(3759,3760),(3760,3761),(3761,3762),(3762,3763),(3763,3764),(3764,3765),(3765,3766),(3766,3767),(3767,3768),(3768,3770),(3770,3769),(3769,3771),(3771,3772),(3772,3773),(3773,3774),(3774,7172),(7172,3775),(3775,3777),(3777,3778),(3778,7690),(7690,3781),(3781,3782),(3782,3783),(3783,3784),(3784,3785),(3785,3786),(3786,3787),(3787,3788),(3788,3789),(3789,3790),(3790,3791),(3791,3792),(3792,3793),(3793,3794),(3794,3795),(3795,3797),(3797,3798),(3798,3799),(3799,6076),(6076,6077),(6077,3569),(3569,3571),(3571,3801),(3801,3802),(3802,3805),(3805,7175),(7175,3806),(3806,3807),(3807,3808),(3808,3809),(3809,7174),(6048,5064),(5064,7173),(7173,3811),(3811,3812),(3812,3664),(3664,3813),(3813,3814),(3814,3815),(3815,3816),(3816,3566),(3566,3567),(3567,7292),(7292,3818),(3818,3819),(3819,3820),(3820,3822),(3822,3823),(3823,3824),(3824,3565),(3565,3825),(3825,3826),(3826,3827),(3827,3828),(3828,3829),(3829,3831),(3831,3832),(3832,3833),(3833,3834),(3834,3835),(3835,3836),(3836,3837),(3837,3838),(3838,3840),(3840,3842),(3842,3843),(3843,3844),(3844,3845),(3845,3846),(3846,3848),(3848,3849),(3849,3850),(3850,3851),(3851,3852),(3852,3853),(3853,3854),(3854,3855),(3855,3856),(3856,3857),(3857,3858),(3858,3859),(3859,3860),(3860,3861),(3861,3862),(3862,3863),(3863,3748),(3748,6050),(6050,3749),(3749,3750),(3750,3752),(3752,3751),(3751,3689),(3689,3691),(3691,3692),(3692,3694),(3694,3695),(3695,3696),(3696,3697),(3697,3698),(3698,3699),(3699,3885),(3885,1631),(52,277),(277,292),(7592,3815),(7340,783),(783,784),(784,785),(785,786),(793,7586),(7586,7587),(7587,7588),(7588,329),(329,1443),(1443,1444),(1445,1647),(1647,1648),(1648,1649),(1649,1911),(1911,802),(802,1654),(1654,1655),(1655,1656),(1656,1657),(1657,4458),(4458,1658),(1658,1659),(1659,1660),(1660,1661),(1661,1662),(1662,1664),(1664,1665),(1665,1666),(1666,1667),(1667,1668),(1668,1669),(1669,1670),(1670,1671),(1671,1672),(1672,1673),(1673,1674),(1674,1675),(1675,7380),(7380,1676),(1676,1517),(1517,1678),(1678,1679),(1679,4520),(4520,4825),(4825,7233),(7233,4573),(4573,4893),(4893,4894),(4894,4895),(4895,7217),(7217,7227),(7227,6172),(6172,4493),(4493,4362),(4362,2959),(4747,1882),(1882,7379),(7379,4903),(4903,7219),(7219,7218),(7218,7031),(7031,4897),(4897,4898),(4898,4574),(4574,4575),(4575,1680),(1680,1681),(1681,1682),(1682,1683),(1683,1684),(1684,1685),(1685,1686),(1686,1687),(1687,1688),(1688,1689),(1689,1690),(1690,1691),(1691,1692),(1692,1693),(1693,1694),(1694,1695),(1695,1696),(1696,1697),(1697,1698),(1698,1699),(1699,1700),(1700,1701),(1701,1702),(1702,1703),(1703,1704),(1704,1705),(1705,1706),(1706,1707),(1707,1708),(1708,1709),(1709,1528),(1528,1710),(1710,1711),(1711,1712),(1712,1713),(1713,1714),(1714,1715),(1715,7453),(315,404),(747,843),(843,845),(846,7340),(315,313),(7333,783),(793,274),(274,2),(2,192),(192,795),(795,796),(796,57),(57,58),(58,59),(59,60),(60,1802),(1802,1795),(1795,1806),(1806,1660),(1666,1807),(1807,7167),(7167,1808),(1808,2961),(2961,4901),(4901,4902),(4902,1672),(1675,1849),(1849,1851),(1851,1813),(1813,1814),(1814,1816),(1816,1817),(1817,1818),(1818,1819),(1819,1822),(1822,4994),(4994,4995),(4995,4996),(4996,1826),(1826,1827),(1827,7382),(7382,7381),(7381,4742),(4742,4743),(4743,4744),(4744,4745),(1816,6171),(6171,4974),(4974,4976),(4976,7126),(7126,7124),(7124,7343),(7343,4980),(4980,2289),(2289,1822),(4745,4989),(4989,4990),(4990,4991),(4991,4992),(4992,1719),(1719,1828),(1828,1829),(1829,4997),(4997,4998),(4998,4999),(4999,1833),(1833,1834),(1834,1835),(1835,1836),(1836,1837),(1837,2338),(2338,1840),(1840,1841),(1841,4314),(4314,1687),(1689,4899),(4899,4900),(4900,4746),(4746,1845),(1845,7166),(7166,1847),(1847,1696),(1703,1905),(1905,1804),(1804,1803),(1803,77),(77,78),(78,79),(79,80),(80,82),(82,817),(817,818),(818,819),(819,265),(265,6059),(6059,404),(747,748),(748,749),(749,750),(750,751),(751,752),(752,753),(753,7370),(7370,7339),(1834,1820),(1820,2468),(2468,7319),(7319,7342),(7342,7125),(7125,7344),(7344,7334),(7334,6274),(6274,1838),(1838,2338),(848,2795),(2795,756),(756,757),(757,758),(758,759),(759,760),(760,761),(761,762),(762,763),(763,2007),(2007,2008),(2008,2009),(2009,2010),(2010,435),(435,2011),(2011,2012),(2012,3322),(3322,3563),(3563,444),(4848,4350),(4350,5143),(5143,2837),(2837,2838),(2838,2839),(2839,2840),(2840,2845),(2852,4987),(4987,2978),(2981,2954),(2954,2955),(2955,7067),(7067,2956),(2956,2965),(2965,2967),(2970,4988),(4988,2853),(2861,2829),(2829,2830),(2830,2831),(2831,2832),(2832,4349),(4349,3184),(3184,449),(453,4477),(4477,3321),(3321,4727),(4727,4728),(4728,461),(461,2068),(2068,2069),(2069,2070),(2070,2084),(2084,768),(768,769),(769,770),(770,771),(771,772),(772,773),(773,774),(774,775),(775,776),(776,777),(777,906),(786,792),(792,7586),(7588,281),(281,261),(3283,3284),(3284,3310),(3310,3481),(3481,3482),(3482,3483),(3483,3484),(3484,3485),(3485,3486),(3486,3487),(3487,7415),(7415,7416),(7416,7325),(7325,3317),(3317,3318),(3318,3320),(3320,2064),(2064,2065),(2065,4727),(7587,303),(6004,3),(3,192),(796,56),(56,58),(58,4499),(4499,4934),(4934,4500),(4500,4481),(4481,4485),(4485,822),(822,823),(823,824),(824,825),(825,826),(826,827),(827,71),(71,72),(72,5144),(5144,7027),(7027,4538),(4538,7396),(7396,5145),(5145,276),(485,487),(487,488),(488,489),(489,490),(490,491),(491,492),(492,493),(400,279),(279,3),(284,4504),(4504,7028),(7028,5147),(5147,68),(68,30),(30,69),(69,828),(828,829),(829,830),(830,831),(831,75),(75,4501),(4501,4935),(4935,4502),(4502,4936),(4936,79),(819,263),(263,6071),(263,4725),(4725,4495),(4495,2809),(2809,408),(408,409),(409,410),(410,411),(411,412),(412,413),(413,414),(414,415),(4525,4526),(4526,1700),(1702,16),(16,73),(73,74),(74,4371),(4371,832),(832,833),(833,77),(82,83),(83,84),(84,51),(52,263),(263,271),(271,1278),(1278,1282),(1285,1286),(1286,1288),(1289,1360),(1360,1362),(1362,1363),(1363,1365),(1365,1365),(1365,1366),(1366,1367),(1367,7455),(7455,7438),(7438,1410),(1410,1411),(1411,1412),(1412,1413),(1413,1414),(1414,1415),(1415,1417),(1417,1418),(1418,5149),(5149,1419),(1419,6094),(1423,1424),(1424,1425),(1425,1426),(1426,1427),(1427,1428),(1428,1429),(1429,1430),(1430,1432),(1432,1433),(1433,1434),(1434,1435),(1435,7439),(7439,7454),(7454,1380),(1380,1381),(1381,1382),(1382,1383),(1383,1384),(1384,1385),(1385,1348),(1359,279),(3,12),(14,53),(53,54),(54,55),(55,56),(60,820),(820,821),(821,63),(63,64),(64,65),(65,66),(66,4498),(4498,1660),(1661,4523),(4523,4524),(4524,4322),(6057,2183),(2185,1423),(1426,1463),(1463,4399),(4399,4400),(4400,1464),(1464,1469),(1469,1465),(1465,1466),(1466,1467),(1467,1468),(1468,1377),(1377,1378),(1378,4514),(4514,4515),(4515,1995),(1359,274),(274,1505),(1505,4509),(4509,499),(499,500),(500,501),(501,1484),(1484,1481),(1481,7320),(7320,1486),(1486,1487),(1487,1488),(1488,1489),(1489,1490),(1491,1492),(1492,1493),(1493,1482),(1482,1494),(1494,1495),(1495,1496),(1496,1515),(1515,1499),(1499,1485),(1485,4518),(4518,509),(509,510),(510,511),(511,513),(513,4513),(4513,272),(272,335),(335,1278),(1278,1934),(1934,1935),(1941,4516),(4516,1369),(1369,1370),(1370,1371),(1371,1454),(1454,1455),(1455,1453),(1453,1456),(1456,1457),(1457,1458),(1458,1459),(1459,1460),(1460,1462),(1462,5149),(6094,2102),(2103,7154),(7154,7153),(7153,6056),(7591,496),(523,1740),(1740,1741),(1741,1742),(1742,1743),(1743,1744),(1744,1745),(1745,1746),(1746,1747),(1747,1748),(1748,1749),(1749,1751),(1751,1752),(1752,1753),(1753,1754),(1754,1759),(1759,1758),(1758,1750),(1750,1769),(1769,1770),(1770,1771),(1771,1760),(1760,1761),(1761,1762),(1762,1763),(1763,1764),(1764,1765),(1765,1767),(1772,1773),(1773,1774),(1774,1775),(1775,1776),(1776,1777),(1777,1778),(1778,1755),(1755,1756),(1756,1757),(1757,1794),(1794,7469),(7469,1724),(1724,1725),(1725,1726),(1726,1727),(1727,1728),(1728,1729),(1729,1730),(1730,1731),(1731,1732),(1732,1733),(1733,1734),(1734,1735),(1735,1736),(1736,1737),(1737,1738),(1738,1739),(1739,614),(620,621),(621,293),(7149,7277),(7277,6363),(6363,6361),(6361,6367),(6367,4985),(4985,6219),(6219,6229),(6229,6364),(6364,1558),(1558,4542),(4542,1531),(1531,1532),(1532,1533),(1533,1534),(1534,1535),(1535,1536),(1536,1537),(1537,186),(186,187),(187,188),(188,189),(189,190),(190,191),(191,819),(819,264),(264,6059),(6059,334),(334,7581),(1076,1021),(1021,6042),(6042,1022),(1022,1023),(1023,1024),(1024,895),(896,897),(897,893),(893,894),(894,1067),(1067,1068),(1068,1069),(7582,319),(319,274),(274,7),(7,194),(194,195),(195,196),(196,197),(197,198),(198,199),(199,1506),(1506,1507),(1507,1508),(1508,1509),(1509,1510),(1510,1511),(1511,1512),(1512,1538),(1538,4543),(4543,1560),(1560,1280),(1280,6366),(6366,6230),(6230,6228),(6228,7111),(7111,6365),(6365,6276),(6276,6362),(6362,7276),(7276,7149),(319,7491),(3605,3606),(3606,3607),(3607,6051),(6051,6053),(6053,7327),(7327,3629),(3629,3630),(3630,3631),(3631,3632),(3632,3633),(3633,3634),(3634,3635),(3635,3636),(3636,905),(905,4331),(4331,4339),(4339,7692),(7692,6010),(6010,3656),(3656,7194),(7194,7196),(7196,6310),(6310,6311),(6311,7399),(7399,7398),(7398,7397),(7397,6252),(6252,301),(2809,2810),(2810,843),(843,1013),(1013,1015),(1015,1016),(895,1025),(1025,1026),(1026,1027),(1027,1028),(1028,2818),(2818,2819),(2819,3012),(3012,877),(877,876),(876,3011),(3011,2896),(2896,2897),(2897,1062),(1062,1063),(1063,1064),(1064,1065),(1065,1066),(1066,1067),(7582,407),(407,2498),(2498,2499),(2499,2500),(2500,2501),(2501,7623),(7623,6312),(6312,7197),(7197,7195),(7195,3653),(3653,6055),(6055,1073),(1073,3641),(3641,3642),(3642,3643),(3643,3583),(3583,3584),(3584,3585),(3585,3625),(3625,3626),(3626,3627),(3627,3628),(3628,6052),(6052,3613),(3613,3615),(3615,3616),(4320,1476),(1479,7622),(7622,334),(334,406),(406,747),(435,7353),(7353,4571),(4571,2013),(2013,2014),(2014,4636),(4636,2016),(2016,2015),(2015,2017),(2017,3259),(3259,7361),(7361,7362),(7362,3129),(3129,3130),(3130,3131),(3131,3132),(3132,3133),(3133,3134),(3134,3135),(3135,3136),(3136,3138),(3138,3139),(3139,4124),(4124,4125),(4125,4126),(4126,4127),(4127,4128),(4128,4129),(4129,4130),(4130,4131),(4131,7294),(7294,4132),(4132,4171),(4171,4172),(4172,4173),(4173,2962),(2962,2963),(2963,2964),(2964,4175),(4175,5091),(5091,5092),(5092,7574),(7574,4177),(4177,4178),(4178,4179),(4179,2993),(2993,2994),(2994,4180),(4180,4181),(4181,4182),(4182,4151),(4151,2997),(2997,4153),(4153,4154),(4154,4416),(4416,4201),(4201,4202),(4202,4203),(4203,5090),(5090,4204),(4204,4205),(4205,4206),(4206,3140),(3140,3141),(3141,3142),(3142,3143),(3143,3144),(3144,3145),(3145,3146),(3146,3147),(3147,3148),(3148,5127),(5127,5128),(5128,2996),(2996,3258),(3258,2060),(2060,2061),(2061,2062),(2062,2063),(2063,2064),(7588,325),(325,1443),(1445,4407),(4407,4320),(4533,4844),(4844,4177),(4495,1279),(1279,1934),(2311,5025),(5025,4854),(4854,1438),(1438,4855),(4855,1439),(1439,1440),(1440,1386),(1386,1387),(1387,1388),(1388,1389),(1389,3355),(3355,3356),(3356,1441),(1441,1442),(1442,7414),(7414,1396),(1396,2317),(2317,2318),(2318,2319),(2319,2320),(2320,2321),(2321,2322),(2322,6163),(6163,1095),(1095,1096),(1096,1097),(1097,1099),(1099,6132),(6132,4861),(4861,4862),(4863,4864),(4864,4865),(4865,4577),(4577,4578),(4578,1100),(1100,100),(100,1111),(1111,2323),(2323,2324),(2324,2325),(2325,2326),(2326,5150),(5150,2327),(2331,1397),(1397,1398),(1398,1399),(1399,5148),(5148,1401),(1401,1402),(1402,1403),(1403,1404),(1404,1405),(1405,4856),(4856,4857),(4857,307),(307,308),(308,309),(309,7412),(7412,7413),(7413,2383),(2383,2003),(2003,1358),(1358,4494),(309,2383),(7514,2279),(2279,6251),(6251,5140),(5140,1500),(2102,2726),(2726,2727),(2727,2105),(2105,6140),(6140,6144),(6144,6145),(6145,6146),(6146,6147),(6147,6149),(6149,6150),(6150,6143),(6143,6011),(6011,2133),(2133,6244),(6244,2134),(2134,2135),(2135,2164),(2164,2165),(4554,7016),(7016,4619),(2166,4554),(4606,7142),(7142,7015),(7015,4617),(3419,4762),(4762,4763),(4763,6245),(6245,2156),(2156,6014),(6014,6242),(6242,6152),(6152,6153),(6153,6243),(6243,6154),(6154,6155),(6155,6141),(6141,6142),(6142,2181),(2181,2182),(2182,2778),(2778,2185),(2383,1479),(7622,297),(297,407),(7623,2508),(2508,2278),(7698,7113),(7113,127),(127,112),(40,146),(146,147),(147,184),(184,185),(185,186),(819,4725),(4725,334),(7294,4156),(4156,4168),(4168,4170),(4170,4153),(792,320),(320,281),(281,6),(6,194),(199,200),(200,201),(201,148),(148,149),(149,150),(150,27),(94,126),(126,6182),(322,7697),(400,7581),(1085,1086),(1086,1087),(1087,1088),(1088,1089),(1089,1090),(1090,1091),(1091,1092),(1092,1093),(1093,1094),(1094,1095),(1096,1101),(1101,1102),(1102,1103),(1103,1104),(1105,1107),(1107,1108),(1108,1109),(1109,1110),(1110,1111),(1111,1112),(1112,1113),(1113,1114),(1114,1115),(1115,2437),(2437,1117),(1117,1118),(1118,1119),(1119,1120),(1120,1164),(7582,340),(2914,2915),(2915,2916),(2916,2917),(2917,2918),(2918,1300),(1303,5129),(5129,5130),(5130,1135),(1135,7064),(7064,7066),(7066,2924),(2924,2925),(2925,7442),(7442,7446),(7446,7070),(7070,7445),(7445,6339),(6339,6324),(6324,6325),(6325,6341),(6341,6277),(6277,6278),(6278,6279),(6279,6280),(6285,6286),(6286,6287),(6287,6288),(6288,6342),(6342,6298),(6298,6336),(6336,7069),(7069,7443),(7443,7444),(7444,2941),(2941,2942),(2942,7065),(7065,7063),(7063,1137),(1137,5131),(5131,5132),(5132,1331),(1334,2985),(2985,2986),(2986,2988),(2988,3001),(2925,2948),(2948,2949),(2949,2950),(2950,2954),(2965,2983),(2983,2984),(2984,2941),(1334,1335),(4407,2637),(2637,2721),(2721,7012),(7012,2201),(4599,4601),(4601,5058),(5058,4602),(4602,4603),(4603,4604),(4604,4605),(4605,4562),(4562,7386),(7386,4607),(4607,4608),(4608,4609),(4609,4610),(4610,4611),(4611,4612),(4612,4613),(4613,3370),(3370,7168),(7168,4616),(4620,7563),(7563,3368),(3368,4626),(4626,4627),(4627,4624),(4624,4625),(4625,4628),(4628,4623),(4623,4619),(4619,4561),(4561,4629),(4629,4630),(4630,4631),(4631,4632),(4632,5164),(5164,4633),(4633,2234),(2242,7435),(7435,2722),(2722,4413),(4413,1476),(4607,7140),(7140,7141),(7141,7138),(7138,3370),(3370,7231),(7231,7128),(7128,7117),(7117,7118),(7118,6289),(6290,7120),(7120,7119),(7119,7127),(7127,7143),(7143,3368),(3368,7137),(7137,7139),(7139,4623),(4599,7184),(7184,2225),(2225,4620),(7563,7231),(7143,7168),(4616,2227),(2227,7049),(7049,2236),(4953,768),(777,779),(779,780),(780,781),(781,782),(782,783),(786,791),(791,7585),(7585,7587),(7587,345),(345,318),(318,312),(2201,4401),(2219,3365),(3365,3366),(3366,3367),(3367,3886),(3886,3888),(3891,7187),(7187,4001),(4001,3949),(3950,3372),(3372,3373),(3373,3374),(3374,2233),(2235,2239),(315,4495),(2810,749),(753,754),(754,755),(755,756),(763,4953),(300,496),(1198,4782),(4782,1782),(1782,1783),(1783,1784),(1784,1785),(1785,1786),(1786,624),(624,625),(625,626),(626,627),(627,628),(628,629),(629,630),(630,4466),(4466,633),(633,634),(634,1005),(1005,1788),(1788,1789),(1789,7199),(956,957),(957,958),(958,959),(959,960),(960,961),(961,962),(962,635),(635,636),(636,637),(637,638),(638,639),(639,640),(640,641),(641,642),(642,643),(643,644),(644,645),(645,1796),(1796,1797),(1797,1798),(1798,1799),(1799,1800),(1800,1801),(1801,1277),(620,7568),(1791,1792),(1792,1793),(1793,962),(300,497),(1198,1222),(1222,1223),(1223,1224),(1224,1225),(1225,1226),(1226,1227),(1227,1228),(1228,1244),(1244,254),(254,1246),(1246,1229),(1229,1231),(1231,1232),(1232,1233),(1233,1234),(1234,7358),(7358,5174),(5174,6078),(6078,7112),(7112,972),(972,1625),(1629,6234),(1244,1246),(1227,1253),(1253,969),(969,1176),(1176,1177),(1177,1178),(1178,1179),(1179,7112),(324,1634),(1638,992),(992,7044),(7044,993),(993,5060),(5060,5061),(5061,1231),(7358,1235),(1235,1251),(1251,253),(253,5033),(5033,7130),(7130,1237),(1237,1238),(1238,1239),(1239,247),(247,1240),(1240,1241),(1241,1242),(1242,1277),(675,303),(1251,7130),(7358,1231),(1231,1235),(1179,5060),(4952,768),(768,2085),(2085,2086),(2086,479),(479,480),(480,481),(481,482),(482,483),(483,485),(485,488),(488,491),(491,493),(495,496),(496,523),(523,4382),(4593,4594),(664,614),(614,4415),(4415,302),(409,411),(411,414),(414,416),(416,417),(417,418),(418,419),(419,420),(420,2088),(2088,2089),(2089,763),(763,4952),(532,649),(649,650),(650,651),(651,4472),(4472,4386),(4386,653),(653,722),(722,723),(723,724),(724,725),(725,726),(726,727),(727,728),(728,729),(729,730),(730,5160),(5160,5155),(5155,4806),(4806,5156),(5156,7552),(7552,924),(924,925),(925,927),(927,938),(950,915),(915,916),(916,917),(917,918),(918,919),(919,7551),(7551,4800),(4800,5157),(5157,5154),(5154,4804),(4804,738),(738,739),(739,740),(740,741),(741,742),(742,743),(743,744),(744,745),(745,600),(600,654),(654,655),(655,656),(656,657),(657,658),(658,659),(659,604),(621,291),(720,680),(680,681),(681,682),(682,683),(683,684),(684,685),(685,686),(686,688),(688,689),(689,690),(690,5062),(5062,691),(691,692),(692,695),(695,696),(696,697),(697,698),(698,699),(699,700),(700,701),(701,702),(702,703),(703,704),(704,705),(705,706),(706,567),(549,677),(677,678),(678,679),(679,680),(692,694),(694,695),(706,707),(721,680),(692,693),(3640,3574),(3574,3641),(3585,3587),(3588,3589),(3589,3605),(3607,3608),(3608,3609),(3609,3598),(928,588),(588,590),(590,1729),(1729,1731),(1731,1738),(1738,614),(614,675),(675,621),(621,4495),(2810,842),(842,846),(846,848),(848,759),(768,770),(770,773),(774,906),(906,909),(909,790),(790,7585),(7587,496),(496,522),(522,1741),(1741,1747),(1747,1749),(1749,544),(544,546),(546,914),(3599,3610),(3610,3611),(3611,6135),(6135,3612),(3612,3613),(3619,6087),(6087,3620),(3620,3621),(3621,3622),(4387,3634),(4339,7616),(7616,5081),(5081,5082),(7623,3719),(3719,3718),(3718,3720),(3720,7691),(7691,3721),(3721,3722),(3722,5069),(5069,3723),(3723,3724),(3724,3725),(3725,3726),(3726,7039),(7039,7204),(7204,3728),(3728,3729),(3729,3730),(3730,3732),(3732,3733),(3733,3734),(3734,3735),(3735,7039),(7204,3736),(3736,3738),(3738,7170),(7170,3740),(3740,3741),(3741,5165),(5165,3743),(3743,3780),(3780,3744),(3744,3745),(3745,3746),(3746,3749),(3750,6252),(6252,302),(302,333),(333,747),(3714,3719),(3725,3738),(5165,7169),(7169,3743),(3744,3715),(3715,3755),(3773,5176),(5176,3774),(3798,6076),(7623,3715),(6050,7398),(7398,6252),(1808,1813),(4743,4745),(1841,1842),(1842,7374),(7374,1845),(1808,1816),(7343,7318),(7318,4980),(2338,1845),(1808,4744),(4989,1845),(324,327),(327,7113),(4725,273),(273,405),(405,408),(415,416),(420,421),(421,422),(422,423),(423,424),(424,425),(425,426),(426,427),(427,428),(428,429),(429,6334),(6334,7660),(7660,3114),(3114,3115),(3115,3116),(3116,6085),(6085,2026),(2026,7226),(7330,2051),(2051,6086),(6086,3161),(3161,3162),(3162,3163),(3163,3164),(3164,3084),(3084,469),(469,470),(470,471),(471,472),(472,473),(473,4705),(4705,475),(475,476),(476,477),(477,478),(478,479),(400,281),(323,332),(4962,4725),(429,3032),(3032,3033),(3033,3034),(3034,3036),(3036,3037),(3037,3038),(3038,3039),(3039,3040),(3040,3041),(3041,3042),(3042,2035),(2035,2036),(2036,2040),(2040,3047),(3047,3202),(3202,3203),(3203,3204),(3204,7641),(7641,7642),(7642,7643),(7643,3214),(3214,3215),(3215,3216),(3216,3217),(3217,3218),(3218,3219),(3219,3220),(3220,5046),(5046,7639),(1174,4725),(7639,7640),(7640,5047),(5047,3224),(3224,3225),(3225,3226),(3226,3227),(3227,3228),(3228,3229),(3229,7644),(7644,7645),(7645,7646),(7646,3238),(3238,3239),(3239,3240),(3240,3068),(3068,3069),(3069,3070),(3070,2041),(2041,3071),(3071,3072),(3072,3073),(3073,3074),(3074,3075),(3075,3076),(3076,3077),(3077,3084),(281,4),(4,4962),(7132,6230),(6228,4788),(4788,6273),(6273,136),(136,137),(137,193),(193,138),(138,167),(167,168),(168,169),(169,141),(141,142),(142,143),(143,115),(263,277),(277,336),(336,1278),(1291,2389),(2389,2390),(2390,2391),(2391,2392),(2392,2393),(2393,2394),(2394,1090),(1090,2413),(2413,2414),(2414,2415),(2415,2416),(2416,2417),(2417,2418),(2418,2419),(2419,2452),(2452,2453),(2453,2454),(2454,2455),(2455,2456),(2456,2445),(2445,2457),(2457,4392),(4392,2458),(2458,2446),(2446,2459),(2459,2460),(2460,2461),(2461,2462),(2462,2463),(2463,2431),(2433,2434),(2434,2435),(2435,2436),(2436,2437),(2437,2438),(2438,2439),(2439,2440),(2440,2441),(2441,2442),(2442,2443),(2443,2444),(2444,1344),(4,194),(91,129),(129,130),(130,131),(131,164),(164,165),(165,166),(166,7458),(7458,4479),(4479,4376),(4376,4377),(4377,4890),(4890,4369),(4369,6219),(6229,7131),(7588,328),(328,1443),(1911,1913),(1913,1914),(1914,1805),(1805,1806),(1814,1852),(1852,1853),(1853,1854),(1854,1844),(1844,6172),(4362,7025),(7025,4464),(4464,1869),(1869,1870),(1870,1871),(1871,1872),(1872,1873),(1873,1874),(1874,1875),(1875,1876),(1876,1877),(1877,1878),(1878,1879),(1879,1899),(1899,1887),(1887,1888),(1888,1889),(1889,1890),(1890,7240),(7240,1891),(1891,7156),(7156,7020),(7020,7029),(7029,7038),(7038,7011),(7011,2171),(2171,7160),(7160,7700),(7700,7047),(7047,7161),(7161,7162),(767,768),(7158,7048),(7048,7699),(7699,7159),(7159,7388),(7388,7017),(7017,7018),(7018,7030),(7030,7021),(7021,7163),(7163,1893),(1893,1894),(1894,1912),(1912,1895),(1895,1896),(1896,1897),(1897,6088),(6088,1898),(1898,1858),(1858,1859),(1859,1860),(1860,4492),(4492,1861),(1861,1862),(1862,1863),(1863,1864),(1864,1865),(1865,1866),(1866,1850),(1850,1867),(1867,4489),(4489,4747),(4903,1883),(1883,1884),(1884,1902),(1902,1903),(1903,1904),(1904,1840),(1905,1906),(1906,1907),(1907,1908),(1908,1909),(1909,1713),(315,335),(335,404),(753,7339),(7132,1558),(1558,2957),(2957,2958),(2958,4938),(4938,1549),(1549,1523),(1523,1524),(1524,1525),(1525,1526),(1526,6080),(6080,1585),(1585,1586),(1586,1587),(1587,4539),(4539,1606),(1606,1607),(1607,1608),(1608,1609),(1609,1610),(1610,7471),(7471,1533),(186,838),(838,839),(839,840),(840,841),(841,49),(1947,2642),(2642,2643),(2643,2644),(2644,2709),(2709,2710),(2710,2711),(2711,2712),(2712,2655),(2655,2656),(2656,2668),(2668,2672),(2672,2673),(2673,2206),(2206,4798),(4798,2674),(2674,2675),(2675,2124),(2124,2125),(2125,2126),(2126,2127),(2127,2128),(2128,2129),(2129,4797),(4797,7473),(7473,2679),(2679,4691),(4691,2682),(2682,4796),(4796,4690),(4690,4411),(4411,4715),(1587,1591),(1591,4542),(190,192),(192,819),(819,261),(4795,4688),(4688,4689),(4689,4410),(4410,2678),(2678,2681),(2681,2680),(2680,2676),(2676,2118),(2118,2119),(2119,2120),(2120,2121),(2121,2123),(2123,2685),(2685,2686),(7510,4799),(4799,2205),(2205,2687),(2687,2688),(2688,2689),(2689,2696),(2696,2697),(2697,2713),(2713,2714),(2714,2715),(2715,2716),(2716,2718),(2718,2719),(2719,1989),(1358,319),(319,279),(279,7),(15,834),(834,835),(835,836),(836,837),(837,1506),(1511,1592),(1592,1593),(1593,1594),(1594,1595),(1595,1596),(1596,4363),(4363,1598),(1598,1597),(1597,1599),(1599,1573),(1573,1574),(1574,1575),(1575,4583),(4583,1518),(1518,1519),(1519,1520),(1520,7661),(7661,1521),(1521,4546),(4546,1559),(1559,987),(987,4937),(4937,1547),(1547,1548),(1548,1280),(1280,6348),(288,1171),(1630,7348),(7348,3669),(3669,3671),(3676,5073),(5073,5074),(5074,6054),(6054,4330),(4330,3679),(3679,5075),(5075,5076),(5076,3682),(3682,3864),(3864,3865),(3865,4910),(4910,4911),(4911,4912),(4912,4913),(4913,4914),(4914,4915),(4915,4957),(4843,4905),(4905,4906),(4906,4907),(4907,4908),(4908,4909),(4909,3884),(3884,3686),(3686,5077),(5077,5078),(5078,3689),(3689,3690),(3690,6117),(6117,5079),(5079,3694),(3885,3663),(3663,7347),(277,288),(7144,3881),(3881,5096),(5096,7145),(7145,4843),(1184,497),(6115,1205),(1205,1206),(1206,1207),(1207,1208),(1208,1209),(1209,1210),(1210,1211),(1211,3575),(3575,3576),(3576,3577),(3577,3578),(3578,3579),(3579,3580),(3580,3640),(5082,3639),(3639,3644),(3644,3645),(3645,3646),(3646,3647),(3647,3661),(3661,3573),(3573,3662),(3662,1212),(1212,1213),(1213,1214),(1214,1215),(1215,1216),(1216,1217),(1217,4563),(675,1184),(1210,3575),(3576,3648),(3648,3649),(3649,3650),(3650,7328),(7328,3651),(3651,3652),(3652,3653),(3653,3654),(3654,5084),(5084,6054),(3712,6359),(6359,7189),(5044,4433),(4433,3752),(6117,3655),(3655,5086),(5086,3656),(3656,3657),(3657,3658),(3658,7329),(7329,3659),(3659,3660),(3660,3647),(4923,4924),(4924,4958),(4958,3752),(3658,3659),(7571,207),(207,208),(350,2809),(2810,2811),(2811,843),(1015,2812),(2812,2813),(2813,2814),(2814,852),(855,2815),(2815,2816),(2816,2817),(2817,2818),(2819,2820),(2820,2822),(2822,2823),(2823,5032),(5032,2824),(2824,2825),(2825,2826),(2826,2827),(2827,2828),(2828,2829),(2832,2833),(2833,7375),(7375,3466),(3466,3467),(3467,3469),(3469,3470),(3470,3471),(3471,3472),(3472,3473),(3473,3474),(3474,3475),(3475,3476),(3476,3477),(3477,3478),(3478,3280),(3280,3282),(3282,3479),(3479,4086),(4086,4087),(4087,4088),(4088,4089),(4089,4090),(4090,4091),(4091,6097),(6097,4093),(4093,4094),(4094,4096),(4096,4114),(4114,4115),(4115,4116),(4116,4117),(4117,4118),(4118,4119),(4119,4120),(4120,4121),(4121,4122),(4122,4123),(4123,3283),(3487,3488),(3488,3489),(3489,3492),(3492,2835),(2835,7176),(7176,2836),(2836,2837),(2840,2841),(2841,2865),(2866,2892),(2892,2893),(2893,2894),(2894,2895),(2895,2896),(2897,2898),(2898,3562),(3562,2899),(2899,884),(887,2900),(2900,2901),(2901,2902),(2902,2903),(2903,1074),(1074,6074),(6074,793),(793,495),(205,206),(206,207),(210,1643),(1643,7571),(7564,340),(380,2087),(2087,7053),(7053,2088),(7057,5142),(5142,7567),(7567,7326),(7326,7418),(7418,7417),(7417,3471),(3473,5015),(5015,7290),(7290,5016),(2008,2071),(2071,2072),(2072,2073),(2073,2074),(2074,2075),(2075,2076),(2076,460),(460,2011),(5013,5014),(5014,7291),(7291,3485),(7325,4984),(4984,448),(448,3181),(3321,2066),(2066,2067),(2067,4728),(2086,4455),(4455,381),(400,7564),(7564,7663),(4728,437),(437,459),(459,2080),(2080,2081),(2081,2082),(2082,2083),(2083,2084),(342,4521),(4521,1934),(1934,2385),(2385,2386),(2386,2387),(2387,2388),(2388,1290),(1298,1121),(1140,2515),(2515,2516),(2516,2517),(2517,2518),(2518,4749),(4749,4750),(4750,4751),(4751,4754),(4754,4755),(4755,3003),(3003,2522),(2522,2524),(2524,2523),(2523,2532),(2532,4877),(4877,4878),(4878,7180),(7180,7123),(7123,4347),(4347,7181),(4342,4348),(4348,7122),(7122,7121),(7121,7183),(7183,4880),(4880,2539),(2539,2540),(2540,2542),(2542,3004),(3004,4756),(4756,3005),(3005,4757),(4757,4758),(4758,6343),(6343,4759),(4759,4760),(4760,2547),(2547,2548),(2548,946),(1163,1337),(1347,2634),(2634,2635),(2635,2636),(2636,5097),(5097,2383),(1358,5192),(5192,342),(2353,2357),(2357,2359),(2359,7122),(4380,496),(4384,518),(518,2255),(2255,2256),(2256,2717),(2717,2257),(2257,2258),(2258,2259),(2259,2260),(2260,2261),(2261,4469),(4469,2262),(2262,2264),(2264,2265),(2265,2269),(2269,2270),(4380,1172),(1172,7671),(7671,513),(4513,286),(286,271),(271,496),(2270,2274),(2274,7601),(7601,2508),(2278,2279),(2279,2720),(2720,2280),(2280,2281),(2281,2282),(2282,4468),(4468,2283),(2283,616),(675,4380),(302,340),(5032,2867),(2867,1035),(1035,1314),(1314,1316),(1316,1315),(1315,1317),(1317,1318),(1318,1319),(1319,1281),(1281,1329),(1330,2919),(2919,2920),(2920,2921),(2921,2922),(2922,2923),(2923,2924),(2954,7067),(7067,2955),(2955,2984),(2984,2927),(2927,2929),(2929,2930),(2930,7441),(7441,2933),(2984,7449),(7449,2941),(2942,2943),(2943,2944),(2944,2945),(2945,2946),(2946,2947),(2947,1305),(1305,1306),(1306,1307),(1307,1308),(1308,1309),(1309,1310),(1310,1311),(1311,1312),(1312,1313),(1313,1055),(1055,2868),(2868,2892),(400,303),(2934,2935),(2935,2936),(2936,2937),(2937,2938),(2938,7440),(7440,2940),(2940,2948),(7564,4521),(4521,1283),(1283,4456),(1130,2550),(2550,2551),(2551,2553),(2553,2554),(2554,2555),(2555,2556),(2556,2557),(2351,4445),(4445,2352),(2352,2573),(2573,2574),(2574,2577),(2577,2578),(2578,2579),(2579,7395),(7395,6124),(6124,4004),(4004,4005),(4005,4006),(4006,4007),(4007,4008),(4008,4010),(4010,4009),(4009,4011),(4011,5110),(5110,4012),(4012,4013),(4013,4014),(4014,4098),(4098,4015),(4015,4017),(4017,4018),(4018,4019),(4019,4020),(4020,4100),(4100,4021),(4021,4022),(4022,4023),(4023,4024),(4024,4027),(4027,7250),(7250,7248),(7248,7207),(7207,7288),(7288,7286),(7286,7283),(7283,7282),(7282,7278),(7278,7289),(7250,7252),(7252,7248),(7248,7254),(7254,7257),(7257,7258),(7258,7260),(7260,7263),(7263,7264),(7264,7268),(7268,7270),(5111,4051),(4051,4052),(4052,4054),(4054,4055),(4055,4057),(4057,4101),(4101,4102),(4102,4058),(4058,4059),(4059,4060),(4060,4061),(4061,4062),(4062,4063),(4063,4064),(4064,4056),(4056,4133),(4133,4065),(4065,4066),(4066,4067),(4067,4068),(4068,4069),(4069,4070),(4070,4071),(4071,4072),(4072,4073),(4073,2583),(2583,2584),(2584,2585),(2585,2586),(2586,2587),(2587,2588),(2588,2589),(2589,2590),(2590,2591),(2591,2592),(2592,2357),(2357,2358),(2358,2359),(4435,2594),(2594,2595),(2595,2596),(2596,2597),(2597,2599),(2599,2600),(2600,1155),(1072,1352),(1357,1358),(1358,7564),(7289,7280),(7280,7281),(7281,7284),(7284,7287),(7287,7208),(7208,7249),(7249,7253),(7253,7251),(7251,4051),(7270,7266),(7266,7265),(7265,7262),(7262,7261),(7261,7259),(7259,7256),(7256,7255),(7255,7249),(3888,7186),(7186,7187),(7187,3954),(3954,3955),(3955,3956),(3956,3957),(3957,3958),(3958,3959),(3959,3960),(3960,3961),(3961,3962),(3962,3963),(3963,3964),(3964,3965),(3965,3968),(3968,7771),(7771,3966),(3966,5113),(5113,3918),(3918,5114),(3980,6089),(6089,3981),(3981,3982),(3982,7704),(7704,3983),(3983,3985),(3985,3986),(3986,3987),(3987,3989),(3989,3990),(3990,3991),(3991,3998),(3998,3999),(3999,4000),(4000,4001),(3986,7313),(7313,6164),(6164,6159),(6159,6160),(6160,6161),(6161,3991),(7187,3947),(3947,3892),(3892,3893),(3893,3895),(3895,3896),(3896,3897),(3897,3898),(3898,3899),(3899,3900),(3900,3901),(3901,3902),(3902,3904),(3904,3905),(3905,3906),(3906,3907),(3907,3908),(3908,3909),(3909,5118),(5118,3910),(3910,3911),(3911,3912),(3912,3913),(3913,3914),(3914,7075),(7075,3925),(3925,3008),(3008,6089),(3917,3918),(3918,3926),(3926,7074),(7074,3919),(3919,3920),(3920,3921),(3921,3922),(3922,3923),(3923,3112),(3112,3924),(3924,3927),(3927,3928),(3928,3929),(3929,7108),(7108,3935),(3935,3936),(3936,7110),(7110,3938),(3938,3939),(3939,3940),(3940,3941),(3941,3943),(3943,3944),(3944,3945),(3945,3946),(3946,4001),(2810,7385),(4495,1278),(1367,1369),(1371,1372),(1372,1373),(1373,1374),(1374,1946),(4379,1968),(1968,1969),(1969,1970),(1970,1971),(1971,2114),(2114,2115),(2115,6011),(2134,3378),(3378,3379),(3379,4669),(4669,2165),(2166,2167),(2167,3383),(3383,3384),(3384,3385),(3385,3386),(3386,3387),(3387,4559),(4559,3389),(3389,3390),(3390,6119),(6119,3392),(3392,3393),(3393,3394),(3394,3396),(3396,3397),(3397,3398),(3398,3399),(3399,6187),(6187,6188),(6188,6215),(6215,6184),(3400,3401),(3401,3402),(3402,3403),(3403,3404),(3404,3405),(3405,3406),(3406,3410),(3410,3407),(3407,3408),(3408,3409),(3409,4558),(4558,3411),(3411,3412),(3412,3413),(3413,3414),(3414,3415),(3415,3416),(3416,3417),(3417,3418),(3419,3420),(3420,3421),(3421,3422),(3422,3423),(3423,2156),(6014,2158),(2158,2159),(4409,1972),(1972,1973),(1973,1974),(1974,1975),(1989,1375),(1375,1376),(1376,1377),(1378,1379),(1379,1380),(1358,7665),(1989,1472),(6185,6186),(6186,6216),(6216,3400),(4409,2173),(2173,2174),(2174,3425),(3425,2176),(2176,2177),(2177,2178),(2178,7354),(7354,1975),(4495,4720),(4720,1443),(1451,1452),(1452,2640),(2640,2641),(2641,1948),(1971,2160),(4667,2162),(2162,2163),(2163,2168),(2168,4536),(4536,6003),(6003,5123),(5123,5124),(5124,5125),(5125,6001),(6001,4438),(4438,3432),(3432,3426),(3426,3434),(3434,3435),(3435,3453),(3453,3437),(3437,3439),(3439,3440),(3440,4557),(4557,3441),(3441,4560),(4560,5126),(4564,3443),(3443,3444),(3444,3445),(3445,3446),(3446,3447),(3447,3448),(3448,3449),(3449,3436),(3436,7432),(7432,3454),(3454,3455),(3455,3456),(3456,3457),(3457,3438),(3438,5022),(5022,6002),(6002,5119),(5119,5120),(5120,5121),(5121,5122),(5122,3461),(3461,4532),(4532,4534),(4534,4535),(4535,2169),(2169,2170),(2170,4672),(1981,4406),(1988,2642),(2642,7435),(2722,1471),(1471,1472),(7453,1480),(1480,2912),(2912,1358),(1808,7389),(7389,4324),(4324,4769),(4769,4770),(4770,4765),(4765,4766),(4766,4767),(4767,4768),(4768,4325),(4325,3328),(3328,3329),(3329,3330),(3330,3331),(3331,3351),(3351,3332),(3332,3333),(3333,3334),(3334,3352),(3352,3335),(3335,3336),(3336,3337),(3337,3338),(3338,3339),(3339,3340),(3340,4324),(4768,1845),(326,1443),(2637,2638),(2638,2640),(2641,2643),(2644,2645),(2645,2646),(2646,2647),(2647,2648),(2648,2649),(2649,2650),(2650,2652),(2652,2653),(2653,2654),(2654,2655),(2656,2657),(2657,2658),(2658,2659),(2659,2660),(2660,2661),(2661,2662),(2662,2909),(2909,2906),(2906,2907),(2907,2908),(2670,2910),(2910,2911),(2911,7507),(7507,2663),(2663,2664),(2664,2665),(2665,2666),(2666,2667),(2667,2696),(2697,2700),(2700,2701),(2701,2702),(2702,2703),(2703,2704),(2704,2706),(2706,6070),(6070,2708),(2708,5134),(5134,2718),(2719,7435),(1358,326),(324,1555),(1555,1556),(1556,1557),(1557,1181),(1181,7350),(7350,6230),(6273,4891),(4891,7458),(7458,7425),(7425,7424),(7424,7423),(7423,1566),(1566,1567),(1567,4372),(4372,179),(179,7431),(7431,175),(175,176),(176,181),(181,182),(182,183),(183,184),(190,1614),(1614,1615),(1615,1478),(315,336),(1279,1283),(1077,2479),(2479,2480),(2480,2481),(2481,2482),(2482,2483),(2483,2484),(2484,2485),(2485,2486),(2486,2487),(2487,2488),(2488,2489),(2489,2448),(2448,5026),(5026,2450),(2450,2490),(2490,2491),(2491,2492),(2492,2493),(2493,2465),(2465,2466),(2466,2467),(2467,2494),(2494,2495),(2495,2496),(2496,2497),(2497,2471),(2471,2472),(2472,2473),(2473,2474),(2474,2475),(2475,2476),(2476,2477),(2477,2478),(2478,1170),(319,1443),(1443,1616),(1616,1617),(1617,1618),(1618,1619),(1619,195),(201,202),(202,151),(151,153),(153,154),(154,155),(155,156),(156,7430),(7430,177),(177,178),(178,1539),(1539,1540),(1540,7426),(7426,7427),(7427,7428),(7428,138),(138,133),(133,134),(134,4890),(6229,6348),(6348,1182),(1182,1550),(1550,1551),(1551,1552),(1552,332),(7188,3082),(3082,7660),(3116,3117),(3117,3118),(3118,3119),(3119,3120),(3120,3122),(3122,3123),(3123,3124),(3124,3125),(3125,3126),(3126,3127),(3127,3128),(3128,3130),(3133,5046),(7640,3134),(4170,4132),(4132,4134),(4134,4135),(4135,4136),(4136,4207),(4207,4208),(4208,4209),(4209,4210),(4210,4211),(4211,4212),(4212,4213),(4213,5139),(5139,4214),(4214,4215),(4215,4216),(4216,4217),(4217,4218),(4218,4219),(4219,4220),(4220,4221),(4221,4222),(4222,4224),(4224,4225),(4225,4226),(4226,4227),(4227,4228),(4228,4229),(4229,4230),(4230,4231),(4231,7352),(7352,4246),(4246,4248),(4248,4249),(4249,4250),(4250,4251),(4251,4252),(4252,4253),(4253,4254),(4254,6137),(6137,4256),(4256,4257),(4257,4258),(4258,4259),(4259,7272),(7272,4956),(4253,4305),(4305,4306),(4306,4309),(4309,4254),(4260,4261),(4261,7271),(7271,5136),(5136,4262),(4262,4263),(4263,4264),(4264,7296),(7296,4265),(4265,4266),(4266,4267),(4267,4268),(4268,4269),(4269,4270),(4270,4271),(4271,4273),(4273,4274),(4274,4275),(4275,4276),(4276,4277),(4277,4278),(4278,4279),(4279,4280),(4280,4281),(4281,4282),(4282,4283),(4283,4284),(4284,4285),(4285,4286),(4286,4287),(4287,4288),(4288,4295),(4295,4296),(4296,4297),(4297,4299),(4299,4300),(4300,4301),(4301,4302),(4302,5135),(5135,4303),(4303,4304),(4304,4148),(4148,4149),(4149,3045),(3045,4150),(4150,4151),(4151,4156),(4203,4204),(3144,5047),(5047,3146),(3148,3149),(3149,3150),(3150,3151),(3151,3153),(3153,3152),(3152,2995),(2995,3154),(3154,3155),(3155,3156),(3156,3158),(3158,3159),(3159,3160),(3160,3161),(3164,3079),(3079,3080),(3080,3081),(3081,7188),(4266,4307),(4307,4308),(4308,4267),(4319,1476),(7453,2001),(1359,4724),(4724,263),(263,4508),(4508,7402),(7402,4717),(4717,2499),(2499,7216),(7216,2501),(7623,7401),(7401,3665),(3665,7400),(7400,3669),(3669,7399),(7399,7611),(7611,7398),(7397,5140),(5140,620),(620,1172),(1172,286),(271,4521),(1935,1445),(1445,4319),(6074,748),(749,494),(495,7588),(7588,4717),(3669,7611),(2809,7612),(7612,2811),(2811,842),(842,1014),(1014,1015),(1015,1074),(7389,7025),(1899,6107),(6107,6108),(6108,6109),(6109,6110),(6110,7020),(7021,6111),(6111,6112),(6112,6113),(6113,1898),(4747,1845),(763,767),(783,785),(7587,328),(1443,1647),(1647,1649),(1911,1805),(1805,1662),(1662,1666),(1666,1808),(1808,7475),(7475,6107),(6107,6109),(6110,7029),(7011,7160),(7159,7017),(7030,6111),(6113,4323),(4323,1845),(1845,1696),(1696,1700),(1700,1906),(1906,1909),(1713,1715),(1715,1478),(1478,315),(749,751),(752,7339),(4592,980),(980,981),(981,1518),(1559,1560),(1552,1553),(1553,7023),(7023,7014),(7014,4315),(4315,4316),(6004,7),(1538,4548),(4548,980),(4316,4317),(4317,7013),(7013,6162),(6162,7022),(7022,1554),(1554,1555),(7350,1558),(1558,1549),(6080,983),(983,984),(984,985),(985,1529),(1529,4542),(52,4725),(4725,6071),(4548,1514),(1514,980),(981,1576),(1576,1577),(1577,1578),(1578,4364),(4364,1579),(1579,1580),(1580,7237),(7237,7241),(7241,7247),(7247,7243),(7243,7126),(7318,4978),(4978,4979),(4979,4986),(4986,7107),(7107,7102),(7102,7103),(7103,7104),(7104,7105),(7105,7106),(7106,6005),(6005,1546),(1546,7072),(7343,4978),(1546,7560),(7073,7097),(7097,7098),(7098,7099),(7099,7100),(7100,7101),(7101,7505),(7505,5071),(5071,5072),(5072,4321),(4321,7342),(7344,2510),(2510,7242),(7242,7238),(7238,7224),(7224,1581),(1581,1582),(1582,4793),(4793,6068),(6068,1584),(1584,983),(5072,7342),(7559,7073),(7672,826),(5144,7702),(7702,7009),(7009,1604),(1604,1573),(4583,1576),(1579,7673),(7673,7674),(7674,7675),(7675,7676),(7676,7677),(7677,7559),(7559,7107),(6005,7072),(7505,7678),(7678,7560),(7560,7679),(7679,7680),(7680,7681),(7681,7738),(7738,7683),(7683,1582),(1584,1585),(1586,1600),(1600,7701),(7701,7010),(7010,7703),(7703,5147),(69,7686),(3865,7133),(7133,7134),(7134,3867),(3867,7135),(7135,7136),(7136,3874),(7348,3671),(4957,7133),(7145,3884),(3690,5079),(3675,5030),(5030,5031),(5031,6033),(6033,3089),(3089,3090),(3090,3706),(3706,3707),(3707,3708),(3708,3709),(3709,3710),(3710,4925),(4925,4926),(4926,3678),(3712,4916),(4916,4917),(4917,4918),(4918,4919),(4919,4920),(4920,4910),(4909,4921),(4921,4922),(4922,4923),(3691,3572),(3572,3701),(3701,3702),(3702,3088),(3088,3704),(3704,3705),(3705,733),(733,6015),(6015,5029),(5029,5028),(5028,3696),(3711,7189),(5044,3751),(7587,407),(7623,3676),(5073,3679),(4915,7654),(7623,5030),(4925,3679),(7654,4843),(5077,3694),(3695,7115),(7115,6310),(6311,4495),(760,762),(3751,7114),(7114,7116),(7116,3694),(5028,7115),(2833,6006),(6006,3493),(3493,3494),(3494,3495),(3495,3496),(3496,3497),(3497,3498),(3498,3499),(3499,3500),(3500,3501),(3501,3502),(3502,3503),(3503,3504),(3504,3505),(3505,3506),(3506,3507),(3507,3508),(3508,3510),(3510,3512),(3512,3514),(3514,3515),(3515,3516),(3516,3517),(3517,3518),(3518,3519),(3519,3521),(3521,3523),(3523,3524),(3524,3525),(3525,3527),(3527,3528),(3528,3529),(3529,3532),(3532,3533),(3533,3491),(3491,6007),(6007,2836),(2866,2825),(807,808),(808,809),(809,810),(810,811),(811,812),(812,813),(813,814),(814,81),(81,817),(2017,2018),(2018,2019),(2019,2020),(2020,2021),(2021,2022),(2022,4565),(4565,4566),(4566,4567),(4567,2031),(2031,2032),(2032,2033),(2033,2034),(2034,2035),(2036,2039),(4962,264),(2039,2040),(2040,2041),(2041,2042),(2042,2043),(2043,2044),(2044,2045),(2045,2046),(2046,4568),(4568,4569),(4569,4570),(4570,2056),(2056,2057),(2057,2058),(2058,2059),(2059,2060),(278,2),(796,797),(797,798),(798,799),(799,800),(800,801),(801,802),(802,803),(803,804),(804,7513),(7513,807),(7491,278),(2,4962),(3085,3034),(3034,3114),(3114,4353),(4353,4355),(4355,4354),(4354,3196),(3196,3345),(3345,4727),(2,4726),(4726,7555),(7555,4962),(7392,1443),(2638,2721),(2213,4688),(2680,4677),(4670,3420),(3422,4671),(2172,4409),(2160,4665),(4668,3376),(3376,3377),(3377,3378),(4669,2146),(6245,6013),(6013,6247),(6247,2679),(4691,4796),(4411,2239),(2811,749),(7433,4677),(2137,2138),(3423,4409),(1975,1982),(1982,2642),(7435,4413),(4413,7453),(342,1279),(2394,2395),(2395,2396),(2396,2397),(2397,2398),(2398,2399),(2399,2400),(2400,2401),(2401,2402),(2402,2403),(2403,1096),(6132,2551),(4640,7214),(7214,4347),(2351,2558),(2558,6170),(6170,5023),(5023,5024),(5161,5162),(5162,2571),(2571,2359),(4647,4435),(2599,4577),(1111,2404),(2404,2405),(2405,2406),(2406,2408),(2408,2409),(2409,2410),(2410,2411),(2411,2412),(2412,2438),(2636,2383),(1358,342),(2105,2728),(2728,2729),(2729,2730),(2730,6083),(6083,2733),(2733,2734),(2734,5169),(5169,2735),(2735,2736),(2736,2725),(2725,2737),(2737,2738),(2738,2739),(2739,7177),(7177,2740),(2740,2741),(2741,2742),(2742,2743),(2743,2745),(2745,2746),(2746,2747),(2747,2748),(2748,7062),(2360,4340),(4344,2349),(2622,2750),(2750,2751),(2751,2752),(2752,2753),(2753,2754),(2754,2755),(2755,2756),(2756,2764),(2764,7178),(7178,2765),(2765,2766),(2766,2767),(2767,2772),(2772,2768),(2768,2769),(2769,2770),(2770,2771),(2771,2773),(2773,2774),(2774,2777),(2777,2775),(2775,2181),(1358,340),(2550,2516),(4750,3006),(3006,4871),(4871,4872),(4872,4873),(4873,4874),(4874,2532),(7180,7331),(7331,2559),(2559,2560),(2560,2561),(2561,2562),(2562,2564),(2564,4927),(4927,4928),(4928,2352),(2574,2575),(2575,4929),(4929,7460),(7460,4930),(4930,4931),(4931,7459),(4713,4714),(4714,4960),(4960,4932),(4932,4933),(4933,2590),(2357,2566),(2566,2567),(2567,2568),(2568,2569),(2569,7550),(7550,2570),(2570,7222),(7222,6170),(6170,7183),(2540,4882),(4882,4883),(4883,4884),(4884,4885),(4885,6343),(2548,2600),(3957,3993),(3993,3994),(3994,3995),(3995,3996),(3996,3997),(3997,3998),(3959,7234),(7234,4002),(4002,5112),(5112,4003),(4003,4404),(4404,4405),(4405,3991),(3960,6156),(6156,6165),(6165,6157),(6157,6158),(6158,7314),(7314,3964),(3968,3966),(3982,3983),(1445,1447),(1447,2637),(2215,7184),(7184,7050),(7050,7185),(7185,7186),(6158,3964),(4001,7193),(7193,7049),(2236,2239),(3996,3998),(3905,7085),(7085,7086),(7086,7087),(7087,7088),(7088,7089),(7089,7091),(7091,3910),(3914,3925),(3907,3909),(3923,7079),(7079,7080),(7080,7081),(7081,7565),(7565,7082),(7082,7083),(7083,7084),(7084,7108),(3936,3938),(1374,1375),(6155,1987),(1987,1989),(1989,1995),(1995,1998),(1998,2002),(2637,1956),(1956,2109),(2109,3427),(3427,3428),(3428,3429),(3429,3432),(3432,3434),(3434,3453),(3437,3440),(3440,3441),(4564,3444),(3444,3447),(3447,3454),(3455,5022),(5022,3461),(3461,4537),(4537,2176),(2176,1982),(1982,7435),(4413,1480),(1480,1358),(2339,2420),(2420,2421),(2421,2423),(2423,2424),(2424,2447),(2447,5133),(5133,2596),(2596,2611),(2611,2602),(2602,2613),(2613,2614),(2614,2615),(2615,2535),(2535,2536),(2536,2616),(2616,2617),(2617,2557),(2558,2559),(2570,2571),(4435,2604),(2604,2605),(2605,2538),(2538,2606),(2606,2607),(2607,2608),(2608,2609),(2609,2610),(2610,2554),(2554,2425),(2425,2426),(2426,2427),(2427,2428),(2428,2429),(2429,2430),(2430,2371),(2569,2570),(5024,5161),(5162,4880),(4880,2606),(2373,1424),(1424,2190),(5192,350),(2810,750),(7012,2644),(2909,6020),(6020,6023),(6023,6026),(6026,6027),(6027,6029),(6030,6028),(6028,6025),(6025,6024),(6024,6021),(6021,7507),(2718,7435),(3041,2035),(3204,3205),(3205,3206),(3206,3207),(3207,3208),(3208,3209),(3209,3210),(3210,4981),(4981,3211),(3211,3212),(3212,3213),(3213,3214),(3220,3221),(3221,4982),(3222,3223),(3223,3224),(3229,7472),(7472,3230),(3230,3232),(3232,3233),(3233,3234),(3234,4462),(4462,3235),(3235,3236),(3236,3237),(3237,3238),(3077,3079),(3081,3084),(4962,6059),(7353,7584),(7584,3344),(3344,7421),(7421,7422),(7422,6085),(6085,2053),(2053,2054),(2054,2055),(2055,4732),(4732,3243),(3243,3244),(3244,3245),(3245,3212),(3219,3537),(3537,3538),(3538,7556),(7556,3539),(3539,3540),(3540,3541),(3541,3542),(3542,3544),(3544,3552),(3552,3553),(3553,3554),(3554,3555),(3555,3556),(3556,3557),(3557,7317),(7317,3224),(3230,3231),(3231,3246),(3246,3247),(3247,7056),(7056,4731),(4731,2023),(2023,2024),(2024,2025),(2025,6086),(6086,7419),(7419,7420),(7420,3345),(345,320),(4725,6059),(2055,4565),(4567,3238),(3240,3048),(3048,3049),(3049,3050),(3050,3051),(3051,3052),(3052,3053),(3053,3054),(3054,3055),(3055,3056),(3056,3057),(3057,4830),(4830,7667),(7667,3091),(3091,3092),(3092,3093),(3093,3535),(3535,3536),(3536,3537),(3542,3543),(3543,3141),(3142,3224),(478,7053),(2089,770),(772,417),(417,485),(495,7587),(3057,3059),(3059,3060),(3060,3061),(3061,3062),(3062,3063),(3063,3064),(3064,3065),(3065,3066),(3066,3067),(3067,3202),(3204,4568),(4569,2023),(4,7555),(1567,173),(173,174),(174,175),(156,157),(157,158),(158,159),(159,1539),(3082,3114),(3144,7639),(3164,7188),(3080,3082),(3082,3084),(4495,746),(746,844),(844,846),(848,758),(758,760),(760,2007),(2007,4571),(4571,2014),(2014,2016),(2016,2017),(2017,7362),(7362,3134),(3135,7363),(7363,4588),(4588,4589),(4589,7364),(7364,7365),(7365,4210),(4211,4215),(4216,4222),(4225,7462),(7462,7461),(7461,4230),(4230,4248),(4248,4252),(4252,6137),(6137,4956),(342,4495),(4252,4305),(4306,4307),(4308,4253),(4253,4268),(4268,4273),(4273,4277),(4277,7462),(7461,4281),(4284,4295),(4296,4301),(4302,4591),(4591,4590),(4590,7366),(7366,7367),(7367,7368),(7368,3143),(3145,5128),(5128,2060),(2060,2062),(2062,2064),(2064,4727),(4727,768),(768,772),(772,775),(775,906),(909,792),(4260,7296),(7296,4266),(4266,4253)]

### 8.3.1 Continuous Feature Correlations with time_diff by Segment

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    x = df_temp['actualtime_arr_stop_first']
    y = df_temp['time_diff']
    plt.scatter(x,y,c='purple')
    plt.title(str(segment))
    plt.xlabel("actualtime_arr_stop_first")
    plt.ylabel("time_diff")
    plt.show()

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    x = df_temp['rain']
    y = df_temp['time_diff']
    plt.scatter(x,y,c='purple')
    plt.title(str(segment))
    plt.xlabel("rain")
    plt.ylabel("time_diff")
    plt.show()

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    x = df_temp['temp']
    y = df_temp['time_diff']
    plt.scatter(x,y,c='purple')
    plt.title(str(segment))
    plt.xlabel("temp")
    plt.ylabel("time_diff")
    plt.show()

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    x = df_temp['rhum']
    y = df_temp['time_diff']
    plt.scatter(x,y,c='purple')
    plt.title(str(segment))
    plt.xlabel("rhum")
    plt.ylabel("time_diff")
    plt.show()

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    x = df_temp['msl']
    y = df_temp['time_diff']
    plt.scatter(x,y,c='purple')
    plt.title(str(segment))
    plt.xlabel("msl")
    plt.ylabel("time_diff")
    plt.show()

### 8.3.2 Categorical Feature Correlations with time_diff by Segment

I am leaving out some categorical features with high cardinality e.g. dayofservice.

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    df_temp.boxplot(column=['time_diff'], by=['lineid'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    df_temp.boxplot(column=['time_diff'], by=['routeid'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    df_temp.boxplot(column=['time_diff'], by=['direction'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    df_temp.boxplot(column=['time_diff'], by=['month'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    df_temp.boxplot(column=['time_diff'], by=['day'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    df_temp.boxplot(column=['time_diff'], by=['hour'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    df_temp.boxplot(column=['time_diff'], by=['day_of_week'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    df_temp.boxplot(column=['time_diff'], by=['weekday'], figsize=(8,5))

In [ ]:
for segment in segments:
    df_temp = df.loc[df['segment'] == (str(segment[0]) + ',' + str(segment[1]))]
    df_temp.boxplot(column=['time_diff'], by=['bank_holiday'], figsize=(8,5))

# 15. Data Quality Plan

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| stoppointid | There are many rows where the value for this feature does not appear in the GTFS data i.e. the stops are no longer used by Dublin Bus | Drop affected rows|
| time_diff | There are many rows where this feature is zero or negative | Drop affected rows|
| time_diff | Outliers | Group by segment and then remove any rows where the time_diff is more than 3 times the standard deviation |